In [ ]:
# XGBoost modeling: active fractions and soil organic carbon in topsoil
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import joblib
import os
import re
from pathlib import Path
import warnings
import builtins
from scipy import stats
open = builtins.open

warnings.filterwarnings('ignore')

# SHAP availability check
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False

class XGBoostTuner:
    def __init__(self, seed=42, n_jobs=-1):
        self.seed = seed
        self.n_jobs = n_jobs if n_jobs != -1 else None
        self.study = None
        self.best_model = None
        self.best_params = None
        self.feature_names = None  # Store feature names

    @staticmethod
    def clean_numeric_value(value):
        """Convert string values to numeric, handling special cases"""
        if pd.isna(value):
            return np.nan
        if isinstance(value, str):
            # Remove any non-numeric characters except minus, decimal point
            cleaned = re.sub(r'[^\d.-]', '', value)
            try:
                return float(cleaned) if cleaned else np.nan
            except ValueError:
                return np.nan
        return float(value)

    def objective(self, trial, dtrain, num_boost_round=500):
        """Objective function with enhanced regularization"""
        params = {
            'eta': trial.suggest_float('eta', 0.005, 0.1, log=True),  # Lower learning rate
            'max_depth': trial.suggest_int('max_depth', 2, 4),  # Shallower trees
            'subsample': trial.suggest_float('subsample', 0.4, 0.7),  # Lower subsampling
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 0.7),  # Lower feature sampling
            'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.4, 0.7),
            'min_child_weight': trial.suggest_int('min_child_weight', 10, 20),  # Higher min child weight
            'lambda': trial.suggest_float('lambda', 10, 25.0),  # Stronger L2 regularization
            'alpha': trial.suggest_float('alpha', 5.0, 15.0),  # Stronger L1 regularization
            'gamma': trial.suggest_float('gamma', 0.5, 2.0),  # Higher pruning parameter
            'max_delta_step': trial.suggest_int('max_delta_step', 0, 1),
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'verbosity': 0,
            'seed': self.seed,
            'nthread': 1
        }

        cv_results = xgb.cv(
            params=params,
            dtrain=dtrain,
            num_boost_round=800,  # More rounds but more conservative early stopping
            nfold=5,
            metrics='rmse',
            early_stopping_rounds=25,  # More conservative early stopping
            as_pandas=True,
            seed=self.seed,
            shuffle=True
        )

        # Stronger overfitting penalty
        train_rmse = cv_results['train-rmse-mean'].iloc[-1]
        val_rmse = cv_results['test-rmse-mean'].iloc[-1]
        
        # Calculate overfitting degree
        overfitting_ratio = (train_rmse - val_rmse) / train_rmse
        gap_penalty = max(0, overfitting_ratio) * 0.3  # Stronger penalty
        
        # Add extra penalty if severe overfitting
        if overfitting_ratio > 0.1:
            gap_penalty += (overfitting_ratio - 0.1) * 0.5
            
        return val_rmse + gap_penalty
    
    def tune_hyperparameters(self, dtrain, n_trials=20):  # Increased number of trials
        """Hyperparameter tuning focused on generalization"""
        study = optuna.create_study(direction='minimize')
        
        print("Tuning hyperparameters with aggressive overfitting prevention...")
        print("Using very strong regularization and shallow trees")

        with ThreadPoolExecutor(max_workers=self.n_jobs) as executor:
            futures = [executor.submit(lambda: study.optimize(
                lambda trial: self.objective(trial, dtrain),
                n_trials=5,
                n_jobs=1
            )) for _ in range(n_trials//5)]

            for future in futures:
                future.result()

        self.study = study
        self.best_params = study.best_params
        self.best_params.update({
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'seed': self.seed,
            'nthread': self.n_jobs
        })

        print(f"Best parameters found: {self.best_params}")
        return self.best_params


    def train_model(self, params, dtrain, dvalid=None, num_boost_round=500):
        """Train model with focus on preventing overfitting"""
        evals = [(dtrain, 'train')]
        if dvalid is not None:
            evals.append((dvalid, 'valid'))

        evals_result = {}
        model = xgb.train(
            params=params,
            dtrain=dtrain,
            num_boost_round=num_boost_round,
            evals=evals,
            early_stopping_rounds=25,  # More conservative early stopping
            verbose_eval=100,  # Reduced output frequency
            evals_result=evals_result
        )

        self.feature_names = dtrain.feature_names
        if self.feature_names is None:
            self.feature_names = [f'f{i}' for i in range(dtrain.num_col())]
            model.feature_names = self.feature_names

        return model, evals_result

    def cross_validate(self, params, X, y, n_splits=5, cv_models_dir='cv_models', n_repeats=10): 
        """Enhanced CV with 10 repetitions of 5-fold cross-validation and model saving"""
        Path(cv_models_dir).mkdir(parents=True, exist_ok=True)

        all_cv_results = {
            'train_rmse': [], 'val_rmse': [],
            'train_r2': [], 'val_r2': [],
            'best_iterations': [],
            'model_paths': [],
            'feature_names': [],
            'repeat': [],
            'fold': []
        }

        def process_fold(repeat_idx, fold_idx, train_idx, val_idx):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=X_train.columns.tolist())
            dval = xgb.DMatrix(X_val, label=y_val, feature_names=X_train.columns.tolist())

            model, _ = self.train_model(params, dtrain, dval)

            # Save the CV model
            model_path = os.path.join(cv_models_dir, f'cv_model_repeat_{repeat_idx}_fold_{fold_idx}.json')
            model.save_model(model_path)

            train_pred = model.predict(dtrain)
            val_pred = model.predict(dval)

            return (
                np.sqrt(mean_squared_error(y_train, train_pred)),
                np.sqrt(mean_squared_error(y_val, val_pred)),
                r2_score(y_train, train_pred),
                r2_score(y_val, val_pred),
                model.best_iteration if hasattr(model, 'best_iteration') else params.get('num_boost_round', 500),
                model_path,
                model.feature_names,
                repeat_idx,
                fold_idx
            )
        
        with ThreadPoolExecutor(max_workers=self.n_jobs) as executor:
            futures = []
            for repeat_idx in range(n_repeats):
                kf = KFold(n_splits=n_splits, shuffle=True, random_state=self.seed + repeat_idx)
                for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X)):
                    futures.append(executor.submit(
                        process_fold, repeat_idx, fold_idx, train_idx, val_idx
                    ))

            for future in futures:
                train_rmse, val_rmse, train_r2, val_r2, best_iter, model_path, feature_names, repeat_idx, fold_idx = future.result()
                all_cv_results['train_rmse'].append(train_rmse)
                all_cv_results['val_rmse'].append(val_rmse)
                all_cv_results['train_r2'].append(train_r2)
                all_cv_results['val_r2'].append(val_r2)
                all_cv_results['best_iterations'].append(best_iter)
                all_cv_results['model_paths'].append(model_path)
                all_cv_results['feature_names'].append(feature_names)
                all_cv_results['repeat'].append(repeat_idx)
                all_cv_results['fold'].append(fold_idx)
        # Create summary statistics
        cv_results_summary = {
            'train_rmse_mean': np.mean(all_cv_results['train_rmse']),
            'train_rmse_std': np.std(all_cv_results['train_rmse']),
            'val_rmse_mean': np.mean(all_cv_results['val_rmse']),
            'val_rmse_std': np.std(all_cv_results['val_rmse']),
            'train_r2_mean': np.mean(all_cv_results['train_r2']),
            'train_r2_std': np.std(all_cv_results['train_r2']),
            'val_r2_mean': np.mean(all_cv_results['val_r2']),
            'val_r2_std': np.std(all_cv_results['val_r2']),
            'best_iterations_mean': np.mean(all_cv_results['best_iterations']),
            'best_iterations_std': np.std(all_cv_results['best_iterations']),
            'detailed_results': pd.DataFrame(all_cv_results)
        }

        return cv_results_summary

    def evaluate_model(self, model, dtest):
        """Evaluate model performance on test set"""
        test_pred = model.predict(dtest)
        return {
            'rmse': np.sqrt(mean_squared_error(dtest.get_label(), test_pred)),
            'r2': r2_score(dtest.get_label(), test_pred),
            'mae': mean_absolute_error(dtest.get_label(), test_pred)
        }, test_pred

    def plot_learning_curves(self, evals_result, title_suffix):
        """Plot training and validation metrics"""
        plt.figure(figsize=(10, 5))
        epochs = len(evals_result['train']['rmse'])
        plt.plot(range(epochs), evals_result['train']['rmse'], label='Train RMSE')
        if 'valid' in evals_result:
            plt.plot(range(epochs), evals_result['valid']['rmse'], label='Validation RMSE')
        plt.legend()
        plt.xlabel('Iterations')
        plt.ylabel('RMSE')
        plt.title(f'Learning Curves - {title_suffix}')
        plt.show()

    def plot_cv_results(self, cv_results_summary):
        """Plot enhanced cross-validation results with both original and new visualization"""
        # Check if it's the new summary format or old list format
        if isinstance(cv_results_summary, dict) and 'detailed_results' in cv_results_summary:
            # New format - use the detailed results
            detailed_results = cv_results_summary['detailed_results']
            
            # Create two visualizations:
            # 1. Original visualization (simple line plot for first repeat)
            plt.figure(figsize=(20, 10))
            
            # Plot 1: Original style - first repeat only
            plt.subplot(2, 2, 1)
            first_repeat = detailed_results[detailed_results['repeat'] == 0]
            plt.plot(first_repeat['train_rmse'], 'o-', label='Train RMSE')
            plt.plot(first_repeat['val_rmse'], 'o-', label='Validation RMSE')
            plt.xlabel('Fold')
            plt.ylabel('RMSE')
            plt.title('Cross-Validation RMSE (First Repeat)')
            plt.legend()
            plt.grid(True)

            # Plot 2: Original style - first repeat R²
            plt.subplot(2, 2, 2)
            plt.plot(first_repeat['train_r2'], 'o-', label='Train R²')
            plt.plot(first_repeat['val_r2'], 'o-', label='Validation R²')
            plt.xlabel('Fold')
            plt.ylabel('R² Score')
            plt.title('Cross-Validation R² Scores (First Repeat)')
            plt.legend()
            plt.grid(True)

            # Plot 3: New visualization - RMSE distribution across all repeats
            plt.subplot(2, 2, 3)
            plt.boxplot([detailed_results['train_rmse'], detailed_results['val_rmse']], 
                       labels=['Train RMSE', 'Validation RMSE'])
            plt.ylabel('RMSE')
            plt.title('RMSE Distribution across 10x5 CV')
            plt.grid(True, alpha=0.3)
            
            # Plot 4: New visualization - R² distribution across all repeats
            plt.subplot(2, 2, 4)
            plt.boxplot([detailed_results['train_r2'], detailed_results['val_r2']], 
                       labels=['Train R²', 'Validation R²'])
            plt.ylabel('R² Score')
            plt.title('R² Distribution across 10x5 CV')
            plt.grid(True, alpha=0.3)

            plt.tight_layout()
            plt.show()
            
            # Print summary statistics
            print("\n=== 10x5 Cross-Validation Summary ===")
            print(f"Average Train RMSE: {cv_results_summary['train_rmse_mean']:.4f} (±{cv_results_summary['train_rmse_std']:.4f})")
            print(f"Average Validation RMSE: {cv_results_summary['val_rmse_mean']:.4f} (±{cv_results_summary['val_rmse_std']:.4f})")
            print(f"Average Train R²: {cv_results_summary['train_r2_mean']:.4f} (±{cv_results_summary['train_r2_std']:.4f})")
            print(f"Average Validation R²: {cv_results_summary['val_r2_mean']:.4f} (±{cv_results_summary['val_r2_std']:.4f})")
            print(f"Average Best Iterations: {cv_results_summary['best_iterations_mean']:.1f} (±{cv_results_summary['best_iterations_std']:.1f})")
            
            # Also print the format that main() expects
            print("\n=== 交叉验证结果 (旧格式兼容) ===")
            print(f"平均训练RMSE: {cv_results_summary['train_rmse_mean']:.4f} (±{cv_results_summary['train_rmse_std']:.4f})")
            print(f"平均验证RMSE: {cv_results_summary['val_rmse_mean']:.4f} (±{cv_results_summary['val_rmse_std']:.4f})")
            print(f"平均训练R²: {cv_results_summary['train_r2_mean']:.4f} (±{cv_results_summary['train_r2_std']:.4f})")
            print(f"平均验证R²: {cv_results_summary['val_r2_mean']:.4f} (±{cv_results_summary['val_r2_std']:.4f})")
            
        else:
            # Old format - keep original plotting
            plt.figure(figsize=(15, 5))

            # Plot RMSE
            plt.subplot(1, 2, 1)
            plt.plot(cv_results_summary['train_rmse'], 'o-', label='Train RMSE')
            plt.plot(cv_results_summary['val_rmse'], 'o-', label='Validation RMSE')
            plt.xlabel('Fold')
            plt.ylabel('RMSE')
            plt.title('Cross-Validation RMSE')
            plt.legend()
            plt.grid(True)

            # Plot R2
            plt.subplot(1, 2, 2)
            plt.plot(cv_results_summary['train_r2'], 'o-', label='Train R²')
            plt.plot(cv_results_summary['val_r2'], 'o-', label='Validation R²')
            plt.xlabel('Fold')
            plt.ylabel('R² Score')
            plt.title('Cross-Validation R² Scores')
            plt.legend()
            plt.grid(True)

            plt.tight_layout()
            plt.show()

    def plot_final_model_performance(self, model, dtest):
        """Plot actual vs predicted values for test set"""
        test_metrics, test_pred = self.evaluate_model(model, dtest)
        y_test = dtest.get_label()

        plt.figure(figsize=(12, 5))

        # Scatter plot of actual vs predicted
        plt.subplot(1, 2, 1)
        plt.scatter(y_test, test_pred, alpha=0.5)
        plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--r')
        plt.xlabel('Actual Values')
        plt.ylabel('Predicted Values')
        plt.title(f'Actual vs Predicted (R² = {test_metrics["r2"]:.3f})')

        # Residual plot
        plt.subplot(1, 2, 2)
        residuals = y_test - test_pred
        plt.scatter(test_pred, residuals, alpha=0.5)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.xlabel('Predicted Values')
        plt.ylabel('Residuals')
        plt.title('Residual Plot')
        plt.tight_layout()
        plt.show()

    def plot_feature_importance(self, model, importance_type='weight', top_n=30):
        """Plot feature importance without category-specific highlighting"""
        fig, ax = plt.subplots(figsize=(12, 10))
        
      
        importance_dict = model.get_score(importance_type=importance_type)
        
        if not importance_dict:
            print("No feature importance data available.")
            return None
            
   
        importance_df = pd.DataFrame({
            'feature': list(importance_dict.keys()),
            'importance': list(importance_dict.values())
        }).sort_values('importance', ascending=True)
        
  
        if len(importance_df) > top_n:
            importance_df = importance_df.tail(top_n)
        
    
        y_pos = np.arange(len(importance_df))
        colors = ['lightblue'] * len(importance_df)
        
        ax.barh(y_pos, importance_df['importance'], color=colors, alpha=0.8)
        ax.set_yticks(y_pos)
        ax.set_yticklabels(importance_df['feature'])
        ax.set_xlabel('Importance')
        
        title = f'Feature Importance ({importance_type})'
        ax.set_title(title)
        

        lu_type_features = importance_df[importance_df['feature'].str.contains('LUtype', na=False)]
        lu_type_importance = lu_type_features['importance'].sum() if not lu_type_features.empty else 0
        
        ax.text(0.7, 0.95, f'LUtype total: {lu_type_importance:.3f}', 
                transform=ax.transAxes, fontsize=10, bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow"))
        
        plt.tight_layout()
        plt.show()
        
        return importance_df

    def analyze_lutype_importance(self, model, X):
        """分析LUtype相关特征的重要性，不关注特定类别"""
        importance_dict = model.get_score(importance_type='weight')
        
        if not importance_dict:
            print("No feature importance data available.")
            return
            
 
        lutype_features = {k: v for k, v in importance_dict.items() if 'LUtype' in k}
        other_features = {k: v for k, v in importance_dict.items() if 'LUtype' not in k}
        
        print(f"\n=== LUtype Feature Importance Analysis ===")
        print(f"Total LUtype-related features: {len(lutype_features)}")
        print(f"Total importance of LUtype features: {sum(lutype_features.values()):.3f}")
        print(f"Average importance of LUtype features: {np.mean(list(lutype_features.values())) if lutype_features else 0:.3f}")
        
        total_importance = sum(importance_dict.values())
        lu_type_percentage = (sum(lutype_features.values()) / total_importance * 100) if total_importance > 0 else 0
        
        print(f"\nLUtype importance percentage: {lu_type_percentage:.2f}%")
        

        if lutype_features:
            sorted_lutype = sorted(lutype_features.items(), key=lambda x: x[1], reverse=True)
            print(f"\nTop LUtype-related features:")
            for feature, importance in sorted_lutype[:10]:
                print(f"  {feature}: {importance:.4f}")
        else:
            print("\nNo LUtype-related features found in importance scores.")

    def plot_shap_summary(self, model, X, feature_names=None):
        """Generate SHAP summary plot if SHAP is available"""
        if SHAP_AVAILABLE:
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X)

            plt.figure(figsize=(10, 8))
            shap.summary_plot(shap_values, X, feature_names=feature_names)
            plt.tight_layout()
            plt.show()
        else:
            print("SHAP not available. Install with: pip install shap")

    def save_results(self, model, output_dir='results'):
        """Save all results to disk including feature names"""
        Path(output_dir).mkdir(exist_ok=True)

        # Save model
        model.save_model(f'{output_dir}/xgb_model.json')

        # Save feature names
        if self.feature_names is not None:
            with open(f'{output_dir}/feature_names.txt', 'w') as f:
                f.write('\n'.join(self.feature_names))

        # Save other artifacts
        joblib.dump(self.best_params, f'{output_dir}/best_params.pkl')

        if self.study:
            joblib.dump(self.study, f'{output_dir}/study.pkl')
            study_df = self.study.trials_dataframe()
            study_df.to_csv(f'{output_dir}/trials.csv', index=False)

def detect_outliers_iqr(df, columns, threshold=1.5):
    """Detect outliers using IQR method"""
    outlier_indices = []
    
    for col in columns:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
            outlier_indices.extend(outliers)
            
            if len(outliers) > 0:
                print(f"  {col}: Found {len(outliers)} outliers "
                      f"({len(outliers)/len(df)*100:.2f}%)")
    
    return list(set(outlier_indices))

def remove_outliers_robust(df, y, columns, method='iqr', threshold=2.0):
    """Robust outlier handling method"""
    print("Starting outlier detection...")
    
    if method == 'iqr':
        outlier_indices = detect_outliers_iqr(df, columns, threshold)
    elif method == 'zscore':
        # Z-score method
        z_scores = np.abs(stats.zscore(df[columns]))
        outlier_indices = np.where(z_scores > threshold)[0]
        outlier_indices = list(outlier_indices)
    else:
        return df, y, []
    
    print(f"\nTotal outliers found: {len(outlier_indices)}")
    
    if len(outlier_indices) > 0:
        df_clean = df.drop(outlier_indices)
        y_clean = y.drop(outlier_indices)
        print(f"Data size after removing outliers: {len(df_clean)} (removed {len(outlier_indices)} samples)")
        return df_clean, y_clean, outlier_indices
    
    print("No outliers found")
    return df, y, []

def check_data_leakage(X_train, X_test, y_train, y_test):
    """Check for data leakage"""
    print("\n=== Data Leakage Check ===")
    
    # Check for overlap between train and test sets
    train_indices = set(X_train.index)
    test_indices = set(X_test.index)
    overlap = train_indices.intersection(test_indices)
    
    if overlap:
        print(f"❌ Data leakage detected: {len(overlap)} overlapping samples between train and test sets")
    else:
        print("✅ No overlap between train and test sets")
    
    # Check feature distributions
    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")
    print(f"Number of features: {X_train.shape[1]}")
    
    return len(overlap) == 0

def safe_feature_engineering(X_train, X_test):
    """Safe feature engineering to avoid data leakage"""
    print("Performing safe feature engineering...")
    
    # 1. Logarithmic transformation
    columns_to_log = ['Lon', 'Lat', 'Age', 'BD', 'pH']
    for col in columns_to_log:
        if col in X_train.columns:
            X_train[col + '_log'] = np.log(X_train[col] + 1e-8)
            X_test[col + '_log'] = np.log(X_test[col] + 1e-8)
            print(f"  Created log feature: {col}_log")
    
    # 2. Binning (based on training set quantiles)
    if 'Altitude' in X_train.columns:
        # Use training set to compute bin boundaries
        alt_bins = pd.cut(X_train['Altitude'], bins=5, retbins=True)[1]
        X_train['Altitude_bins'] = pd.cut(X_train['Altitude'], bins=alt_bins, labels=False)
        X_test['Altitude_bins'] = pd.cut(X_test['Altitude'], bins=alt_bins, labels=False)
        print("  Created binned feature: Altitude_bins")
    
    return X_train, X_test

def augment_continuous_features(X, y, continuous_cols, augmentation_factor=2, noise_scale=0.01, random_state=42):
    """Augment continuous features with noise - only for training set"""
    np.random.seed(random_state)
    n_samples = len(X)
    n_augment = int(n_samples * augmentation_factor)

    X_augmented = X.copy()
    y_augmented = y.copy()

    for _ in range(n_augment):
        idx = np.random.randint(0, n_samples)
        base_sample = X.iloc[idx].copy()

        for col in continuous_cols:
            std = X[col].std()
            noise = np.random.normal(loc=0, scale=std * noise_scale)
            if X[col].min() >= 0:
                base_sample[col] = max(0, base_sample[col] + noise)
            else:
                base_sample[col] += noise

        X_augmented = X_augmented._append(base_sample, ignore_index=True)
        
        y_augmented = y_augmented._append(
            pd.Series(y.iloc[idx], index=[len(y_augmented)]),  
            ignore_index=True
        )

    return X_augmented, y_augmented

def enhanced_load_and_prepare_data(augment=True, augmentation_factor=2, 
                                 lu_type_importance_boost=3.0, 
                                 outlier_threshold=2.0):
    """Fixed data leakage version with outlier handling"""
    
    # Load data
    dtype_dict = {
        'Soillayer': 'int8',
        'yi': 'float32'
    }

    df_clean = pd.read_csv('F:/model/df.clean.yi.csv', dtype=dtype_dict)
    data_predictor_VIF = pd.read_csv('F:/model/data.predictor.VIF.yi.csv')

    # Select target soil layer
    target_layer = "topsoil" ###################################################################################SELECT LAYER
    df = df_clean[df_clean['Soillayer'] == (1 if target_layer == 'topsoil' else 2)].copy()

    # Select features
    predictor_columns = data_predictor_VIF['x'].tolist()
    valid_x_cols = [col for col in predictor_columns if col != "Soillayer" and col in df.columns]

    # Data cleaning
    for col in valid_x_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    X = df[valid_x_cols].astype('float32').fillna(0)
    y = df['yi'].astype('float32')

    print(f"Original data size: {len(X)}")
    
    # Outlier detection and handling
    continuous_cols = ['Lon', 'Lat', 'Age', 'BD', 'pH', 'Altitude']
    continuous_cols = [col for col in continuous_cols if col in X.columns]
    
    print("\n=== Outlier Handling ===")
    X_clean, y_clean, outliers = remove_outliers_robust(
        X, y, continuous_cols, threshold=outlier_threshold
    )

    # Data splitting
    X_train, X_test, y_train, y_test = train_test_split(
        X_clean, y_clean, test_size=0.3, random_state=42
    )

    # Safe feature engineering
    X_train, X_test = safe_feature_engineering(X_train, X_test)

    # Data leakage check
    check_data_leakage(X_train, X_test, y_train, y_test)

    # LUtype feature enhancement (only for training set)
    if 'LUtype' in X_train.columns:
        print(f"\n=== LUtype Feature Enhancement ===")
        print(f"Original LUtype distribution:")
        print(X_train['LUtype'].value_counts().sort_index())
        
        print(f"\nEnhancing LUtype importance with factor: {lu_type_importance_boost}")
        
        # Create LUtype feature enhancements
        for i in range(int(lu_type_importance_boost) - 1):
            lu_type_col_name = f'LUtype_boost_{i+1}'
            X_train[lu_type_col_name] = X_train['LUtype']
            X_test[lu_type_col_name] = X_test['LUtype']
            print(f"  Created LUtype copy: {lu_type_col_name}")
        
        # Create interaction terms between LUtype and other important features
        important_features = ['pH', 'BD', 'Age', 'Altitude']
        for feature in important_features:
            if feature in X_train.columns:
                interaction_name = f'LUtype_{feature}_interaction'
                X_train[interaction_name] = X_train['LUtype'] * X_train[feature]
                X_test[interaction_name] = X_test['LUtype'] * X_test[feature]
                print(f"  Created interaction feature: {interaction_name}")
        
        # Create LUtype squared term (non-linear effect)
        X_train['LUtype_squared'] = X_train['LUtype'] ** 2
        X_test['LUtype_squared'] = X_test['LUtype'] ** 2
        print("  Created LUtype_squared")
    
    # Data augmentation (only for training set)
    if augment:
        continuous_cols = [
            'Lon', 'Lat', 'Age', 'BD', 'pH',
            'Lon_log', 'Lat_log', 'Age_log', 'BD_log', 'pH_log',
            'Altitude'
        ]
        continuous_cols = [col for col in continuous_cols if col in X_train.columns]
        
        print(f"\n=== Data Augmentation ===")
        print(f"Training set size before augmentation: {len(X_train)}")
        X_train, y_train = augment_continuous_features(
            X=X_train,  # Only augment training set
            y=y_train,  # Only augment training set
            continuous_cols=continuous_cols,
            augmentation_factor=augmentation_factor,
            noise_scale=0.01,  
            random_state=42
        )
        print(f"Training set size after augmentation: {len(X_train)}")
    else:
        print("\nSkipping data augmentation")

    print(f"\n=== Final Dataset Statistics ===")
    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")
    print(f"Total number of features: {len(X_train.columns)}")
    
    # Statistical feature distribution
    lu_type_cols = [col for col in X_train.columns if 'LUtype' in col]
    
    print(f"\nFeature statistics:")
    print(f"LUtype-related features: {len(lu_type_cols)}")

    dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=X_train.columns.tolist())
    dtest = xgb.DMatrix(X_test, label=y_test, feature_names=X_train.columns.tolist())

    return dtrain, dtest, X, y, X_test, y_test, X_train, y_train

def main():
    """Main function with data leakage prevention"""
    outlier_threshold = 2.0  # Outlier detection threshold
    
    print("=== Starting Enhanced XGBoost Modeling (Fixed Data Leakage Version) ===")
    
    # Use fixed data processing (with outlier handling and data leakage prevention)
    dtrain, dtest, x1, y1, X_test, y_test, X_train, y_train = enhanced_load_and_prepare_data(
        augment=True,
        augmentation_factor=3,
        lu_type_importance_boost=3.0,
        outlier_threshold=outlier_threshold
    )

    tuner = XGBoostTuner(seed=42, n_jobs=17)

    # 1. Hyperparameter tuning
    print("\n=== Starting Hyperparameter Tuning ===")
    best_params = tuner.tune_hyperparameters(dtrain, n_trials=10)
    print(f"Best parameters: {best_params}")

    # 2. Enhanced Cross-validation with 10x5 folds and model saving
    print("\n=== Starting 10x5 Cross-Validation ===")
    cv_results_summary = tuner.cross_validate(best_params, X_train, y_train, n_splits=5,
                                              cv_models_dir='F:/model/results/cv_models',
                                              n_repeats=10)  # 10 repetitions of 5-fold CV
    
    print("\nSaved CV models:")
    for path in cv_results_summary['detailed_results']['model_paths'][:5]:  # Show first 5
        print(f"- {path}")
    print(f"... and {len(cv_results_summary['detailed_results']['model_paths']) - 5} more")
    
    # Plot enhanced CV results
    print("\nPlotting enhanced cross-validation results...")
    tuner.plot_cv_results(cv_results_summary)

    # 3. Train final model
    optimal_rounds = int(cv_results_summary['best_iterations_mean'])
    print(f"\nTraining final model with {optimal_rounds} rounds...")
    final_model, evals_result = tuner.train_model(best_params, dtrain, num_boost_round=optimal_rounds)

    # Plot learning curves
    print("\nPlotting learning curves...")
    tuner.plot_learning_curves(evals_result, "Final Model")

    # 4. Evaluate and plot final performance
    print("\nEvaluating final model...")
    test_metrics, _ = tuner.evaluate_model(final_model, dtest)
    print("\n=== Final Test Metrics ===")
    print(f"RMSE: {test_metrics['rmse']:.4f}")
    print(f"R²: {test_metrics['r2']:.4f}")
    print(f"MAE: {test_metrics['mae']:.4f}")

    print("\nPlotting final model performance...")
    tuner.plot_final_model_performance(final_model, dtest)

    # 5. Analyze LUtype importance
    print(f"\nAnalyzing LUtype feature importance...")
    importance_df = tuner.plot_feature_importance(final_model, top_n=35)
    
    # Detailed analysis of LUtype importance
    tuner.analyze_lutype_importance(final_model, X_test)
    
    # 6. SHAP analysis
    if SHAP_AVAILABLE:
        print("\nGenerating SHAP summary plot...")
        tuner.plot_shap_summary(final_model, X_test)
        
    # 7. Save results
    print("\nSaving results...")
    tuner.save_results(final_model, 'F:/model/results')
    
    # Save CV results summary
    cv_summary_path = 'F:/model/results/cv_summary.csv'
    cv_results_summary['detailed_results'].to_csv(cv_summary_path, index=False)
    print(f"Saved detailed CV results to: {cv_summary_path}")
    
    print(f"\n=== Model Training Summary ===")
    print("✓ Fixed data leakage issues")
    print("✓ Added outlier detection and handling")
    print("✓ Safe feature engineering (avoided data leakage)")
    print("✓ Created multiple LUtype feature copies")
    print("✓ Added LUtype interaction features with other important variables")
    print("✓ Added LUtype squared term (non-linear effect)")
    print("✓ Improved hyperparameter search space for better handling of categorical features")
    print("✓ Test set uses original data (no augmentation noise)")
    print("✓ Implemented 10 repetitions of 5-fold cross-validation for robust evaluation")
    
    print("\n=== Complete! ===")

if __name__ == "__main__":
    main()

In [ ]:
# XGBoost modeling: passive fractions  in topsoil
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import joblib
import os
import re
from pathlib import Path
import warnings
import builtins
from scipy import stats
open = builtins.open

warnings.filterwarnings('ignore')

# SHAP availability check
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False

class XGBoostTuner:
    def __init__(self, seed=42, n_jobs=-1):
        self.seed = seed
        self.n_jobs = n_jobs if n_jobs != -1 else None
        self.study = None
        self.best_model = None
        self.best_params = None
        self.feature_names = None  # Store feature names

    @staticmethod
    def clean_numeric_value(value):
        """Convert string values to numeric, handling special cases"""
        if pd.isna(value):
            return np.nan
        if isinstance(value, str):
            # Remove any non-numeric characters except minus, decimal point
            cleaned = re.sub(r'[^\d.-]', '', value)
            try:
                return float(cleaned) if cleaned else np.nan
            except ValueError:
                return np.nan
        return float(value)

    def objective(self, trial, dtrain, num_boost_round=500):
        """Objective function with enhanced regularization"""
        params = {
            'eta': trial.suggest_float('eta', 0.005, 0.1, log=True),  # Lower learning rate
            'max_depth': trial.suggest_int('max_depth', 2, 4),  # Shallower trees
            'subsample': trial.suggest_float('subsample', 0.4, 0.7),  # Lower subsampling
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 0.7),  # Lower feature sampling
           # 'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.4, 0.7),
            'min_child_weight': trial.suggest_int('min_child_weight', 10, 20),  # Higher min child weight
            'lambda': trial.suggest_float('lambda', 10, 25.0),  # Stronger L2 regularization
            'alpha': trial.suggest_float('alpha', 5.0, 15.0),  # Stronger L1 regularization
            'gamma': trial.suggest_float('gamma', 0.5, 2.0),  # Higher pruning parameter
            'max_delta_step': trial.suggest_int('max_delta_step', 0, 1),
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'verbosity': 0,
            'seed': self.seed,
            'nthread': 1
        }

        cv_results = xgb.cv(
            params=params,
            dtrain=dtrain,
            num_boost_round=800,  # More rounds but more conservative early stopping
            nfold=5,
            metrics='rmse',
            early_stopping_rounds=25,  # More conservative early stopping
            as_pandas=True,
            seed=self.seed,
            shuffle=True
        )

        # Stronger overfitting penalty
        train_rmse = cv_results['train-rmse-mean'].iloc[-1]
        val_rmse = cv_results['test-rmse-mean'].iloc[-1]
        
        # Calculate overfitting degree
        overfitting_ratio = (train_rmse - val_rmse) / train_rmse
        gap_penalty = max(0, overfitting_ratio) * 0.3  # Stronger penalty
        
        # Add extra penalty if severe overfitting
        if overfitting_ratio > 0.1:
            gap_penalty += (overfitting_ratio - 0.1) * 0.5
            
        return val_rmse + gap_penalty
    
    def tune_hyperparameters(self, dtrain, n_trials=10):  # Increased number of trials
        """Hyperparameter tuning focused on generalization"""
        study = optuna.create_study(direction='minimize')
        
        print("Tuning hyperparameters with aggressive overfitting prevention...")
        print("Using very strong regularization and shallow trees")

        with ThreadPoolExecutor(max_workers=self.n_jobs) as executor:
            futures = [executor.submit(lambda: study.optimize(
                lambda trial: self.objective(trial, dtrain),
                n_trials=5,
                n_jobs=1
            )) for _ in range(n_trials//5)]

            for future in futures:
                future.result()

        self.study = study
        self.best_params = study.best_params
        self.best_params.update({
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'seed': self.seed,
            'nthread': self.n_jobs
        })

        print(f"Best parameters found: {self.best_params}")
        return self.best_params


    def train_model(self, params, dtrain, dvalid=None, num_boost_round=500):
        """Train model with focus on preventing overfitting"""
        evals = [(dtrain, 'train')]
        if dvalid is not None:
            evals.append((dvalid, 'valid'))

        evals_result = {}
        model = xgb.train(
            params=params,
            dtrain=dtrain,
            num_boost_round=num_boost_round,
            evals=evals,
            early_stopping_rounds=25,  # More conservative early stopping
            verbose_eval=100,  # Reduced output frequency
            evals_result=evals_result
        )

        self.feature_names = dtrain.feature_names
        if self.feature_names is None:
            self.feature_names = [f'f{i}' for i in range(dtrain.num_col())]
            model.feature_names = self.feature_names

        return model, evals_result

    def cross_validate(self, params, X, y, n_splits=5, cv_models_dir='cv_models', n_repeats=10): 
        """Enhanced CV with 10 repetitions of 5-fold cross-validation and model saving"""
        Path(cv_models_dir).mkdir(parents=True, exist_ok=True)

        all_cv_results = {
            'train_rmse': [], 'val_rmse': [],
            'train_r2': [], 'val_r2': [],
            'best_iterations': [],
            'model_paths': [],
            'feature_names': [],
            'repeat': [],
            'fold': []
        }

        def process_fold(repeat_idx, fold_idx, train_idx, val_idx):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=X_train.columns.tolist())
            dval = xgb.DMatrix(X_val, label=y_val, feature_names=X_train.columns.tolist())

            model, _ = self.train_model(params, dtrain, dval)

            # Save the CV model
            model_path = os.path.join(cv_models_dir, f'cv_model_repeat_{repeat_idx}_fold_{fold_idx}.json')
            model.save_model(model_path)

            train_pred = model.predict(dtrain)
            val_pred = model.predict(dval)

            return (
                np.sqrt(mean_squared_error(y_train, train_pred)),
                np.sqrt(mean_squared_error(y_val, val_pred)),
                r2_score(y_train, train_pred),
                r2_score(y_val, val_pred),
                model.best_iteration if hasattr(model, 'best_iteration') else params.get('num_boost_round', 500),
                model_path,
                model.feature_names,
                repeat_idx,
                fold_idx
            )
        
        with ThreadPoolExecutor(max_workers=self.n_jobs) as executor:
            futures = []
            for repeat_idx in range(n_repeats):
                kf = KFold(n_splits=n_splits, shuffle=True, random_state=self.seed + repeat_idx)
                for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X)):
                    futures.append(executor.submit(
                        process_fold, repeat_idx, fold_idx, train_idx, val_idx
                    ))

            for future in futures:
                train_rmse, val_rmse, train_r2, val_r2, best_iter, model_path, feature_names, repeat_idx, fold_idx = future.result()
                all_cv_results['train_rmse'].append(train_rmse)
                all_cv_results['val_rmse'].append(val_rmse)
                all_cv_results['train_r2'].append(train_r2)
                all_cv_results['val_r2'].append(val_r2)
                all_cv_results['best_iterations'].append(best_iter)
                all_cv_results['model_paths'].append(model_path)
                all_cv_results['feature_names'].append(feature_names)
                all_cv_results['repeat'].append(repeat_idx)
                all_cv_results['fold'].append(fold_idx)
        # Create summary statistics
        cv_results_summary = {
            'train_rmse_mean': np.mean(all_cv_results['train_rmse']),
            'train_rmse_std': np.std(all_cv_results['train_rmse']),
            'val_rmse_mean': np.mean(all_cv_results['val_rmse']),
            'val_rmse_std': np.std(all_cv_results['val_rmse']),
            'train_r2_mean': np.mean(all_cv_results['train_r2']),
            'train_r2_std': np.std(all_cv_results['train_r2']),
            'val_r2_mean': np.mean(all_cv_results['val_r2']),
            'val_r2_std': np.std(all_cv_results['val_r2']),
            'best_iterations_mean': np.mean(all_cv_results['best_iterations']),
            'best_iterations_std': np.std(all_cv_results['best_iterations']),
            'detailed_results': pd.DataFrame(all_cv_results)
        }

        return cv_results_summary

    def evaluate_model(self, model, dtest):
        """Evaluate model performance on test set"""
        test_pred = model.predict(dtest)
        return {
            'rmse': np.sqrt(mean_squared_error(dtest.get_label(), test_pred)),
            'r2': r2_score(dtest.get_label(), test_pred),
            'mae': mean_absolute_error(dtest.get_label(), test_pred)
        }, test_pred

    def plot_learning_curves(self, evals_result, title_suffix):
        """Plot training and validation metrics"""
        plt.figure(figsize=(10, 5))
        epochs = len(evals_result['train']['rmse'])
        plt.plot(range(epochs), evals_result['train']['rmse'], label='Train RMSE')
        if 'valid' in evals_result:
            plt.plot(range(epochs), evals_result['valid']['rmse'], label='Validation RMSE')
        plt.legend()
        plt.xlabel('Iterations')
        plt.ylabel('RMSE')
        plt.title(f'Learning Curves - {title_suffix}')
        plt.show()

    def plot_cv_results(self, cv_results_summary):
        """Plot enhanced cross-validation results with both original and new visualization"""
        # Check if it's the new summary format or old list format
        if isinstance(cv_results_summary, dict) and 'detailed_results' in cv_results_summary:
            # New format - use the detailed results
            detailed_results = cv_results_summary['detailed_results']
            
            # Create two visualizations:
            # 1. Original visualization (simple line plot for first repeat)
            plt.figure(figsize=(15, 10))
            
            # Plot 1: Original style - first repeat only
            plt.subplot(2, 2, 1)
            first_repeat = detailed_results[detailed_results['repeat'] == 0]
            plt.plot(first_repeat['train_rmse'], 'o-', label='Train RMSE')
            plt.plot(first_repeat['val_rmse'], 'o-', label='Validation RMSE')
            plt.xlabel('Fold')
            plt.ylabel('RMSE')
            plt.title('Cross-Validation RMSE (First Repeat)')
            plt.legend()
            plt.grid(True)

            # Plot 2: Original style - first repeat R²
            plt.subplot(2, 2, 2)
            plt.plot(first_repeat['train_r2'], 'o-', label='Train R²')
            plt.plot(first_repeat['val_r2'], 'o-', label='Validation R²')
            plt.xlabel('Fold')
            plt.ylabel('R² Score')
            plt.title('Cross-Validation R² Scores (First Repeat)')
            plt.legend()
            plt.grid(True)

            # Plot 3: New visualization - RMSE distribution across all repeats
            plt.subplot(2, 2, 3)
            plt.boxplot([detailed_results['train_rmse'], detailed_results['val_rmse']], 
                       labels=['Train RMSE', 'Validation RMSE'])
            plt.ylabel('RMSE')
            plt.title('RMSE Distribution across 10x5 CV')
            plt.grid(True, alpha=0.3)
            
            # Plot 4: New visualization - R² distribution across all repeats
            plt.subplot(2, 2, 4)
            plt.boxplot([detailed_results['train_r2'], detailed_results['val_r2']], 
                       labels=['Train R²', 'Validation R²'])
            plt.ylabel('R² Score')
            plt.title('R² Distribution across 10x5 CV')
            plt.grid(True, alpha=0.3)

            plt.tight_layout()
            plt.show()
            
            # Print summary statistics
            print("\n=== 10x5 Cross-Validation Summary ===")
            print(f"Average Train RMSE: {cv_results_summary['train_rmse_mean']:.4f} (±{cv_results_summary['train_rmse_std']:.4f})")
            print(f"Average Validation RMSE: {cv_results_summary['val_rmse_mean']:.4f} (±{cv_results_summary['val_rmse_std']:.4f})")
            print(f"Average Train R²: {cv_results_summary['train_r2_mean']:.4f} (±{cv_results_summary['train_r2_std']:.4f})")
            print(f"Average Validation R²: {cv_results_summary['val_r2_mean']:.4f} (±{cv_results_summary['val_r2_std']:.4f})")
            print(f"Average Best Iterations: {cv_results_summary['best_iterations_mean']:.1f} (±{cv_results_summary['best_iterations_std']:.1f})")
            
            # Also print the format that main() expects
            print("\n=== 交叉验证结果 (旧格式兼容) ===")
            print(f"平均训练RMSE: {cv_results_summary['train_rmse_mean']:.4f} (±{cv_results_summary['train_rmse_std']:.4f})")
            print(f"平均验证RMSE: {cv_results_summary['val_rmse_mean']:.4f} (±{cv_results_summary['val_rmse_std']:.4f})")
            print(f"平均训练R²: {cv_results_summary['train_r2_mean']:.4f} (±{cv_results_summary['train_r2_std']:.4f})")
            print(f"平均验证R²: {cv_results_summary['val_r2_mean']:.4f} (±{cv_results_summary['val_r2_std']:.4f})")
            
        else:
            # Old format - keep original plotting
            plt.figure(figsize=(15, 5))

            # Plot RMSE
            plt.subplot(1, 2, 1)
            plt.plot(cv_results_summary['train_rmse'], 'o-', label='Train RMSE')
            plt.plot(cv_results_summary['val_rmse'], 'o-', label='Validation RMSE')
            plt.xlabel('Fold')
            plt.ylabel('RMSE')
            plt.title('Cross-Validation RMSE')
            plt.legend()
            plt.grid(True)

            # Plot R2
            plt.subplot(1, 2, 2)
            plt.plot(cv_results_summary['train_r2'], 'o-', label='Train R²')
            plt.plot(cv_results_summary['val_r2'], 'o-', label='Validation R²')
            plt.xlabel('Fold')
            plt.ylabel('R² Score')
            plt.title('Cross-Validation R² Scores')
            plt.legend()
            plt.grid(True)

            plt.tight_layout()
            plt.show()

    def plot_final_model_performance(self, model, dtest):
        """Plot actual vs predicted values for test set"""
        test_metrics, test_pred = self.evaluate_model(model, dtest)
        y_test = dtest.get_label()

        plt.figure(figsize=(12, 5))

        # Scatter plot of actual vs predicted
        plt.subplot(1, 2, 1)
        plt.scatter(y_test, test_pred, alpha=0.5)
        plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--r')
        plt.xlabel('Actual Values')
        plt.ylabel('Predicted Values')
        plt.title(f'Actual vs Predicted (R² = {test_metrics["r2"]:.3f})')

        # Residual plot
        plt.subplot(1, 2, 2)
        residuals = y_test - test_pred
        plt.scatter(test_pred, residuals, alpha=0.5)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.xlabel('Predicted Values')
        plt.ylabel('Residuals')
        plt.title('Residual Plot')
        plt.tight_layout()
        plt.show()

    def plot_feature_importance(self, model, importance_type='weight', top_n=30):
        """Plot feature importance without category-specific highlighting"""
        fig, ax = plt.subplots(figsize=(12, 10))
        
    
        importance_dict = model.get_score(importance_type=importance_type)
        
        if not importance_dict:
            print("No feature importance data available.")
            return None
            

        importance_df = pd.DataFrame({
            'feature': list(importance_dict.keys()),
            'importance': list(importance_dict.values())
        }).sort_values('importance', ascending=True)
        

        if len(importance_df) > top_n:
            importance_df = importance_df.tail(top_n)
        

        y_pos = np.arange(len(importance_df))
        colors = ['lightblue'] * len(importance_df)
        
        ax.barh(y_pos, importance_df['importance'], color=colors, alpha=0.8)
        ax.set_yticks(y_pos)
        ax.set_yticklabels(importance_df['feature'])
        ax.set_xlabel('Importance')
        
        title = f'Feature Importance ({importance_type})'
        ax.set_title(title)
        

        lu_type_features = importance_df[importance_df['feature'].str.contains('LUtype', na=False)]
        lu_type_importance = lu_type_features['importance'].sum() if not lu_type_features.empty else 0
        
        ax.text(0.7, 0.95, f'LUtype total: {lu_type_importance:.3f}', 
                transform=ax.transAxes, fontsize=10, bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow"))
        
        plt.tight_layout()
        plt.show()
        
        return importance_df

    def analyze_lutype_importance(self, model, X):
     
        importance_dict = model.get_score(importance_type='weight')
        
        if not importance_dict:
            print("No feature importance data available.")
            return
            

        lutype_features = {k: v for k, v in importance_dict.items() if 'LUtype' in k}
        other_features = {k: v for k, v in importance_dict.items() if 'LUtype' not in k}
        
        print(f"\n=== LUtype Feature Importance Analysis ===")
        print(f"Total LUtype-related features: {len(lutype_features)}")
        print(f"Total importance of LUtype features: {sum(lutype_features.values()):.3f}")
        print(f"Average importance of LUtype features: {np.mean(list(lutype_features.values())) if lutype_features else 0:.3f}")
        
        total_importance = sum(importance_dict.values())
        lu_type_percentage = (sum(lutype_features.values()) / total_importance * 100) if total_importance > 0 else 0
        
        print(f"\nLUtype importance percentage: {lu_type_percentage:.2f}%")
        
     
        if lutype_features:
            sorted_lutype = sorted(lutype_features.items(), key=lambda x: x[1], reverse=True)
            print(f"\nTop LUtype-related features:")
            for feature, importance in sorted_lutype[:10]:
                print(f"  {feature}: {importance:.4f}")
        else:
            print("\nNo LUtype-related features found in importance scores.")

    def plot_shap_summary(self, model, X, feature_names=None):
        """Generate SHAP summary plot if SHAP is available"""
        if SHAP_AVAILABLE:
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X)

            plt.figure(figsize=(10, 8))
            shap.summary_plot(shap_values, X, feature_names=feature_names)
            plt.tight_layout()
            plt.show()
        else:
            print("SHAP not available. Install with: pip install shap")

    def save_results(self, model, output_dir='results'):
        """Save all results to disk including feature names"""
        Path(output_dir).mkdir(exist_ok=True)

        # Save model
        model.save_model(f'{output_dir}/xgb_model.json')

        # Save feature names
        if self.feature_names is not None:
            with open(f'{output_dir}/feature_names.txt', 'w') as f:
                f.write('\n'.join(self.feature_names))

        # Save other artifacts
        joblib.dump(self.best_params, f'{output_dir}/best_params.pkl')

        if self.study:
            joblib.dump(self.study, f'{output_dir}/study.pkl')
            study_df = self.study.trials_dataframe()
            study_df.to_csv(f'{output_dir}/trials.csv', index=False)

def detect_outliers_iqr(df, columns, threshold=1.5):
    """Detect outliers using IQR method"""
    outlier_indices = []
    
    for col in columns:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
            outlier_indices.extend(outliers)
            
            if len(outliers) > 0:
                print(f"  {col}: Found {len(outliers)} outliers "
                      f"({len(outliers)/len(df)*100:.2f}%)")
    
    return list(set(outlier_indices))

def remove_outliers_robust(df, y, columns, method='iqr', threshold=2.0):
    """Robust outlier handling method"""
    print("Starting outlier detection...")
    
    if method == 'iqr':
        outlier_indices = detect_outliers_iqr(df, columns, threshold)
    elif method == 'zscore':
        # Z-score method
        z_scores = np.abs(stats.zscore(df[columns]))
        outlier_indices = np.where(z_scores > threshold)[0]
        outlier_indices = list(outlier_indices)
    else:
        return df, y, []
    
    print(f"\nTotal outliers found: {len(outlier_indices)}")
    
    if len(outlier_indices) > 0:
        df_clean = df.drop(outlier_indices)
        y_clean = y.drop(outlier_indices)
        print(f"Data size after removing outliers: {len(df_clean)} (removed {len(outlier_indices)} samples)")
        return df_clean, y_clean, outlier_indices
    
    print("No outliers found")
    return df, y, []

def check_data_leakage(X_train, X_test, y_train, y_test):
    """Check for data leakage"""
    print("\n=== Data Leakage Check ===")
    
    # Check for overlap between train and test sets
    train_indices = set(X_train.index)
    test_indices = set(X_test.index)
    overlap = train_indices.intersection(test_indices)
    
    if overlap:
        print(f"❌ Data leakage detected: {len(overlap)} overlapping samples between train and test sets")
    else:
        print("✅ No overlap between train and test sets")
    
    # Check feature distributions
    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")
    print(f"Number of features: {X_train.shape[1]}")
    
    return len(overlap) == 0

def safe_feature_engineering(X_train, X_test):
    """Safe feature engineering to avoid data leakage"""
    print("Performing safe feature engineering...")
    
    # 1. Logarithmic transformation
    columns_to_log = ['Lon', 'Lat', 'Age', 'BD', 'pH']
    for col in columns_to_log:
        if col in X_train.columns:
            X_train[col + '_log'] = np.log(X_train[col] + 1e-8)
            X_test[col + '_log'] = np.log(X_test[col] + 1e-8)
            print(f"  Created log feature: {col}_log")
    
    # 2. Binning (based on training set quantiles)
    if 'Altitude' in X_train.columns:
        # Use training set to compute bin boundaries
        alt_bins = pd.cut(X_train['Altitude'], bins=5, retbins=True)[1]
        X_train['Altitude_bins'] = pd.cut(X_train['Altitude'], bins=alt_bins, labels=False)
        X_test['Altitude_bins'] = pd.cut(X_test['Altitude'], bins=alt_bins, labels=False)
        print("  Created binned feature: Altitude_bins")
    
    return X_train, X_test

def augment_continuous_features(X, y, continuous_cols, augmentation_factor=2, noise_scale=0.01, random_state=42):
    """
    Augment continuous features with noise - with proper index resetting 
    to prevent alignment issues during training/evaluation.
    """
    np.random.seed(random_state)
    
    # Reset indices to ensure positional indexing (iloc) matches the index
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    
    n_samples = len(X)
    n_augment = int(n_samples * augmentation_factor)

    augmented_X_list = []
    augmented_y_list = []

    for _ in range(n_augment):
        idx = np.random.randint(0, n_samples)
        base_sample = X.iloc[idx].copy()
        
        # Add noise to specified continuous columns
        for col in continuous_cols:
            if col in base_sample:
                std = X[col].std()
                noise = np.random.normal(loc=0, scale=std * noise_scale)
                
                # Logical check: don't allow negative values for typically positive soil metrics
                if col in ['BD', 'Age', 'Altitude'] or 'log' in col:
                    base_sample[col] = max(1e-8, base_sample[col] + noise)
                else:
                    base_sample[col] += noise

        augmented_X_list.append(base_sample)
        augmented_y_list.append(y.iloc[idx])

    # Efficiently combine original and augmented data
    X_augmented = pd.concat([X, pd.DataFrame(augmented_X_list)], ignore_index=True)
    y_augmented = pd.concat([y, pd.Series(augmented_y_list)], ignore_index=True)

    return X_augmented, y_augmented

def enhanced_load_and_prepare_data(augment=True, augmentation_factor=2, 
                                 lu_type_importance_boost=3.0, 
                                 outlier_threshold=2.0):
    """Fixed data leakage version with outlier handling"""
    
    # Load data
    dtype_dict = {
        'Soillayer': 'int8',
        'yi': 'float32'
    }

    df_clean = pd.read_csv('F:/model/df.clean.yi.csv', dtype=dtype_dict)
    data_predictor_VIF = pd.read_csv('F:/model/data.predictor.VIF.yi.csv')

    # Select target soil layer
    target_layer = "topsoil" ###################################################################################SELECT LAYER
    df = df_clean[df_clean['Soillayer'] == (1 if target_layer == 'topsoil' else 2)].copy()

    # Select features
    predictor_columns = data_predictor_VIF['x'].tolist()
    valid_x_cols = [col for col in predictor_columns if col != "Soillayer" and col in df.columns]

    # Data cleaning
    for col in valid_x_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    X = df[valid_x_cols].astype('float32').fillna(0)
    y = df['yi'].astype('float32')

    print(f"Original data size: {len(X)}")
    
    # Outlier detection and handling
    continuous_cols = ['Lon', 'Lat', 'Age', 'BD', 'pH', 'Altitude']
    continuous_cols = [col for col in continuous_cols if col in X.columns]
    
    print("\n=== Outlier Handling ===")
    X_clean, y_clean, outliers = remove_outliers_robust(
        X, y, continuous_cols, threshold=outlier_threshold
    )

    # Data splitting
    X_train, X_test, y_train, y_test = train_test_split(
        X_clean, y_clean, test_size=0.3, random_state=42
    )

    # Safe feature engineering
    X_train, X_test = safe_feature_engineering(X_train, X_test)

    # Data leakage check
    check_data_leakage(X_train, X_test, y_train, y_test)

    # LUtype feature enhancement (only for training set)
    if 'LUtype' in X_train.columns:
        print(f"\n=== LUtype Feature Enhancement ===")
        print(f"Original LUtype distribution:")
        print(X_train['LUtype'].value_counts().sort_index())
        
        print(f"\nEnhancing LUtype importance with factor: {lu_type_importance_boost}")
        
        # Create LUtype feature enhancements
        for i in range(int(lu_type_importance_boost) - 1):
            lu_type_col_name = f'LUtype_boost_{i+1}'
            X_train[lu_type_col_name] = X_train['LUtype']
            X_test[lu_type_col_name] = X_test['LUtype']
            print(f"  Created LUtype copy: {lu_type_col_name}")
        
        # Create interaction terms between LUtype and other important features
        important_features = ['pH', 'BD', 'Age', 'Altitude']
        for feature in important_features:
            if feature in X_train.columns:
                interaction_name = f'LUtype_{feature}_interaction'
                X_train[interaction_name] = X_train['LUtype'] * X_train[feature]
                X_test[interaction_name] = X_test['LUtype'] * X_test[feature]
                print(f"  Created interaction feature: {interaction_name}")
        
        # Create LUtype squared term (non-linear effect)
        X_train['LUtype_squared'] = X_train['LUtype'] ** 2
        X_test['LUtype_squared'] = X_test['LUtype'] ** 2
        print("  Created LUtype_squared")
    
    # Data augmentation (only for training set)
    if augment:
        continuous_cols = [
            'Lon', 'Lat', 'Age', 'BD', 'pH',
            'Lon_log', 'Lat_log', 'Age_log', 'BD_log', 'pH_log',
            'Altitude'
        ]
        continuous_cols = [col for col in continuous_cols if col in X_train.columns]
        
        print(f"\n=== Data Augmentation ===")
        print(f"Training set size before augmentation: {len(X_train)}")
        X_train, y_train = augment_continuous_features(
            X=X_train,  # Only augment training set
            y=y_train,  # Only augment training set
            continuous_cols=continuous_cols,
            augmentation_factor=augmentation_factor,
            noise_scale=0.005,  
            random_state=42
        )
        print(f"Training set size after augmentation: {len(X_train)}")
    else:
        print("\nSkipping data augmentation")

    print(f"\n=== Final Dataset Statistics ===")
    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")
    print(f"Total number of features: {len(X_train.columns)}")
    
    # Statistical feature distribution
    lu_type_cols = [col for col in X_train.columns if 'LUtype' in col]
    
    print(f"\nFeature statistics:")
    print(f"LUtype-related features: {len(lu_type_cols)}")

    dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=X_train.columns.tolist())
    dtest = xgb.DMatrix(X_test, label=y_test, feature_names=X_train.columns.tolist())

    return dtrain, dtest, X, y, X_test, y_test, X_train, y_train

def main():
    """Main function with data leakage prevention"""
    outlier_threshold = 2.0  # Outlier detection threshold
    
    print("=== Starting Enhanced XGBoost Modeling (Fixed Data Leakage Version) ===")
    
    # Use fixed data processing (with outlier handling and data leakage prevention)
    dtrain, dtest, x1, y1, X_test, y_test, X_train, y_train = enhanced_load_and_prepare_data(
        augment=True,
        augmentation_factor=3,
        lu_type_importance_boost=3.0,
        outlier_threshold=outlier_threshold
    )

    tuner = XGBoostTuner(seed=42, n_jobs=17)

    # 1. Hyperparameter tuning
    print("\n=== Starting Hyperparameter Tuning ===")
    best_params = tuner.tune_hyperparameters(dtrain, n_trials=10)
    print(f"Best parameters: {best_params}")

    # 2. Enhanced Cross-validation with 10x5 folds and model saving
    print("\n=== Starting 10x5 Cross-Validation ===")
    cv_results_summary = tuner.cross_validate(best_params, X_train, y_train, n_splits=5,
                                              cv_models_dir='F:/model/results/cv_models',
                                              n_repeats=10)  # 10 repetitions of 5-fold CV
    
    print("\nSaved CV models:")
    for path in cv_results_summary['detailed_results']['model_paths'][:5]:  # Show first 5
        print(f"- {path}")
    print(f"... and {len(cv_results_summary['detailed_results']['model_paths']) - 5} more")
    
    # Plot enhanced CV results
    print("\nPlotting enhanced cross-validation results...")
    tuner.plot_cv_results(cv_results_summary)

    # 3. Train final model
    optimal_rounds = int(cv_results_summary['best_iterations_mean'])
    print(f"\nTraining final model with {optimal_rounds} rounds...")
    final_model, evals_result = tuner.train_model(best_params, dtrain, num_boost_round=optimal_rounds)

    # Plot learning curves
    print("\nPlotting learning curves...")
    tuner.plot_learning_curves(evals_result, "Final Model")

    # 4. Evaluate and plot final performance
    print("\nEvaluating final model...")
    test_metrics, _ = tuner.evaluate_model(final_model, dtest)
    print("\n=== Final Test Metrics ===")
    print(f"RMSE: {test_metrics['rmse']:.4f}")
    print(f"R²: {test_metrics['r2']:.4f}")
    print(f"MAE: {test_metrics['mae']:.4f}")

    print("\nPlotting final model performance...")
    tuner.plot_final_model_performance(final_model, dtest)

    # 5. Analyze LUtype importance
    print(f"\nAnalyzing LUtype feature importance...")
    importance_df = tuner.plot_feature_importance(final_model, top_n=35)
    
    # Detailed analysis of LUtype importance
    tuner.analyze_lutype_importance(final_model, X_test)
    
    # 6. SHAP analysis
    if SHAP_AVAILABLE:
        print("\nGenerating SHAP summary plot...")
        tuner.plot_shap_summary(final_model, X_test)
        
    # 7. Save results
    print("\nSaving results...")
    tuner.save_results(final_model, 'F:/model/results')
    
    # Save CV results summary
    cv_summary_path = 'F:/model/results/cv_summary.csv'
    cv_results_summary['detailed_results'].to_csv(cv_summary_path, index=False)
    print(f"Saved detailed CV results to: {cv_summary_path}")
    
    print(f"\n=== Model Training Summary ===")
    print("✓ Fixed data leakage issues")
    print("✓ Added outlier detection and handling")
    print("✓ Safe feature engineering (avoided data leakage)")
    print("✓ Created multiple LUtype feature copies")
    print("✓ Added LUtype interaction features with other important variables")
    print("✓ Added LUtype squared term (non-linear effect)")
    print("✓ Improved hyperparameter search space for better handling of categorical features")
    print("✓ Test set uses original data (no augmentation noise)")
    print("✓ Implemented 10 repetitions of 5-fold cross-validation for robust evaluation")
    
    print("\n=== Complete! ===")

if __name__ == "__main__":
    main()

In [ ]:
# XGBoost modeling: soil organic carbon in subsoil
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import joblib
import os
import re
from pathlib import Path
import warnings
import builtins
from scipy import stats
open = builtins.open

warnings.filterwarnings('ignore')

# SHAP availability check
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False

class XGBoostTuner:
    def __init__(self, seed=42, n_jobs=-1):
        self.seed = seed
        self.n_jobs = n_jobs if n_jobs != -1 else None
        self.study = None
        self.best_model = None
        self.best_params = None
        self.feature_names = None  # Store feature names

    @staticmethod
    def clean_numeric_value(value):
        """Convert string values to numeric, handling special cases"""
        if pd.isna(value):
            return np.nan
        if isinstance(value, str):
            # Remove any non-numeric characters except minus, decimal point
            cleaned = re.sub(r'[^\d.-]', '', value)
            try:
                return float(cleaned) if cleaned else np.nan
            except ValueError:
                return np.nan
        return float(value)

    def objective(self, trial, dtrain, num_boost_round=500):
        """Enhanced objective function with stronger overfitting prevention"""
        params = {
            'eta': trial.suggest_float('eta', 0.01, 0.15, log=True),  # 更小的学习率
            'max_depth': trial.suggest_int('max_depth', 2, 4),  # 更浅的树
            'subsample': trial.suggest_float('subsample', 0.5, 0.7),  # 降低采样率
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.7),  # 降低特征采样
            'min_child_weight': trial.suggest_int('min_child_weight', 5, 20),  # 增加限制
            'lambda': trial.suggest_float('lambda', 0.5, 5.0, log=True),  # L2正则化
            'alpha': trial.suggest_float('alpha', 0.5, 5.0, log=True),  # L1正则化
            'gamma': trial.suggest_float('gamma', 0.1, 1.0),  # 增加剪枝强度
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'verbosity': 0,
            'seed': self.seed,
            'nthread': 1
        }

        cv_results = xgb.cv(
            params=params,
            dtrain=dtrain,
            num_boost_round=500,  # 减少迭代轮数
            nfold=5,
            metrics='rmse',
            early_stopping_rounds=20,  # 更早的停止
            as_pandas=True,
            seed=self.seed,
            shuffle=True
        )

        # Strong overfitting penalty
        train_rmse = cv_results['train-rmse-mean'].iloc[-1]
        val_rmse = cv_results['test-rmse-mean'].iloc[-1]
        
        # 计算过拟合程度（更严格的判断）
        overfitting_ratio = max(0, (train_rmse - val_rmse) / train_rmse)
        
        # 更严厉的惩罚
        if overfitting_ratio > 0.05:  # 如果过拟合超过5%
            penalty = (overfitting_ratio - 0.05) * 2.0
            val_rmse += penalty
            
        # 额外惩罚：如果验证误差比训练误差高太多
        if val_rmse > train_rmse * 1.5:  # 验证误差是训练误差的1.5倍以上
            val_rmse *= 1.2
            
        return val_rmse
    
    def tune_hyperparameters(self, dtrain, n_trials=30):
        """Hyperparameter tuning focused on generalization"""
        study = optuna.create_study(direction='minimize')
        
        print("Tuning hyperparameters with aggressive overfitting prevention...")
        print("Using very strong regularization and shallow trees")

        with ThreadPoolExecutor(max_workers=self.n_jobs) as executor:
            futures = [executor.submit(lambda: study.optimize(
                lambda trial: self.objective(trial, dtrain),
                n_trials=10,  # 每批10个试验
                n_jobs=1
            )) for _ in range(max(1, n_trials//10))]

            for future in futures:
                future.result()

        self.study = study
        self.best_params = study.best_params
        self.best_params.update({
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'seed': self.seed,
            'nthread': self.n_jobs
        })

        print(f"Best parameters found: {self.best_params}")
        return self.best_params

    def train_model(self, params, dtrain, dvalid=None, num_boost_round=500):
        """Train model with focus on preventing overfitting"""
        evals = [(dtrain, 'train')]
        if dvalid is not None:
            evals.append((dvalid, 'valid'))

        # 确保使用强正则化参数
        if 'eta' not in params:
            params['eta'] = 0.05
        if 'max_depth' not in params:
            params['max_depth'] = 3
        if 'subsample' not in params:
            params['subsample'] = 0.6
        if 'colsample_bytree' not in params:
            params['colsample_bytree'] = 0.6

        evals_result = {}
        model = xgb.train(
            params=params,
            dtrain=dtrain,
            num_boost_round=num_boost_round,
            evals=evals,
            early_stopping_rounds=30,  # 更保守的早停
            verbose_eval=50,  # 每50轮输出一次
            evals_result=evals_result
        )

        self.feature_names = dtrain.feature_names
        if self.feature_names is None:
            self.feature_names = [f'f{i}' for i in range(dtrain.num_col())]
            model.feature_names = self.feature_names

        return model, evals_result

    def cross_validate(self, params, X, y, n_splits=5, cv_models_dir='cv_models', n_repeats=10):
        """Simplified cross-validation method"""
        Path(cv_models_dir).mkdir(parents=True, exist_ok=True)

        results = {
            'train_rmse': [],
            'val_rmse': [],
            'train_r2': [],
            'val_r2': [],
            'best_iterations': []
        }

        print(f"Performing {n_repeats} repeats of {n_splits}-fold cross-validation...")
        
        for repeat in range(n_repeats):
            kf = KFold(n_splits=n_splits, shuffle=True, random_state=self.seed + repeat)
            
            for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
                X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
                y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

                # 准备DMatrix
                dtrain_fold = xgb.DMatrix(X_train_fold, label=y_train_fold, 
                                        feature_names=X_train_fold.columns.tolist())
                dval_fold = xgb.DMatrix(X_val_fold, label=y_val_fold,
                                      feature_names=X_train_fold.columns.tolist())

                # 训练模型
                model, evals_result = self.train_model(params, dtrain_fold, dval_fold, num_boost_round=200)

                # 预测和评估
                train_pred = model.predict(dtrain_fold)
                val_pred = model.predict(dval_fold)

                # 记录结果
                results['train_rmse'].append(np.sqrt(mean_squared_error(y_train_fold, train_pred)))
                results['val_rmse'].append(np.sqrt(mean_squared_error(y_val_fold, val_pred)))
                results['train_r2'].append(r2_score(y_train_fold, train_pred))
                results['val_r2'].append(r2_score(y_val_fold, val_pred))
                results['best_iterations'].append(model.best_iteration if hasattr(model, 'best_iteration') else 200)

                # 保存模型
                model_path = os.path.join(cv_models_dir, f'model_repeat_{repeat}_fold_{fold}.json')
                model.save_model(model_path)

        # 计算汇总统计
        cv_summary = {
            'train_rmse_mean': np.mean(results['train_rmse']),
            'train_rmse_std': np.std(results['train_rmse']),
            'val_rmse_mean': np.mean(results['val_rmse']),
            'val_rmse_std': np.std(results['val_rmse']),
            'train_r2_mean': np.mean(results['train_r2']),
            'train_r2_std': np.std(results['train_r2']),
            'val_r2_mean': np.mean(results['val_r2']),
            'val_r2_std': np.std(results['val_r2']),
            'best_iterations_mean': np.mean(results['best_iterations']),
            'best_iterations_std': np.std(results['best_iterations']),
            'detailed_results': pd.DataFrame({
                'train_rmse': results['train_rmse'],
                'val_rmse': results['val_rmse'],
                'train_r2': results['train_r2'],
                'val_r2': results['val_r2'],
                'best_iteration': results['best_iterations']
            })
        }

        print(f"\nCross-validation completed: {len(results['train_rmse'])} folds total")
        return cv_summary

    def evaluate_model(self, model, dtest):
        """Evaluate model performance on test set"""
        test_pred = model.predict(dtest)
        y_true = dtest.get_label()
        return {
            'rmse': np.sqrt(mean_squared_error(y_true, test_pred)),
            'r2': r2_score(y_true, test_pred),
            'mae': mean_absolute_error(y_true, test_pred)
        }, test_pred

    def plot_learning_curves(self, evals_result, title_suffix):
        """Plot training and validation metrics"""
        plt.figure(figsize=(10, 5))
        epochs = len(evals_result['train']['rmse'])
        plt.plot(range(epochs), evals_result['train']['rmse'], label='Train RMSE')
        if 'valid' in evals_result:
            plt.plot(range(epochs), evals_result['valid']['rmse'], label='Validation RMSE')
        plt.legend()
        plt.xlabel('Iterations')
        plt.ylabel('RMSE')
        plt.title(f'Learning Curves - {title_suffix}')
        plt.grid(True, alpha=0.3)
        plt.show()

    def plot_feature_importance(self, model, importance_type='weight', top_n=20):
        """Plot feature importance"""
        try:
            importance_dict = model.get_score(importance_type=importance_type)
            
            if not importance_dict:
                print("No feature importance data available.")
                return None
                
            importance_df = pd.DataFrame({
                'feature': list(importance_dict.keys()),
                'importance': list(importance_dict.values())
            }).sort_values('importance', ascending=True)
            
            if len(importance_df) > top_n:
                importance_df = importance_df.tail(top_n)
            
            plt.figure(figsize=(10, max(6, len(importance_df) * 0.2)))
            y_pos = np.arange(len(importance_df))
            plt.barh(y_pos, importance_df['importance'], color='lightblue', alpha=0.8)
            plt.yticks(y_pos, importance_df['feature'])
            plt.xlabel('Importance')
            plt.title(f'Feature Importance ({importance_type}) - Top {top_n}')
            plt.grid(True, axis='x', alpha=0.3)
            plt.tight_layout()
            plt.show()
            
            return importance_df
        except Exception as e:
            print(f"Error plotting feature importance: {e}")
            return None

    def save_results(self, model, output_dir='results'):
        """Save all results to disk"""
        Path(output_dir).mkdir(exist_ok=True)

        # Save model
        model.save_model(f'{output_dir}/xgb_model.json')

        # Save feature names
        if self.feature_names is not None:
            with open(f'{output_dir}/feature_names.txt', 'w') as f:
                f.write('\n'.join(self.feature_names))

        # Save parameters
        if self.best_params:
            joblib.dump(self.best_params, f'{output_dir}/best_params.pkl')

        print(f"Results saved to: {output_dir}")

# 辅助函数
def detect_outliers_iqr(df, columns, threshold=1.5):
    """Detect outliers using IQR method"""
    outlier_indices = []
    
    for col in columns:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
            outlier_indices.extend(outliers)
            
            if len(outliers) > 0:
                print(f"  {col}: Found {len(outliers)} outliers "
                      f"({len(outliers)/len(df)*100:.2f}%)")
    
    return list(set(outlier_indices))

def remove_outliers_robust(df, y, columns, method='iqr', threshold=2.0):
    """Robust outlier handling method"""
    print("Starting outlier detection...")
    
    if method == 'iqr':
        outlier_indices = detect_outliers_iqr(df, columns, threshold)
    elif method == 'zscore':
        # Z-score method
        z_scores = np.abs(stats.zscore(df[columns]))
        outlier_indices = np.where(z_scores > threshold)[0]
        outlier_indices = list(outlier_indices)
    else:
        return df, y, []
    
    print(f"\nTotal outliers found: {len(outlier_indices)}")
    
    if len(outlier_indices) > 0:
        df_clean = df.drop(outlier_indices)
        y_clean = y.drop(outlier_indices)
        print(f"Data size after removing outliers: {len(df_clean)} (removed {len(outlier_indices)} samples)")
        return df_clean, y_clean, outlier_indices
    
    print("No outliers found")
    return df, y, []

def check_data_leakage(X_train, X_test, y_train, y_test):
    """Check for data leakage"""
    print("\n=== Data Leakage Check ===")
    
    # Check for overlap between train and test sets
    train_indices = set(X_train.index)
    test_indices = set(X_test.index)
    overlap = train_indices.intersection(test_indices)
    
    if overlap:
        print(f"❌ Data leakage detected: {len(overlap)} overlapping samples between train and test sets")
    else:
        print("✅ No overlap between train and test sets")
    
    # Check feature distributions
    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")
    print(f"Number of features: {X_train.shape[1]}")
    
    return len(overlap) == 0

def safe_feature_engineering_simple(X_train, X_test):
    """Minimal feature engineering to avoid overfitting"""
    print("Performing minimal feature engineering...")
    
    # 只做最基本的转换
    columns_to_log = []
    for col in ['BD', 'Age']:
        if col in X_train.columns:
            X_train[col + '_log'] = np.log(X_train[col] + 1e-8)
            X_test[col + '_log'] = np.log(X_test[col] + 1e-8)
            print(f"  Created log feature: {col}_log")
    
    return X_train, X_test

def augment_continuous_features(X, y, continuous_cols, augmentation_factor=1.0, noise_scale=0.002, random_state=42):
    """
    Reduced augmentation with proper index resetting
    """
    if augmentation_factor <= 0:
        return X, y
    
    np.random.seed(random_state)
    
    # Reset indices
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    
    n_samples = len(X)
    n_augment = int(n_samples * augmentation_factor)

    if n_augment == 0:
        return X, y

    augmented_X_list = []
    augmented_y_list = []

    for _ in range(n_augment):
        idx = np.random.randint(0, n_samples)
        base_sample = X.iloc[idx].copy()
        
        # Add minimal noise to specified continuous columns
        for col in continuous_cols:
            if col in base_sample:
                std = X[col].std()
                noise = np.random.normal(loc=0, scale=std * noise_scale)
                base_sample[col] += noise

        augmented_X_list.append(base_sample)
        augmented_y_list.append(y.iloc[idx])

    # Combine data
    X_augmented = pd.concat([X, pd.DataFrame(augmented_X_list)], ignore_index=True)
    y_augmented = pd.concat([y, pd.Series(augmented_y_list)], ignore_index=True)

    return X_augmented, y_augmented

def enhanced_load_and_prepare_data(augment=True, augmentation_factor=1.5, 
                                 lu_type_importance_boost=1.0,
                                 outlier_threshold=2.0,
                                 use_simple_features=True):
    """Data loading with overfitting prevention"""
    
    # Load data
    dtype_dict = {
        'Soillayer': 'int8',
        'yi': 'float32'
    }

    df_clean = pd.read_csv('F:/model/df.clean.yi.csv', dtype=dtype_dict)
    data_predictor_VIF = pd.read_csv('F:/model/data.predictor.VIF.yi.csv')

    # Select target soil layer
    target_layer = "subsoil"
    df = df_clean[df_clean['Soillayer'] == (1 if target_layer == 'topsoil' else 2)].copy()

    # Select features
    predictor_columns = data_predictor_VIF['x'].tolist()
    valid_x_cols = [col for col in predictor_columns if col != "Soillayer" and col in df.columns]
    
    # 如果是简单特征模式，只保留原始特征
    if use_simple_features:
        print("Using simple feature set (no engineered features)")
        valid_x_cols = [col for col in valid_x_cols if '_log' not in col and '_bins' not in col 
                       and 'boost' not in col and 'interaction' not in col and 'squared' not in col]

    # Data cleaning
    for col in valid_x_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    X = df[valid_x_cols].astype('float32').fillna(0)
    y = df['yi'].astype('float32')

    print(f"Original data size: {len(X)}")
    print(f"Number of features: {len(X.columns)}")
    
    # Outlier detection and handling
    continuous_cols = ['Lon', 'Lat', 'Age', 'BD', 'pH', 'Altitude']
    continuous_cols = [col for col in continuous_cols if col in X.columns]
    
    print("\n=== Outlier Handling ===")
    X_clean, y_clean, outliers = remove_outliers_robust(
        X, y, continuous_cols, threshold=outlier_threshold
    )

    # Data splitting
    X_train, X_test, y_train, y_test = train_test_split(
        X_clean, y_clean, test_size=0.3, random_state=42
    )

    # 可选：简化的特征工程
    if not use_simple_features:
        X_train, X_test = safe_feature_engineering_simple(X_train, X_test)
    
    # Data leakage check
    check_data_leakage(X_train, X_test, y_train, y_test)

    # 简化LUtype处理
    if 'LUtype' in X_train.columns and not use_simple_features:
        print(f"\n=== Simple LUtype Feature Enhancement ===")
        print(f"Original LUtype distribution:")
        print(X_train['LUtype'].value_counts().sort_index())
        
        # 只创建一个副本，而不是多个
        X_train['LUtype_copy'] = X_train['LUtype']
        X_test['LUtype_copy'] = X_test['LUtype']
        print("  Created single LUtype copy")
    
    # 减少数据增强
    if augment and not use_simple_features:
        continuous_cols = [
            'Lon', 'Lat', 'Age', 'BD', 'pH', 'Altitude'
        ]
        continuous_cols = [col for col in continuous_cols if col in X_train.columns]
        
        print(f"\n=== Reduced Data Augmentation ===")
        print(f"Training set size before augmentation: {len(X_train)}")
        X_train, y_train = augment_continuous_features(
            X=X_train,
            y=y_train,
            continuous_cols=continuous_cols,
            augmentation_factor=augmentation_factor,
            noise_scale=0.002,
            random_state=42
        )
        print(f"Training set size after augmentation: {len(X_train)}")
    elif augment and use_simple_features:
        print("\nSkipping data augmentation for simple feature mode")

    print(f"\n=== Final Dataset Statistics ===")
    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")
    print(f"Total number of features: {len(X_train.columns)}")
    
    dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=X_train.columns.tolist())
    dtest = xgb.DMatrix(X_test, label=y_test, feature_names=X_train.columns.tolist())

    return dtrain, dtest, X, y, X_test, y_test, X_train, y_train

def main():
    """Main function with overfitting prevention"""
    outlier_threshold = 1.5  # 更宽松的异常值阈值
    
    print("=== Starting XGBoost Modeling with Overfitting Prevention ===")
    
    # 尝试两种模式：简单特征和增强特征
    for mode in ['simple', 'enhanced']:
        print(f"\n{'='*60}")
        print(f"Running in {mode.upper()} mode")
        print(f"{'='*60}")
        
        use_simple = (mode == 'simple')
        
        # 加载数据
        dtrain, dtest, x1, y1, X_test, y_test, X_train, y_train = enhanced_load_and_prepare_data(
            augment=(mode == 'enhanced'),
            augmentation_factor=1.5 if mode == 'enhanced' else 0,
            lu_type_importance_boost=1.0 if use_simple else 2.0,
            outlier_threshold=outlier_threshold,
            use_simple_features=use_simple
        )

        tuner = XGBoostTuner(seed=42, n_jobs=17)

        # 1. Hyperparameter tuning
        print("\n=== Starting Hyperparameter Tuning ===")
        best_params = tuner.tune_hyperparameters(dtrain, n_trials=30)
        print(f"Best parameters: {best_params}")

        # 2. Cross-validation
        print("\n=== Starting 5x5 Cross-Validation ===")
        cv_results_summary = tuner.cross_validate(best_params, X_train, y_train, n_splits=5,
                                                  cv_models_dir=f'F:/model/results/cv_models_{mode}',
                                                  n_repeats=10)
        
        # 分析过拟合程度
        train_val_gap = cv_results_summary['val_rmse_mean'] - cv_results_summary['train_rmse_mean']
        overfitting_ratio = train_val_gap / cv_results_summary['train_rmse_mean']
        
        print(f"\n=== Overfitting Analysis ===")
        print(f"Train RMSE: {cv_results_summary['train_rmse_mean']:.4f} (±{cv_results_summary['train_rmse_std']:.4f})")
        print(f"Val RMSE: {cv_results_summary['val_rmse_mean']:.4f} (±{cv_results_summary['val_rmse_std']:.4f})")
        print(f"Train-Val RMSE gap: {train_val_gap:.4f}")
        print(f"Overfitting ratio: {overfitting_ratio:.2%}")
        
        if overfitting_ratio > 0.3:
            print("⚠️  Warning: Significant overfitting detected!")
        
        # 3. Train final model
        optimal_rounds = max(50, int(cv_results_summary['best_iterations_mean'] * 0.8))
        print(f"\nTraining final model with {optimal_rounds} rounds...")
        final_model, evals_result = tuner.train_model(best_params, dtrain, num_boost_round=optimal_rounds)

        # 4. Evaluate
        print("\nEvaluating final model...")
        test_metrics, _ = tuner.evaluate_model(final_model, dtest)
        
        print(f"\n=== Results for {mode.upper()} mode ===")
        print(f"CV Train RMSE: {cv_results_summary['train_rmse_mean']:.4f}")
        print(f"CV Val RMSE: {cv_results_summary['val_rmse_mean']:.4f}")
        print(f"Test RMSE: {test_metrics['rmse']:.4f}")
        print(f"Test R²: {test_metrics['r2']:.4f}")
        print(f"Generalization gap (Test - CV Val): {test_metrics['rmse'] - cv_results_summary['val_rmse_mean']:.4f}")
        
        # 保存结果
        output_dir = f'F:/model/results/{mode}'
        tuner.save_results(final_model, output_dir)
        
        # 绘制特征重要性
        print("\nPlotting feature importance...")
        importance_df = tuner.plot_feature_importance(final_model, top_n=20)
        
        # 分析特征
        if importance_df is not None:
            print(f"\nTop 5 most important features:")
            top_features = importance_df.sort_values('importance', ascending=False).head(5)
            for idx, row in top_features.iterrows():
                print(f"  {row['feature']}: {row['importance']:.4f}")
        
        print(f"\nCompleted {mode.upper()} mode")
        print(f"{'='*60}\n")
    
    print("=== All modes completed ===")
    print("\nRecommendation:")
    print("1. Compare the results from both modes")
    print("2. Choose the model with better generalization (smaller Test-CV gap)")
    print("3. Consider using simpler model if overfitting is severe")

if __name__ == "__main__":
    main()

In [ ]:
# XGBoost modeling: passive fractions  in topsoil
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import joblib
import os
import re
from pathlib import Path
import warnings
import builtins
from scipy import stats
open = builtins.open

warnings.filterwarnings('ignore')

# SHAP availability check
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False

class XGBoostTuner:
    def __init__(self, seed=42, n_jobs=-1):
        self.seed = seed
        self.n_jobs = n_jobs if n_jobs != -1 else None
        self.study = None
        self.best_model = None
        self.best_params = None
        self.feature_names = None  # Store feature names

    @staticmethod
    def clean_numeric_value(value):
        """Convert string values to numeric, handling special cases"""
        if pd.isna(value):
            return np.nan
        if isinstance(value, str):
            # Remove any non-numeric characters except minus, decimal point
            cleaned = re.sub(r'[^\d.-]', '', value)
            try:
                return float(cleaned) if cleaned else np.nan
            except ValueError:
                return np.nan
        return float(value)

    def objective(self, trial, dtrain, num_boost_round=500):
        """Objective function with enhanced regularization"""
        params = {
            'eta': trial.suggest_float('eta', 0.005, 0.1, log=True),  # Lower learning rate
            'max_depth': trial.suggest_int('max_depth', 2, 4),  # Shallower trees
            'subsample': trial.suggest_float('subsample', 0.4, 0.7),  # Lower subsampling
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 0.7),  # Lower feature sampling
           # 'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.4, 0.7),
            'min_child_weight': trial.suggest_int('min_child_weight', 10, 20),  # Higher min child weight
            'lambda': trial.suggest_float('lambda', 10, 25.0),  # Stronger L2 regularization
            'alpha': trial.suggest_float('alpha', 5.0, 15.0),  # Stronger L1 regularization
            'gamma': trial.suggest_float('gamma', 0.5, 2.0),  # Higher pruning parameter
            'max_delta_step': trial.suggest_int('max_delta_step', 0, 1),
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'verbosity': 0,
            'seed': self.seed,
            'nthread': 1
        }

        cv_results = xgb.cv(
            params=params,
            dtrain=dtrain,
            num_boost_round=800,  # More rounds but more conservative early stopping
            nfold=5,
            metrics='rmse',
            early_stopping_rounds=25,  # More conservative early stopping
            as_pandas=True,
            seed=self.seed,
            shuffle=True
        )

        # Stronger overfitting penalty
        train_rmse = cv_results['train-rmse-mean'].iloc[-1]
        val_rmse = cv_results['test-rmse-mean'].iloc[-1]
        
        # Calculate overfitting degree
        overfitting_ratio = (train_rmse - val_rmse) / train_rmse
        gap_penalty = max(0, overfitting_ratio) * 0.3  # Stronger penalty
        
        # Add extra penalty if severe overfitting
        if overfitting_ratio > 0.1:
            gap_penalty += (overfitting_ratio - 0.1) * 0.5
            
        return val_rmse + gap_penalty
    
    def tune_hyperparameters(self, dtrain, n_trials=10):  # Increased number of trials
        """Hyperparameter tuning focused on generalization"""
        study = optuna.create_study(direction='minimize')
        
        print("Tuning hyperparameters with aggressive overfitting prevention...")
        print("Using very strong regularization and shallow trees")

        with ThreadPoolExecutor(max_workers=self.n_jobs) as executor:
            futures = [executor.submit(lambda: study.optimize(
                lambda trial: self.objective(trial, dtrain),
                n_trials=5,
                n_jobs=1
            )) for _ in range(n_trials//5)]

            for future in futures:
                future.result()

        self.study = study
        self.best_params = study.best_params
        self.best_params.update({
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'seed': self.seed,
            'nthread': self.n_jobs
        })

        print(f"Best parameters found: {self.best_params}")
        return self.best_params


    def train_model(self, params, dtrain, dvalid=None, num_boost_round=500):
        """Train model with focus on preventing overfitting"""
        evals = [(dtrain, 'train')]
        if dvalid is not None:
            evals.append((dvalid, 'valid'))

        evals_result = {}
        model = xgb.train(
            params=params,
            dtrain=dtrain,
            num_boost_round=num_boost_round,
            evals=evals,
            early_stopping_rounds=25,  # More conservative early stopping
            verbose_eval=100,  # Reduced output frequency
            evals_result=evals_result
        )

        self.feature_names = dtrain.feature_names
        if self.feature_names is None:
            self.feature_names = [f'f{i}' for i in range(dtrain.num_col())]
            model.feature_names = self.feature_names

        return model, evals_result

    def cross_validate(self, params, X, y, n_splits=5, cv_models_dir='cv_models', n_repeats=10): 
        """Enhanced CV with 10 repetitions of 5-fold cross-validation and model saving"""
        Path(cv_models_dir).mkdir(parents=True, exist_ok=True)

        all_cv_results = {
            'train_rmse': [], 'val_rmse': [],
            'train_r2': [], 'val_r2': [],
            'best_iterations': [],
            'model_paths': [],
            'feature_names': [],
            'repeat': [],
            'fold': []
        }

        def process_fold(repeat_idx, fold_idx, train_idx, val_idx):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=X_train.columns.tolist())
            dval = xgb.DMatrix(X_val, label=y_val, feature_names=X_train.columns.tolist())

            model, _ = self.train_model(params, dtrain, dval)

            # Save the CV model
            model_path = os.path.join(cv_models_dir, f'cv_model_repeat_{repeat_idx}_fold_{fold_idx}.json')
            model.save_model(model_path)

            train_pred = model.predict(dtrain)
            val_pred = model.predict(dval)

            return (
                np.sqrt(mean_squared_error(y_train, train_pred)),
                np.sqrt(mean_squared_error(y_val, val_pred)),
                r2_score(y_train, train_pred),
                r2_score(y_val, val_pred),
                model.best_iteration if hasattr(model, 'best_iteration') else params.get('num_boost_round', 500),
                model_path,
                model.feature_names,
                repeat_idx,
                fold_idx
            )
        
        with ThreadPoolExecutor(max_workers=self.n_jobs) as executor:
            futures = []
            for repeat_idx in range(n_repeats):
                kf = KFold(n_splits=n_splits, shuffle=True, random_state=self.seed + repeat_idx)
                for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X)):
                    futures.append(executor.submit(
                        process_fold, repeat_idx, fold_idx, train_idx, val_idx
                    ))

            for future in futures:
                train_rmse, val_rmse, train_r2, val_r2, best_iter, model_path, feature_names, repeat_idx, fold_idx = future.result()
                all_cv_results['train_rmse'].append(train_rmse)
                all_cv_results['val_rmse'].append(val_rmse)
                all_cv_results['train_r2'].append(train_r2)
                all_cv_results['val_r2'].append(val_r2)
                all_cv_results['best_iterations'].append(best_iter)
                all_cv_results['model_paths'].append(model_path)
                all_cv_results['feature_names'].append(feature_names)
                all_cv_results['repeat'].append(repeat_idx)
                all_cv_results['fold'].append(fold_idx)
        # Create summary statistics
        cv_results_summary = {
            'train_rmse_mean': np.mean(all_cv_results['train_rmse']),
            'train_rmse_std': np.std(all_cv_results['train_rmse']),
            'val_rmse_mean': np.mean(all_cv_results['val_rmse']),
            'val_rmse_std': np.std(all_cv_results['val_rmse']),
            'train_r2_mean': np.mean(all_cv_results['train_r2']),
            'train_r2_std': np.std(all_cv_results['train_r2']),
            'val_r2_mean': np.mean(all_cv_results['val_r2']),
            'val_r2_std': np.std(all_cv_results['val_r2']),
            'best_iterations_mean': np.mean(all_cv_results['best_iterations']),
            'best_iterations_std': np.std(all_cv_results['best_iterations']),
            'detailed_results': pd.DataFrame(all_cv_results)
        }

        return cv_results_summary

    def evaluate_model(self, model, dtest):
        """Evaluate model performance on test set"""
        test_pred = model.predict(dtest)
        return {
            'rmse': np.sqrt(mean_squared_error(dtest.get_label(), test_pred)),
            'r2': r2_score(dtest.get_label(), test_pred),
            'mae': mean_absolute_error(dtest.get_label(), test_pred)
        }, test_pred

    def plot_learning_curves(self, evals_result, title_suffix):
        """Plot training and validation metrics"""
        plt.figure(figsize=(10, 5))
        epochs = len(evals_result['train']['rmse'])
        plt.plot(range(epochs), evals_result['train']['rmse'], label='Train RMSE')
        if 'valid' in evals_result:
            plt.plot(range(epochs), evals_result['valid']['rmse'], label='Validation RMSE')
        plt.legend()
        plt.xlabel('Iterations')
        plt.ylabel('RMSE')
        plt.title(f'Learning Curves - {title_suffix}')
        plt.show()

    def plot_cv_results(self, cv_results_summary):
        """Plot enhanced cross-validation results with both original and new visualization"""
        # Check if it's the new summary format or old list format
        if isinstance(cv_results_summary, dict) and 'detailed_results' in cv_results_summary:
            # New format - use the detailed results
            detailed_results = cv_results_summary['detailed_results']
            
            # Create two visualizations:
            # 1. Original visualization (simple line plot for first repeat)
            plt.figure(figsize=(15, 10))
            
            # Plot 1: Original style - first repeat only
            plt.subplot(2, 2, 1)
            first_repeat = detailed_results[detailed_results['repeat'] == 0]
            plt.plot(first_repeat['train_rmse'], 'o-', label='Train RMSE')
            plt.plot(first_repeat['val_rmse'], 'o-', label='Validation RMSE')
            plt.xlabel('Fold')
            plt.ylabel('RMSE')
            plt.title('Cross-Validation RMSE (First Repeat)')
            plt.legend()
            plt.grid(True)

            # Plot 2: Original style - first repeat R²
            plt.subplot(2, 2, 2)
            plt.plot(first_repeat['train_r2'], 'o-', label='Train R²')
            plt.plot(first_repeat['val_r2'], 'o-', label='Validation R²')
            plt.xlabel('Fold')
            plt.ylabel('R² Score')
            plt.title('Cross-Validation R² Scores (First Repeat)')
            plt.legend()
            plt.grid(True)

            # Plot 3: New visualization - RMSE distribution across all repeats
            plt.subplot(2, 2, 3)
            plt.boxplot([detailed_results['train_rmse'], detailed_results['val_rmse']], 
                       labels=['Train RMSE', 'Validation RMSE'])
            plt.ylabel('RMSE')
            plt.title('RMSE Distribution across 10x5 CV')
            plt.grid(True, alpha=0.3)
            
            # Plot 4: New visualization - R² distribution across all repeats
            plt.subplot(2, 2, 4)
            plt.boxplot([detailed_results['train_r2'], detailed_results['val_r2']], 
                       labels=['Train R²', 'Validation R²'])
            plt.ylabel('R² Score')
            plt.title('R² Distribution across 10x5 CV')
            plt.grid(True, alpha=0.3)

            plt.tight_layout()
            plt.show()
            
            # Print summary statistics
            print("\n=== 10x5 Cross-Validation Summary ===")
            print(f"Average Train RMSE: {cv_results_summary['train_rmse_mean']:.4f} (±{cv_results_summary['train_rmse_std']:.4f})")
            print(f"Average Validation RMSE: {cv_results_summary['val_rmse_mean']:.4f} (±{cv_results_summary['val_rmse_std']:.4f})")
            print(f"Average Train R²: {cv_results_summary['train_r2_mean']:.4f} (±{cv_results_summary['train_r2_std']:.4f})")
            print(f"Average Validation R²: {cv_results_summary['val_r2_mean']:.4f} (±{cv_results_summary['val_r2_std']:.4f})")
            print(f"Average Best Iterations: {cv_results_summary['best_iterations_mean']:.1f} (±{cv_results_summary['best_iterations_std']:.1f})")
            
            # Also print the format that main() expects
            print("\n=== 交叉验证结果 (旧格式兼容) ===")
            print(f"平均训练RMSE: {cv_results_summary['train_rmse_mean']:.4f} (±{cv_results_summary['train_rmse_std']:.4f})")
            print(f"平均验证RMSE: {cv_results_summary['val_rmse_mean']:.4f} (±{cv_results_summary['val_rmse_std']:.4f})")
            print(f"平均训练R²: {cv_results_summary['train_r2_mean']:.4f} (±{cv_results_summary['train_r2_std']:.4f})")
            print(f"平均验证R²: {cv_results_summary['val_r2_mean']:.4f} (±{cv_results_summary['val_r2_std']:.4f})")
            
        else:
            # Old format - keep original plotting
            plt.figure(figsize=(15, 5))

            # Plot RMSE
            plt.subplot(1, 2, 1)
            plt.plot(cv_results_summary['train_rmse'], 'o-', label='Train RMSE')
            plt.plot(cv_results_summary['val_rmse'], 'o-', label='Validation RMSE')
            plt.xlabel('Fold')
            plt.ylabel('RMSE')
            plt.title('Cross-Validation RMSE')
            plt.legend()
            plt.grid(True)

            # Plot R2
            plt.subplot(1, 2, 2)
            plt.plot(cv_results_summary['train_r2'], 'o-', label='Train R²')
            plt.plot(cv_results_summary['val_r2'], 'o-', label='Validation R²')
            plt.xlabel('Fold')
            plt.ylabel('R² Score')
            plt.title('Cross-Validation R² Scores')
            plt.legend()
            plt.grid(True)

            plt.tight_layout()
            plt.show()

    def plot_final_model_performance(self, model, dtest):
        """Plot actual vs predicted values for test set"""
        test_metrics, test_pred = self.evaluate_model(model, dtest)
        y_test = dtest.get_label()

        plt.figure(figsize=(12, 5))

        # Scatter plot of actual vs predicted
        plt.subplot(1, 2, 1)
        plt.scatter(y_test, test_pred, alpha=0.5)
        plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--r')
        plt.xlabel('Actual Values')
        plt.ylabel('Predicted Values')
        plt.title(f'Actual vs Predicted (R² = {test_metrics["r2"]:.3f})')

        # Residual plot
        plt.subplot(1, 2, 2)
        residuals = y_test - test_pred
        plt.scatter(test_pred, residuals, alpha=0.5)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.xlabel('Predicted Values')
        plt.ylabel('Residuals')
        plt.title('Residual Plot')
        plt.tight_layout()
        plt.show()

    def plot_feature_importance(self, model, importance_type='weight', top_n=30):
        """Plot feature importance without category-specific highlighting"""
        fig, ax = plt.subplots(figsize=(12, 10))
        
    
        importance_dict = model.get_score(importance_type=importance_type)
        
        if not importance_dict:
            print("No feature importance data available.")
            return None
            

        importance_df = pd.DataFrame({
            'feature': list(importance_dict.keys()),
            'importance': list(importance_dict.values())
        }).sort_values('importance', ascending=True)
        

        if len(importance_df) > top_n:
            importance_df = importance_df.tail(top_n)
        

        y_pos = np.arange(len(importance_df))
        colors = ['lightblue'] * len(importance_df)
        
        ax.barh(y_pos, importance_df['importance'], color=colors, alpha=0.8)
        ax.set_yticks(y_pos)
        ax.set_yticklabels(importance_df['feature'])
        ax.set_xlabel('Importance')
        
        title = f'Feature Importance ({importance_type})'
        ax.set_title(title)
        

        lu_type_features = importance_df[importance_df['feature'].str.contains('LUtype', na=False)]
        lu_type_importance = lu_type_features['importance'].sum() if not lu_type_features.empty else 0
        
        ax.text(0.7, 0.95, f'LUtype total: {lu_type_importance:.3f}', 
                transform=ax.transAxes, fontsize=10, bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow"))
        
        plt.tight_layout()
        plt.show()
        
        return importance_df

    def analyze_lutype_importance(self, model, X):
     
        importance_dict = model.get_score(importance_type='weight')
        
        if not importance_dict:
            print("No feature importance data available.")
            return
            

        lutype_features = {k: v for k, v in importance_dict.items() if 'LUtype' in k}
        other_features = {k: v for k, v in importance_dict.items() if 'LUtype' not in k}
        
        print(f"\n=== LUtype Feature Importance Analysis ===")
        print(f"Total LUtype-related features: {len(lutype_features)}")
        print(f"Total importance of LUtype features: {sum(lutype_features.values()):.3f}")
        print(f"Average importance of LUtype features: {np.mean(list(lutype_features.values())) if lutype_features else 0:.3f}")
        
        total_importance = sum(importance_dict.values())
        lu_type_percentage = (sum(lutype_features.values()) / total_importance * 100) if total_importance > 0 else 0
        
        print(f"\nLUtype importance percentage: {lu_type_percentage:.2f}%")
        
     
        if lutype_features:
            sorted_lutype = sorted(lutype_features.items(), key=lambda x: x[1], reverse=True)
            print(f"\nTop LUtype-related features:")
            for feature, importance in sorted_lutype[:10]:
                print(f"  {feature}: {importance:.4f}")
        else:
            print("\nNo LUtype-related features found in importance scores.")

    def plot_shap_summary(self, model, X, feature_names=None):
        """Generate SHAP summary plot if SHAP is available"""
        if SHAP_AVAILABLE:
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X)

            plt.figure(figsize=(10, 8))
            shap.summary_plot(shap_values, X, feature_names=feature_names)
            plt.tight_layout()
            plt.show()
        else:
            print("SHAP not available. Install with: pip install shap")

    def save_results(self, model, output_dir='results'):
        """Save all results to disk including feature names"""
        Path(output_dir).mkdir(exist_ok=True)

        # Save model
        model.save_model(f'{output_dir}/xgb_model.json')

        # Save feature names
        if self.feature_names is not None:
            with open(f'{output_dir}/feature_names.txt', 'w') as f:
                f.write('\n'.join(self.feature_names))

        # Save other artifacts
        joblib.dump(self.best_params, f'{output_dir}/best_params.pkl')

        if self.study:
            joblib.dump(self.study, f'{output_dir}/study.pkl')
            study_df = self.study.trials_dataframe()
            study_df.to_csv(f'{output_dir}/trials.csv', index=False)

def detect_outliers_iqr(df, columns, threshold=1.5):
    """Detect outliers using IQR method"""
    outlier_indices = []
    
    for col in columns:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
            outlier_indices.extend(outliers)
            
            if len(outliers) > 0:
                print(f"  {col}: Found {len(outliers)} outliers "
                      f"({len(outliers)/len(df)*100:.2f}%)")
    
    return list(set(outlier_indices))

def remove_outliers_robust(df, y, columns, method='iqr', threshold=2.0):
    """Robust outlier handling method"""
    print("Starting outlier detection...")
    
    if method == 'iqr':
        outlier_indices = detect_outliers_iqr(df, columns, threshold)
    elif method == 'zscore':
        # Z-score method
        z_scores = np.abs(stats.zscore(df[columns]))
        outlier_indices = np.where(z_scores > threshold)[0]
        outlier_indices = list(outlier_indices)
    else:
        return df, y, []
    
    print(f"\nTotal outliers found: {len(outlier_indices)}")
    
    if len(outlier_indices) > 0:
        df_clean = df.drop(outlier_indices)
        y_clean = y.drop(outlier_indices)
        print(f"Data size after removing outliers: {len(df_clean)} (removed {len(outlier_indices)} samples)")
        return df_clean, y_clean, outlier_indices
    
    print("No outliers found")
    return df, y, []

def check_data_leakage(X_train, X_test, y_train, y_test):
    """Check for data leakage"""
    print("\n=== Data Leakage Check ===")
    
    # Check for overlap between train and test sets
    train_indices = set(X_train.index)
    test_indices = set(X_test.index)
    overlap = train_indices.intersection(test_indices)
    
    if overlap:
        print(f"❌ Data leakage detected: {len(overlap)} overlapping samples between train and test sets")
    else:
        print("✅ No overlap between train and test sets")
    
    # Check feature distributions
    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")
    print(f"Number of features: {X_train.shape[1]}")
    
    return len(overlap) == 0

def safe_feature_engineering(X_train, X_test):
    """Safe feature engineering to avoid data leakage"""
    print("Performing safe feature engineering...")
    
    # 1. Logarithmic transformation
    columns_to_log = ['Lon', 'Lat', 'Age', 'BD', 'pH']
    for col in columns_to_log:
        if col in X_train.columns:
            X_train[col + '_log'] = np.log(X_train[col] + 1e-8)
            X_test[col + '_log'] = np.log(X_test[col] + 1e-8)
            print(f"  Created log feature: {col}_log")
    
    # 2. Binning (based on training set quantiles)
    if 'Altitude' in X_train.columns:
        # Use training set to compute bin boundaries
        alt_bins = pd.cut(X_train['Altitude'], bins=5, retbins=True)[1]
        X_train['Altitude_bins'] = pd.cut(X_train['Altitude'], bins=alt_bins, labels=False)
        X_test['Altitude_bins'] = pd.cut(X_test['Altitude'], bins=alt_bins, labels=False)
        print("  Created binned feature: Altitude_bins")
    
    return X_train, X_test

def augment_continuous_features(X, y, continuous_cols, augmentation_factor=2, noise_scale=0.01, random_state=42):
    """
    Augment continuous features with noise - with proper index resetting 
    to prevent alignment issues during training/evaluation.
    """
    np.random.seed(random_state)
    
    # Reset indices to ensure positional indexing (iloc) matches the index
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    
    n_samples = len(X)
    n_augment = int(n_samples * augmentation_factor)

    augmented_X_list = []
    augmented_y_list = []

    for _ in range(n_augment):
        idx = np.random.randint(0, n_samples)
        base_sample = X.iloc[idx].copy()
        
        # Add noise to specified continuous columns
        for col in continuous_cols:
            if col in base_sample:
                std = X[col].std()
                noise = np.random.normal(loc=0, scale=std * noise_scale)
                
                # Logical check: don't allow negative values for typically positive soil metrics
                if col in ['BD', 'Age', 'Altitude'] or 'log' in col:
                    base_sample[col] = max(1e-8, base_sample[col] + noise)
                else:
                    base_sample[col] += noise

        augmented_X_list.append(base_sample)
        augmented_y_list.append(y.iloc[idx])

    # Efficiently combine original and augmented data
    X_augmented = pd.concat([X, pd.DataFrame(augmented_X_list)], ignore_index=True)
    y_augmented = pd.concat([y, pd.Series(augmented_y_list)], ignore_index=True)

    return X_augmented, y_augmented

def enhanced_load_and_prepare_data(augment=True, augmentation_factor=2, 
                                 lu_type_importance_boost=3.0, 
                                 outlier_threshold=2.0):
    """Fixed data leakage version with outlier handling"""
    
    # Load data
    dtype_dict = {
        'Soillayer': 'int8',
        'yi': 'float32'
    }

    df_clean = pd.read_csv('F:/model/df.clean.yi.csv', dtype=dtype_dict)
    data_predictor_VIF = pd.read_csv('F:/model/data.predictor.VIF.yi.csv')

    # Select target soil layer
    target_layer = "topsoil" ###################################################################################SELECT LAYER
    df = df_clean[df_clean['Soillayer'] == (1 if target_layer == 'topsoil' else 2)].copy()

    # Select features
    predictor_columns = data_predictor_VIF['x'].tolist()
    valid_x_cols = [col for col in predictor_columns if col != "Soillayer" and col in df.columns]

    # Data cleaning
    for col in valid_x_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    X = df[valid_x_cols].astype('float32').fillna(0)
    y = df['yi'].astype('float32')

    print(f"Original data size: {len(X)}")
    
    # Outlier detection and handling
    continuous_cols = ['Lon', 'Lat', 'Age', 'BD', 'pH', 'Altitude']
    continuous_cols = [col for col in continuous_cols if col in X.columns]
    
    print("\n=== Outlier Handling ===")
    X_clean, y_clean, outliers = remove_outliers_robust(
        X, y, continuous_cols, threshold=outlier_threshold
    )

    # Data splitting
    X_train, X_test, y_train, y_test = train_test_split(
        X_clean, y_clean, test_size=0.3, random_state=42
    )

    # Safe feature engineering
    X_train, X_test = safe_feature_engineering(X_train, X_test)

    # Data leakage check
    check_data_leakage(X_train, X_test, y_train, y_test)

    # LUtype feature enhancement (only for training set)
    if 'LUtype' in X_train.columns:
        print(f"\n=== LUtype Feature Enhancement ===")
        print(f"Original LUtype distribution:")
        print(X_train['LUtype'].value_counts().sort_index())
        
        print(f"\nEnhancing LUtype importance with factor: {lu_type_importance_boost}")
        
        # Create LUtype feature enhancements
        for i in range(int(lu_type_importance_boost) - 1):
            lu_type_col_name = f'LUtype_boost_{i+1}'
            X_train[lu_type_col_name] = X_train['LUtype']
            X_test[lu_type_col_name] = X_test['LUtype']
            print(f"  Created LUtype copy: {lu_type_col_name}")
        
        # Create interaction terms between LUtype and other important features
        important_features = ['pH', 'BD', 'Age', 'Altitude']
        for feature in important_features:
            if feature in X_train.columns:
                interaction_name = f'LUtype_{feature}_interaction'
                X_train[interaction_name] = X_train['LUtype'] * X_train[feature]
                X_test[interaction_name] = X_test['LUtype'] * X_test[feature]
                print(f"  Created interaction feature: {interaction_name}")
        
        # Create LUtype squared term (non-linear effect)
        X_train['LUtype_squared'] = X_train['LUtype'] ** 2
        X_test['LUtype_squared'] = X_test['LUtype'] ** 2
        print("  Created LUtype_squared")
    
    # Data augmentation (only for training set)
    if augment:
        continuous_cols = [
            'Lon', 'Lat', 'Age', 'BD', 'pH',
            'Lon_log', 'Lat_log', 'Age_log', 'BD_log', 'pH_log',
            'Altitude'
        ]
        continuous_cols = [col for col in continuous_cols if col in X_train.columns]
        
        print(f"\n=== Data Augmentation ===")
        print(f"Training set size before augmentation: {len(X_train)}")
        X_train, y_train = augment_continuous_features(
            X=X_train,  # Only augment training set
            y=y_train,  # Only augment training set
            continuous_cols=continuous_cols,
            augmentation_factor=augmentation_factor,
            noise_scale=0.005,  
            random_state=42
        )
        print(f"Training set size after augmentation: {len(X_train)}")
    else:
        print("\nSkipping data augmentation")

    print(f"\n=== Final Dataset Statistics ===")
    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")
    print(f"Total number of features: {len(X_train.columns)}")
    
    # Statistical feature distribution
    lu_type_cols = [col for col in X_train.columns if 'LUtype' in col]
    
    print(f"\nFeature statistics:")
    print(f"LUtype-related features: {len(lu_type_cols)}")

    dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=X_train.columns.tolist())
    dtest = xgb.DMatrix(X_test, label=y_test, feature_names=X_train.columns.tolist())

    return dtrain, dtest, X, y, X_test, y_test, X_train, y_train

def main():
    """Main function with data leakage prevention"""
    outlier_threshold = 2.0  # Outlier detection threshold
    
    print("=== Starting Enhanced XGBoost Modeling (Fixed Data Leakage Version) ===")
    
    # Use fixed data processing (with outlier handling and data leakage prevention)
    dtrain, dtest, x1, y1, X_test, y_test, X_train, y_train = enhanced_load_and_prepare_data(
        augment=True,
        augmentation_factor=3,
        lu_type_importance_boost=3.0,
        outlier_threshold=outlier_threshold
    )

    tuner = XGBoostTuner(seed=42, n_jobs=17)

    # 1. Hyperparameter tuning
    print("\n=== Starting Hyperparameter Tuning ===")
    best_params = tuner.tune_hyperparameters(dtrain, n_trials=10)
    print(f"Best parameters: {best_params}")

    # 2. Enhanced Cross-validation with 10x5 folds and model saving
    print("\n=== Starting 10x5 Cross-Validation ===")
    cv_results_summary = tuner.cross_validate(best_params, X_train, y_train, n_splits=5,
                                              cv_models_dir='F:/model/results/cv_models',
                                              n_repeats=10)  # 10 repetitions of 5-fold CV
    
    print("\nSaved CV models:")
    for path in cv_results_summary['detailed_results']['model_paths'][:5]:  # Show first 5
        print(f"- {path}")
    print(f"... and {len(cv_results_summary['detailed_results']['model_paths']) - 5} more")
    
    # Plot enhanced CV results
    print("\nPlotting enhanced cross-validation results...")
    tuner.plot_cv_results(cv_results_summary)

    # 3. Train final model
    optimal_rounds = int(cv_results_summary['best_iterations_mean'])
    print(f"\nTraining final model with {optimal_rounds} rounds...")
    final_model, evals_result = tuner.train_model(best_params, dtrain, num_boost_round=optimal_rounds)

    # Plot learning curves
    print("\nPlotting learning curves...")
    tuner.plot_learning_curves(evals_result, "Final Model")

    # 4. Evaluate and plot final performance
    print("\nEvaluating final model...")
    test_metrics, _ = tuner.evaluate_model(final_model, dtest)
    print("\n=== Final Test Metrics ===")
    print(f"RMSE: {test_metrics['rmse']:.4f}")
    print(f"R²: {test_metrics['r2']:.4f}")
    print(f"MAE: {test_metrics['mae']:.4f}")

    print("\nPlotting final model performance...")
    tuner.plot_final_model_performance(final_model, dtest)

    # 5. Analyze LUtype importance
    print(f"\nAnalyzing LUtype feature importance...")
    importance_df = tuner.plot_feature_importance(final_model, top_n=35)
    
    # Detailed analysis of LUtype importance
    tuner.analyze_lutype_importance(final_model, X_test)
    
    # 6. SHAP analysis
    if SHAP_AVAILABLE:
        print("\nGenerating SHAP summary plot...")
        tuner.plot_shap_summary(final_model, X_test)
        
    # 7. Save results
    print("\nSaving results...")
    tuner.save_results(final_model, 'F:/model/results')
    
    # Save CV results summary
    cv_summary_path = 'F:/model/results/cv_summary.csv'
    cv_results_summary['detailed_results'].to_csv(cv_summary_path, index=False)
    print(f"Saved detailed CV results to: {cv_summary_path}")
    
    print(f"\n=== Model Training Summary ===")
    print("✓ Fixed data leakage issues")
    print("✓ Added outlier detection and handling")
    print("✓ Safe feature engineering (avoided data leakage)")
    print("✓ Created multiple LUtype feature copies")
    print("✓ Added LUtype interaction features with other important variables")
    print("✓ Added LUtype squared term (non-linear effect)")
    print("✓ Improved hyperparameter search space for better handling of categorical features")
    print("✓ Test set uses original data (no augmentation noise)")
    print("✓ Implemented 10 repetitions of 5-fold cross-validation for robust evaluation")
    
    print("\n=== Complete! ===")

if __name__ == "__main__":
    main()

In [ ]:
# XGBoost modeling: active fractions in subsoil
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import joblib
import os
import re
from pathlib import Path
import warnings
import builtins
from scipy import stats
open = builtins.open

warnings.filterwarnings('ignore')

# SHAP availability check
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False

class XGBoostTuner:
    def __init__(self, seed=42, n_jobs=-1):
        self.seed = seed
        self.n_jobs = n_jobs if n_jobs != -1 else None
        self.study = None
        self.best_model = None
        self.best_params = None
        self.feature_names = None  # Store feature names

    @staticmethod
    def clean_numeric_value(value):
        """Convert string values to numeric, handling special cases"""
        if pd.isna(value):
            return np.nan
        if isinstance(value, str):
            # Remove any non-numeric characters except minus, decimal point
            cleaned = re.sub(r'[^\d.-]', '', value)
            try:
                return float(cleaned) if cleaned else np.nan
            except ValueError:
                return np.nan
        return float(value)

    def objective(self, trial, dtrain, num_boost_round=500):
        """Objective function with enhanced regularization"""
        params = {
            'eta': trial.suggest_float('eta', 0.005, 0.1, log=True),  # Lower learning rate
            'max_depth': trial.suggest_int('max_depth', 2, 4),  # Shallower trees
            'subsample': trial.suggest_float('subsample', 0.4, 0.7),  # Lower subsampling
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 0.7),  # Lower feature sampling
            'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.4, 0.7),
            'min_child_weight': trial.suggest_int('min_child_weight', 10, 20),  # Higher min child weight
            'lambda': trial.suggest_float('lambda', 10, 25.0),  # Stronger L2 regularization
            'alpha': trial.suggest_float('alpha', 5.0, 15.0),  # Stronger L1 regularization
            'gamma': trial.suggest_float('gamma', 0.5, 2.0),  # Higher pruning parameter
            'max_delta_step': trial.suggest_int('max_delta_step', 0, 1),
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'verbosity': 0,
            'seed': self.seed,
            'nthread': 1
        }

        cv_results = xgb.cv(
            params=params,
            dtrain=dtrain,
            num_boost_round=800,  # More rounds but more conservative early stopping
            nfold=5,
            metrics='rmse',
            early_stopping_rounds=20,  # More conservative early stopping
            as_pandas=True,
            seed=self.seed,
            shuffle=True
        )

        # Stronger overfitting penalty
        train_rmse = cv_results['train-rmse-mean'].iloc[-1]
        val_rmse = cv_results['test-rmse-mean'].iloc[-1]
        
        # Calculate overfitting degree
        overfitting_ratio = (train_rmse - val_rmse) / train_rmse
        gap_penalty = max(0, overfitting_ratio) * 0.3  # Stronger penalty
        
        # Add extra penalty if severe overfitting
        if overfitting_ratio > 0.1:
            gap_penalty += (overfitting_ratio - 0.1) * 0.5
            
        return val_rmse + gap_penalty
    
    def tune_hyperparameters(self, dtrain, n_trials=10):  # Increased number of trials
        """Hyperparameter tuning focused on generalization"""
        study = optuna.create_study(direction='minimize')
        
        print("Tuning hyperparameters with aggressive overfitting prevention...")
        print("Using very strong regularization and shallow trees")

        with ThreadPoolExecutor(max_workers=self.n_jobs) as executor:
            futures = [executor.submit(lambda: study.optimize(
                lambda trial: self.objective(trial, dtrain),
                n_trials=5,
                n_jobs=1
            )) for _ in range(n_trials//5)]

            for future in futures:
                future.result()

        self.study = study
        self.best_params = study.best_params
        self.best_params.update({
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'seed': self.seed,
            'nthread': self.n_jobs
        })

        print(f"Best parameters found: {self.best_params}")
        return self.best_params


    def train_model(self, params, dtrain, dvalid=None, num_boost_round=500):
        """Train model with focus on preventing overfitting"""
        evals = [(dtrain, 'train')]
        if dvalid is not None:
            evals.append((dvalid, 'valid'))

        evals_result = {}
        model = xgb.train(
            params=params,
            dtrain=dtrain,
            num_boost_round=num_boost_round,
            evals=evals,
            early_stopping_rounds=25,  # More conservative early stopping
            verbose_eval=100,  # Reduced output frequency
            evals_result=evals_result
        )

        self.feature_names = dtrain.feature_names
        if self.feature_names is None:
            self.feature_names = [f'f{i}' for i in range(dtrain.num_col())]
            model.feature_names = self.feature_names

        return model, evals_result

    def cross_validate(self, params, X, y, n_splits=5, cv_models_dir='cv_models', n_repeats=10): 
        """Enhanced CV with 10 repetitions of 5-fold cross-validation and model saving"""
        Path(cv_models_dir).mkdir(parents=True, exist_ok=True)

        all_cv_results = {
            'train_rmse': [], 'val_rmse': [],
            'train_r2': [], 'val_r2': [],
            'best_iterations': [],
            'model_paths': [],
            'feature_names': [],
            'repeat': [],
            'fold': []
        }

        def process_fold(repeat_idx, fold_idx, train_idx, val_idx):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=X_train.columns.tolist())
            dval = xgb.DMatrix(X_val, label=y_val, feature_names=X_train.columns.tolist())

            model, _ = self.train_model(params, dtrain, dval)

            # Save the CV model
            model_path = os.path.join(cv_models_dir, f'cv_model_repeat_{repeat_idx}_fold_{fold_idx}.json')
            model.save_model(model_path)

            train_pred = model.predict(dtrain)
            val_pred = model.predict(dval)

            return (
                np.sqrt(mean_squared_error(y_train, train_pred)),
                np.sqrt(mean_squared_error(y_val, val_pred)),
                r2_score(y_train, train_pred),
                r2_score(y_val, val_pred),
                model.best_iteration if hasattr(model, 'best_iteration') else params.get('num_boost_round', 500),
                model_path,
                model.feature_names,
                repeat_idx,
                fold_idx
            )
        
        with ThreadPoolExecutor(max_workers=self.n_jobs) as executor:
            futures = []
            for repeat_idx in range(n_repeats):
                kf = KFold(n_splits=n_splits, shuffle=True, random_state=self.seed + repeat_idx)
                for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X)):
                    futures.append(executor.submit(
                        process_fold, repeat_idx, fold_idx, train_idx, val_idx
                    ))

            for future in futures:
                train_rmse, val_rmse, train_r2, val_r2, best_iter, model_path, feature_names, repeat_idx, fold_idx = future.result()
                all_cv_results['train_rmse'].append(train_rmse)
                all_cv_results['val_rmse'].append(val_rmse)
                all_cv_results['train_r2'].append(train_r2)
                all_cv_results['val_r2'].append(val_r2)
                all_cv_results['best_iterations'].append(best_iter)
                all_cv_results['model_paths'].append(model_path)
                all_cv_results['feature_names'].append(feature_names)
                all_cv_results['repeat'].append(repeat_idx)
                all_cv_results['fold'].append(fold_idx)
        # Create summary statistics
        cv_results_summary = {
            'train_rmse_mean': np.mean(all_cv_results['train_rmse']),
            'train_rmse_std': np.std(all_cv_results['train_rmse']),
            'val_rmse_mean': np.mean(all_cv_results['val_rmse']),
            'val_rmse_std': np.std(all_cv_results['val_rmse']),
            'train_r2_mean': np.mean(all_cv_results['train_r2']),
            'train_r2_std': np.std(all_cv_results['train_r2']),
            'val_r2_mean': np.mean(all_cv_results['val_r2']),
            'val_r2_std': np.std(all_cv_results['val_r2']),
            'best_iterations_mean': np.mean(all_cv_results['best_iterations']),
            'best_iterations_std': np.std(all_cv_results['best_iterations']),
            'detailed_results': pd.DataFrame(all_cv_results)
        }

        return cv_results_summary

    def evaluate_model(self, model, dtest):
        """Evaluate model performance on test set"""
        test_pred = model.predict(dtest)
        return {
            'rmse': np.sqrt(mean_squared_error(dtest.get_label(), test_pred)),
            'r2': r2_score(dtest.get_label(), test_pred),
            'mae': mean_absolute_error(dtest.get_label(), test_pred)
        }, test_pred

    def plot_learning_curves(self, evals_result, title_suffix):
        """Plot training and validation metrics"""
        plt.figure(figsize=(10, 5))
        epochs = len(evals_result['train']['rmse'])
        plt.plot(range(epochs), evals_result['train']['rmse'], label='Train RMSE')
        if 'valid' in evals_result:
            plt.plot(range(epochs), evals_result['valid']['rmse'], label='Validation RMSE')
        plt.legend()
        plt.xlabel('Iterations')
        plt.ylabel('RMSE')
        plt.title(f'Learning Curves - {title_suffix}')
        plt.show()

    def plot_cv_results(self, cv_results_summary):
        """Plot enhanced cross-validation results with both original and new visualization"""
        # Check if it's the new summary format or old list format
        if isinstance(cv_results_summary, dict) and 'detailed_results' in cv_results_summary:
            # New format - use the detailed results
            detailed_results = cv_results_summary['detailed_results']
            
            # Create two visualizations:
            # 1. Original visualization (simple line plot for first repeat)
            plt.figure(figsize=(20, 10))
            
            # Plot 1: Original style - first repeat only
            plt.subplot(2, 2, 1)
            first_repeat = detailed_results[detailed_results['repeat'] == 0]
            plt.plot(first_repeat['train_rmse'], 'o-', label='Train RMSE')
            plt.plot(first_repeat['val_rmse'], 'o-', label='Validation RMSE')
            plt.xlabel('Fold')
            plt.ylabel('RMSE')
            plt.title('Cross-Validation RMSE (First Repeat)')
            plt.legend()
            plt.grid(True)

            # Plot 2: Original style - first repeat R²
            plt.subplot(2, 2, 2)
            plt.plot(first_repeat['train_r2'], 'o-', label='Train R²')
            plt.plot(first_repeat['val_r2'], 'o-', label='Validation R²')
            plt.xlabel('Fold')
            plt.ylabel('R² Score')
            plt.title('Cross-Validation R² Scores (First Repeat)')
            plt.legend()
            plt.grid(True)

            # Plot 3: New visualization - RMSE distribution across all repeats
            plt.subplot(2, 2, 3)
            plt.boxplot([detailed_results['train_rmse'], detailed_results['val_rmse']], 
                       labels=['Train RMSE', 'Validation RMSE'])
            plt.ylabel('RMSE')
            plt.title('RMSE Distribution across 10x5 CV')
            plt.grid(True, alpha=0.3)
            
            # Plot 4: New visualization - R² distribution across all repeats
            plt.subplot(2, 2, 4)
            plt.boxplot([detailed_results['train_r2'], detailed_results['val_r2']], 
                       labels=['Train R²', 'Validation R²'])
            plt.ylabel('R² Score')
            plt.title('R² Distribution across 10x5 CV')
            plt.grid(True, alpha=0.3)

            plt.tight_layout()
            plt.show()
            
            # Print summary statistics
            print("\n=== 10x5 Cross-Validation Summary ===")
            print(f"Average Train RMSE: {cv_results_summary['train_rmse_mean']:.4f} (±{cv_results_summary['train_rmse_std']:.4f})")
            print(f"Average Validation RMSE: {cv_results_summary['val_rmse_mean']:.4f} (±{cv_results_summary['val_rmse_std']:.4f})")
            print(f"Average Train R²: {cv_results_summary['train_r2_mean']:.4f} (±{cv_results_summary['train_r2_std']:.4f})")
            print(f"Average Validation R²: {cv_results_summary['val_r2_mean']:.4f} (±{cv_results_summary['val_r2_std']:.4f})")
            print(f"Average Best Iterations: {cv_results_summary['best_iterations_mean']:.1f} (±{cv_results_summary['best_iterations_std']:.1f})")
            
            # Also print the format that main() expects
            print("\n=== 交叉验证结果 (旧格式兼容) ===")
            print(f"平均训练RMSE: {cv_results_summary['train_rmse_mean']:.4f} (±{cv_results_summary['train_rmse_std']:.4f})")
            print(f"平均验证RMSE: {cv_results_summary['val_rmse_mean']:.4f} (±{cv_results_summary['val_rmse_std']:.4f})")
            print(f"平均训练R²: {cv_results_summary['train_r2_mean']:.4f} (±{cv_results_summary['train_r2_std']:.4f})")
            print(f"平均验证R²: {cv_results_summary['val_r2_mean']:.4f} (±{cv_results_summary['val_r2_std']:.4f})")
            
        else:
            # Old format - keep original plotting
            plt.figure(figsize=(15, 5))

            # Plot RMSE
            plt.subplot(1, 2, 1)
            plt.plot(cv_results_summary['train_rmse'], 'o-', label='Train RMSE')
            plt.plot(cv_results_summary['val_rmse'], 'o-', label='Validation RMSE')
            plt.xlabel('Fold')
            plt.ylabel('RMSE')
            plt.title('Cross-Validation RMSE')
            plt.legend()
            plt.grid(True)

            # Plot R2
            plt.subplot(1, 2, 2)
            plt.plot(cv_results_summary['train_r2'], 'o-', label='Train R²')
            plt.plot(cv_results_summary['val_r2'], 'o-', label='Validation R²')
            plt.xlabel('Fold')
            plt.ylabel('R² Score')
            plt.title('Cross-Validation R² Scores')
            plt.legend()
            plt.grid(True)

            plt.tight_layout()
            plt.show()

    def plot_final_model_performance(self, model, dtest):
        """Plot actual vs predicted values for test set"""
        test_metrics, test_pred = self.evaluate_model(model, dtest)
        y_test = dtest.get_label()

        plt.figure(figsize=(12, 5))

        # Scatter plot of actual vs predicted
        plt.subplot(1, 2, 1)
        plt.scatter(y_test, test_pred, alpha=0.5)
        plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--r')
        plt.xlabel('Actual Values')
        plt.ylabel('Predicted Values')
        plt.title(f'Actual vs Predicted (R² = {test_metrics["r2"]:.3f})')

        # Residual plot
        plt.subplot(1, 2, 2)
        residuals = y_test - test_pred
        plt.scatter(test_pred, residuals, alpha=0.5)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.xlabel('Predicted Values')
        plt.ylabel('Residuals')
        plt.title('Residual Plot')
        plt.tight_layout()
        plt.show()

    def plot_feature_importance(self, model, importance_type='weight', top_n=30):
        """Plot feature importance without category-specific highlighting"""
        fig, ax = plt.subplots(figsize=(12, 10))
        
      
        importance_dict = model.get_score(importance_type=importance_type)
        
        if not importance_dict:
            print("No feature importance data available.")
            return None
            
   
        importance_df = pd.DataFrame({
            'feature': list(importance_dict.keys()),
            'importance': list(importance_dict.values())
        }).sort_values('importance', ascending=True)
        
  
        if len(importance_df) > top_n:
            importance_df = importance_df.tail(top_n)
        
    
        y_pos = np.arange(len(importance_df))
        colors = ['lightblue'] * len(importance_df)
        
        ax.barh(y_pos, importance_df['importance'], color=colors, alpha=0.8)
        ax.set_yticks(y_pos)
        ax.set_yticklabels(importance_df['feature'])
        ax.set_xlabel('Importance')
        
        title = f'Feature Importance ({importance_type})'
        ax.set_title(title)
        

        lu_type_features = importance_df[importance_df['feature'].str.contains('LUtype', na=False)]
        lu_type_importance = lu_type_features['importance'].sum() if not lu_type_features.empty else 0
        
        ax.text(0.7, 0.95, f'LUtype total: {lu_type_importance:.3f}', 
                transform=ax.transAxes, fontsize=10, bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow"))
        
        plt.tight_layout()
        plt.show()
        
        return importance_df

    def analyze_lutype_importance(self, model, X):
        """分析LUtype相关特征的重要性，不关注特定类别"""
        importance_dict = model.get_score(importance_type='weight')
        
        if not importance_dict:
            print("No feature importance data available.")
            return
            
 
        lutype_features = {k: v for k, v in importance_dict.items() if 'LUtype' in k}
        other_features = {k: v for k, v in importance_dict.items() if 'LUtype' not in k}
        
        print(f"\n=== LUtype Feature Importance Analysis ===")
        print(f"Total LUtype-related features: {len(lutype_features)}")
        print(f"Total importance of LUtype features: {sum(lutype_features.values()):.3f}")
        print(f"Average importance of LUtype features: {np.mean(list(lutype_features.values())) if lutype_features else 0:.3f}")
        
        total_importance = sum(importance_dict.values())
        lu_type_percentage = (sum(lutype_features.values()) / total_importance * 100) if total_importance > 0 else 0
        
        print(f"\nLUtype importance percentage: {lu_type_percentage:.2f}%")
        

        if lutype_features:
            sorted_lutype = sorted(lutype_features.items(), key=lambda x: x[1], reverse=True)
            print(f"\nTop LUtype-related features:")
            for feature, importance in sorted_lutype[:10]:
                print(f"  {feature}: {importance:.4f}")
        else:
            print("\nNo LUtype-related features found in importance scores.")

    def plot_shap_summary(self, model, X, feature_names=None):
        """Generate SHAP summary plot if SHAP is available"""
        if SHAP_AVAILABLE:
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X)

            plt.figure(figsize=(10, 8))
            shap.summary_plot(shap_values, X, feature_names=feature_names)
            plt.tight_layout()
            plt.show()
        else:
            print("SHAP not available. Install with: pip install shap")

    def save_results(self, model, output_dir='results'):
        """Save all results to disk including feature names"""
        Path(output_dir).mkdir(exist_ok=True)

        # Save model
        model.save_model(f'{output_dir}/xgb_model.json')

        # Save feature names
        if self.feature_names is not None:
            with open(f'{output_dir}/feature_names.txt', 'w') as f:
                f.write('\n'.join(self.feature_names))

        # Save other artifacts
        joblib.dump(self.best_params, f'{output_dir}/best_params.pkl')

        if self.study:
            joblib.dump(self.study, f'{output_dir}/study.pkl')
            study_df = self.study.trials_dataframe()
            study_df.to_csv(f'{output_dir}/trials.csv', index=False)

def detect_outliers_iqr(df, columns, threshold=1.5):
    """Detect outliers using IQR method"""
    outlier_indices = []
    
    for col in columns:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
            outlier_indices.extend(outliers)
            
            if len(outliers) > 0:
                print(f"  {col}: Found {len(outliers)} outliers "
                      f"({len(outliers)/len(df)*100:.2f}%)")
    
    return list(set(outlier_indices))

def remove_outliers_robust(df, y, columns, method='iqr', threshold=2.0):
    """Robust outlier handling method"""
    print("Starting outlier detection...")
    
    if method == 'iqr':
        outlier_indices = detect_outliers_iqr(df, columns, threshold)
    elif method == 'zscore':
        # Z-score method
        z_scores = np.abs(stats.zscore(df[columns]))
        outlier_indices = np.where(z_scores > threshold)[0]
        outlier_indices = list(outlier_indices)
    else:
        return df, y, []
    
    print(f"\nTotal outliers found: {len(outlier_indices)}")
    
    if len(outlier_indices) > 0:
        df_clean = df.drop(outlier_indices)
        y_clean = y.drop(outlier_indices)
        print(f"Data size after removing outliers: {len(df_clean)} (removed {len(outlier_indices)} samples)")
        return df_clean, y_clean, outlier_indices
    
    print("No outliers found")
    return df, y, []

def check_data_leakage(X_train, X_test, y_train, y_test):
    """Check for data leakage"""
    print("\n=== Data Leakage Check ===")
    
    # Check for overlap between train and test sets
    train_indices = set(X_train.index)
    test_indices = set(X_test.index)
    overlap = train_indices.intersection(test_indices)
    
    if overlap:
        print(f"❌ Data leakage detected: {len(overlap)} overlapping samples between train and test sets")
    else:
        print("✅ No overlap between train and test sets")
    
    # Check feature distributions
    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")
    print(f"Number of features: {X_train.shape[1]}")
    
    return len(overlap) == 0

def safe_feature_engineering(X_train, X_test):
    """Safe feature engineering to avoid data leakage"""
    print("Performing safe feature engineering...")
    
    # 1. Logarithmic transformation
    columns_to_log = ['Lon', 'Lat', 'Age', 'BD', 'pH']
    for col in columns_to_log:
        if col in X_train.columns:
            X_train[col + '_log'] = np.log(X_train[col] + 1e-8)
            X_test[col + '_log'] = np.log(X_test[col] + 1e-8)
            print(f"  Created log feature: {col}_log")
    
    # 2. Binning (based on training set quantiles)
    if 'Altitude' in X_train.columns:
        # Use training set to compute bin boundaries
        alt_bins = pd.cut(X_train['Altitude'], bins=5, retbins=True)[1]
        X_train['Altitude_bins'] = pd.cut(X_train['Altitude'], bins=alt_bins, labels=False)
        X_test['Altitude_bins'] = pd.cut(X_test['Altitude'], bins=alt_bins, labels=False)
        print("  Created binned feature: Altitude_bins")
    
    return X_train, X_test

def augment_continuous_features(X, y, continuous_cols, augmentation_factor=2, noise_scale=0.01, random_state=42):
    """Augment continuous features with noise - only for training set"""
    np.random.seed(random_state)
    n_samples = len(X)
    n_augment = int(n_samples * augmentation_factor)

    X_augmented = X.copy()
    y_augmented = y.copy()

    for _ in range(n_augment):
        idx = np.random.randint(0, n_samples)
        base_sample = X.iloc[idx].copy()

        for col in continuous_cols:
            std = X[col].std()
            noise = np.random.normal(loc=0, scale=std * noise_scale)
            if X[col].min() >= 0:
                base_sample[col] = max(0, base_sample[col] + noise)
            else:
                base_sample[col] += noise

        X_augmented = X_augmented._append(base_sample, ignore_index=True)
        
        y_augmented = y_augmented._append(
            pd.Series(y.iloc[idx], index=[len(y_augmented)]),  
            ignore_index=True
        )

    return X_augmented, y_augmented

def enhanced_load_and_prepare_data(augment=True, augmentation_factor=2, 
                                 lu_type_importance_boost=3.0, 
                                 outlier_threshold=2.0):
    """Fixed data leakage version with outlier handling"""
    
    # Load data
    dtype_dict = {
        'Soillayer': 'int8',
        'yi': 'float32'
    }

    df_clean = pd.read_csv('F:/model/df.clean.yi.csv', dtype=dtype_dict)
    data_predictor_VIF = pd.read_csv('F:/model/data.predictor.VIF.yi.csv')

    # Select target soil layer
    target_layer = "subsoil" ###################################################################################SELECT LAYER
    df = df_clean[df_clean['Soillayer'] == (1 if target_layer == 'topsoil' else 2)].copy()

    # Select features
    predictor_columns = data_predictor_VIF['x'].tolist()
    valid_x_cols = [col for col in predictor_columns if col != "Soillayer" and col in df.columns]

    # Data cleaning
    for col in valid_x_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    X = df[valid_x_cols].astype('float32').fillna(0)
    y = df['yi'].astype('float32')

    print(f"Original data size: {len(X)}")
    
    # Outlier detection and handling
    continuous_cols = ['Lon', 'Lat', 'Age', 'BD', 'pH', 'Altitude']
    continuous_cols = [col for col in continuous_cols if col in X.columns]
    
    print("\n=== Outlier Handling ===")
    X_clean, y_clean, outliers = remove_outliers_robust(
        X, y, continuous_cols, threshold=outlier_threshold
    )

    # Data splitting
    X_train, X_test, y_train, y_test = train_test_split(
        X_clean, y_clean, test_size=0.3, random_state=42
    )

    # Safe feature engineering
    X_train, X_test = safe_feature_engineering(X_train, X_test)

    # Data leakage check
    check_data_leakage(X_train, X_test, y_train, y_test)

    # LUtype feature enhancement (only for training set)
    if 'LUtype' in X_train.columns:
        print(f"\n=== LUtype Feature Enhancement ===")
        print(f"Original LUtype distribution:")
        print(X_train['LUtype'].value_counts().sort_index())
        
        print(f"\nEnhancing LUtype importance with factor: {lu_type_importance_boost}")
        
        # Create LUtype feature enhancements
        for i in range(int(lu_type_importance_boost) - 1):
            lu_type_col_name = f'LUtype_boost_{i+1}'
            X_train[lu_type_col_name] = X_train['LUtype']
            X_test[lu_type_col_name] = X_test['LUtype']
            print(f"  Created LUtype copy: {lu_type_col_name}")
        
        # Create interaction terms between LUtype and other important features
        important_features = ['pH', 'BD', 'Age', 'Altitude']
        for feature in important_features:
            if feature in X_train.columns:
                interaction_name = f'LUtype_{feature}_interaction'
                X_train[interaction_name] = X_train['LUtype'] * X_train[feature]
                X_test[interaction_name] = X_test['LUtype'] * X_test[feature]
                print(f"  Created interaction feature: {interaction_name}")
        
        # Create LUtype squared term (non-linear effect)
        X_train['LUtype_squared'] = X_train['LUtype'] ** 2
        X_test['LUtype_squared'] = X_test['LUtype'] ** 2
        print("  Created LUtype_squared")
    
    # Data augmentation (only for training set)
    if augment:
        continuous_cols = [
            'Lon', 'Lat', 'Age', 'BD', 'pH',
            'Lon_log', 'Lat_log', 'Age_log', 'BD_log', 'pH_log',
            'Altitude'
        ]
        continuous_cols = [col for col in continuous_cols if col in X_train.columns]
        
        print(f"\n=== Data Augmentation ===")
        print(f"Training set size before augmentation: {len(X_train)}")
        X_train, y_train = augment_continuous_features(
            X=X_train,  # Only augment training set
            y=y_train,  # Only augment training set
            continuous_cols=continuous_cols,
            augmentation_factor=augmentation_factor,
            noise_scale=0.01,  
            random_state=42
        )
        print(f"Training set size after augmentation: {len(X_train)}")
    else:
        print("\nSkipping data augmentation")

    print(f"\n=== Final Dataset Statistics ===")
    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")
    print(f"Total number of features: {len(X_train.columns)}")
    
    # Statistical feature distribution
    lu_type_cols = [col for col in X_train.columns if 'LUtype' in col]
    
    print(f"\nFeature statistics:")
    print(f"LUtype-related features: {len(lu_type_cols)}")

    dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=X_train.columns.tolist())
    dtest = xgb.DMatrix(X_test, label=y_test, feature_names=X_train.columns.tolist())

    return dtrain, dtest, X, y, X_test, y_test, X_train, y_train

def main():
    """Main function with data leakage prevention"""
    outlier_threshold = 2.0  # Outlier detection threshold
    
    print("=== Starting Enhanced XGBoost Modeling (Fixed Data Leakage Version) ===")
    
    # Use fixed data processing (with outlier handling and data leakage prevention)
    dtrain, dtest, x1, y1, X_test, y_test, X_train, y_train = enhanced_load_and_prepare_data(
        augment=True,
        augmentation_factor=3,
        lu_type_importance_boost=3.0,
        outlier_threshold=outlier_threshold
    )

    tuner = XGBoostTuner(seed=42, n_jobs=17)

    # 1. Hyperparameter tuning
    print("\n=== Starting Hyperparameter Tuning ===")
    best_params = tuner.tune_hyperparameters(dtrain, n_trials=10)
    print(f"Best parameters: {best_params}")

    # 2. Enhanced Cross-validation with 10x5 folds and model saving
    print("\n=== Starting 10x5 Cross-Validation ===")
    cv_results_summary = tuner.cross_validate(best_params, X_train, y_train, n_splits=5,
                                              cv_models_dir='F:/model/results/cv_models',
                                              n_repeats=10)  # 10 repetitions of 5-fold CV
    
    print("\nSaved CV models:")
    for path in cv_results_summary['detailed_results']['model_paths'][:5]:  # Show first 5
        print(f"- {path}")
    print(f"... and {len(cv_results_summary['detailed_results']['model_paths']) - 5} more")
    
    # Plot enhanced CV results
    print("\nPlotting enhanced cross-validation results...")
    tuner.plot_cv_results(cv_results_summary)

    # 3. Train final model
    optimal_rounds = int(cv_results_summary['best_iterations_mean'])
    print(f"\nTraining final model with {optimal_rounds} rounds...")
    final_model, evals_result = tuner.train_model(best_params, dtrain, num_boost_round=optimal_rounds)

    # Plot learning curves
    print("\nPlotting learning curves...")
    tuner.plot_learning_curves(evals_result, "Final Model")

    # 4. Evaluate and plot final performance
    print("\nEvaluating final model...")
    test_metrics, _ = tuner.evaluate_model(final_model, dtest)
    print("\n=== Final Test Metrics ===")
    print(f"RMSE: {test_metrics['rmse']:.4f}")
    print(f"R²: {test_metrics['r2']:.4f}")
    print(f"MAE: {test_metrics['mae']:.4f}")

    print("\nPlotting final model performance...")
    tuner.plot_final_model_performance(final_model, dtest)

    # 5. Analyze LUtype importance
    print(f"\nAnalyzing LUtype feature importance...")
    importance_df = tuner.plot_feature_importance(final_model, top_n=35)
    
    # Detailed analysis of LUtype importance
    tuner.analyze_lutype_importance(final_model, X_test)
    
    # 6. SHAP analysis
    if SHAP_AVAILABLE:
        print("\nGenerating SHAP summary plot...")
        tuner.plot_shap_summary(final_model, X_test)
        
    # 7. Save results
    print("\nSaving results...")
    tuner.save_results(final_model, 'F:/model/results')
    
    # Save CV results summary
    cv_summary_path = 'F:/model/results/cv_summary.csv'
    cv_results_summary['detailed_results'].to_csv(cv_summary_path, index=False)
    print(f"Saved detailed CV results to: {cv_summary_path}")
    
    print(f"\n=== Model Training Summary ===")
    print("✓ Fixed data leakage issues")
    print("✓ Added outlier detection and handling")
    print("✓ Safe feature engineering (avoided data leakage)")
    print("✓ Created multiple LUtype feature copies")
    print("✓ Added LUtype interaction features with other important variables")
    print("✓ Added LUtype squared term (non-linear effect)")
    print("✓ Improved hyperparameter search space for better handling of categorical features")
    print("✓ Test set uses original data (no augmentation noise)")
    print("✓ Implemented 10 repetitions of 5-fold cross-validation for robust evaluation")
    
    print("\n=== Complete! ===")

if __name__ == "__main__":
    main()

In [ ]:
# XGBoost modeling: passive fractions in subsoil
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import joblib
import os
import re
from pathlib import Path
import warnings
import builtins
from scipy import stats
open = builtins.open

warnings.filterwarnings('ignore')

# SHAP availability check
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False

class XGBoostTuner:
    def __init__(self, seed=42, n_jobs=-1):
        self.seed = seed
        self.n_jobs = n_jobs if n_jobs != -1 else None
        self.study = None
        self.best_model = None
        self.best_params = None
        self.feature_names = None  # Store feature names

    @staticmethod
    def clean_numeric_value(value):
        """Convert string values to numeric, handling special cases"""
        if pd.isna(value):
            return np.nan
        if isinstance(value, str):
            # Remove any non-numeric characters except minus, decimal point
            cleaned = re.sub(r'[^\d.-]', '', value)
            try:
                return float(cleaned) if cleaned else np.nan
            except ValueError:
                return np.nan
        return float(value)

    def objective(self, trial, dtrain, num_boost_round=500):
        """Objective function with enhanced regularization"""
        params = {
            'eta': trial.suggest_float('eta', 0.005, 0.1, log=True),  # Lower learning rate
            'max_depth': trial.suggest_int('max_depth', 2, 4),  # Shallower trees
            'subsample': trial.suggest_float('subsample', 0.4, 0.7),  # Lower subsampling
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 0.7),  # Lower feature sampling
            'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.4, 0.7),
            'min_child_weight': trial.suggest_int('min_child_weight', 10, 20),  # Higher min child weight
            'lambda': trial.suggest_float('lambda', 10, 25.0),  # Stronger L2 regularization
            'alpha': trial.suggest_float('alpha', 5.0, 15.0),  # Stronger L1 regularization
            'gamma': trial.suggest_float('gamma', 0.5, 2.0),  # Higher pruning parameter
            'max_delta_step': trial.suggest_int('max_delta_step', 0, 1),
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'verbosity': 0,
            'seed': self.seed,
            'nthread': 1
        }

        cv_results = xgb.cv(
            params=params,
            dtrain=dtrain,
            num_boost_round=800,  # More rounds but more conservative early stopping
            nfold=5,
            metrics='rmse',
            early_stopping_rounds=10,  # More conservative early stopping
            as_pandas=True,
            seed=self.seed,
            shuffle=True
        )

        # Stronger overfitting penalty
        train_rmse = cv_results['train-rmse-mean'].iloc[-1]
        val_rmse = cv_results['test-rmse-mean'].iloc[-1]
        
        # Calculate overfitting degree
        overfitting_ratio = (train_rmse - val_rmse) / train_rmse
        gap_penalty = max(0, overfitting_ratio) * 0.3  # Stronger penalty
        
        # Add extra penalty if severe overfitting
        if overfitting_ratio > 0.1:
            gap_penalty += (overfitting_ratio - 0.1) * 0.5
            
        return val_rmse + gap_penalty
    
    def tune_hyperparameters(self, dtrain, n_trials=10):  # Increased number of trials
        """Hyperparameter tuning focused on generalization"""
        study = optuna.create_study(direction='minimize')
        
        print("Tuning hyperparameters with aggressive overfitting prevention...")
        print("Using very strong regularization and shallow trees")

        with ThreadPoolExecutor(max_workers=self.n_jobs) as executor:
            futures = [executor.submit(lambda: study.optimize(
                lambda trial: self.objective(trial, dtrain),
                n_trials=5,
                n_jobs=1
            )) for _ in range(n_trials//5)]

            for future in futures:
                future.result()

        self.study = study
        self.best_params = study.best_params
        self.best_params.update({
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'seed': self.seed,
            'nthread': self.n_jobs
        })

        print(f"Best parameters found: {self.best_params}")
        return self.best_params


    def train_model(self, params, dtrain, dvalid=None, num_boost_round=500):
        """Train model with focus on preventing overfitting"""
        evals = [(dtrain, 'train')]
        if dvalid is not None:
            evals.append((dvalid, 'valid'))

        evals_result = {}
        model = xgb.train(
            params=params,
            dtrain=dtrain,
            num_boost_round=num_boost_round,
            evals=evals,
            early_stopping_rounds=25,  # More conservative early stopping
            verbose_eval=100,  # Reduced output frequency
            evals_result=evals_result
        )

        self.feature_names = dtrain.feature_names
        if self.feature_names is None:
            self.feature_names = [f'f{i}' for i in range(dtrain.num_col())]
            model.feature_names = self.feature_names

        return model, evals_result

    def cross_validate(self, params, X, y, n_splits=5, cv_models_dir='cv_models', n_repeats=10): 
        """Enhanced CV with 10 repetitions of 5-fold cross-validation and model saving"""
        Path(cv_models_dir).mkdir(parents=True, exist_ok=True)

        all_cv_results = {
            'train_rmse': [], 'val_rmse': [],
            'train_r2': [], 'val_r2': [],
            'best_iterations': [],
            'model_paths': [],
            'feature_names': [],
            'repeat': [],
            'fold': []
        }

        def process_fold(repeat_idx, fold_idx, train_idx, val_idx):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=X_train.columns.tolist())
            dval = xgb.DMatrix(X_val, label=y_val, feature_names=X_train.columns.tolist())

            model, _ = self.train_model(params, dtrain, dval)

            # Save the CV model
            model_path = os.path.join(cv_models_dir, f'cv_model_repeat_{repeat_idx}_fold_{fold_idx}.json')
            model.save_model(model_path)

            train_pred = model.predict(dtrain)
            val_pred = model.predict(dval)

            return (
                np.sqrt(mean_squared_error(y_train, train_pred)),
                np.sqrt(mean_squared_error(y_val, val_pred)),
                r2_score(y_train, train_pred),
                r2_score(y_val, val_pred),
                model.best_iteration if hasattr(model, 'best_iteration') else params.get('num_boost_round', 500),
                model_path,
                model.feature_names,
                repeat_idx,
                fold_idx
            )
        
        with ThreadPoolExecutor(max_workers=self.n_jobs) as executor:
            futures = []
            for repeat_idx in range(n_repeats):
                kf = KFold(n_splits=n_splits, shuffle=True, random_state=self.seed + repeat_idx)
                for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X)):
                    futures.append(executor.submit(
                        process_fold, repeat_idx, fold_idx, train_idx, val_idx
                    ))

            for future in futures:
                train_rmse, val_rmse, train_r2, val_r2, best_iter, model_path, feature_names, repeat_idx, fold_idx = future.result()
                all_cv_results['train_rmse'].append(train_rmse)
                all_cv_results['val_rmse'].append(val_rmse)
                all_cv_results['train_r2'].append(train_r2)
                all_cv_results['val_r2'].append(val_r2)
                all_cv_results['best_iterations'].append(best_iter)
                all_cv_results['model_paths'].append(model_path)
                all_cv_results['feature_names'].append(feature_names)
                all_cv_results['repeat'].append(repeat_idx)
                all_cv_results['fold'].append(fold_idx)
        # Create summary statistics
        cv_results_summary = {
            'train_rmse_mean': np.mean(all_cv_results['train_rmse']),
            'train_rmse_std': np.std(all_cv_results['train_rmse']),
            'val_rmse_mean': np.mean(all_cv_results['val_rmse']),
            'val_rmse_std': np.std(all_cv_results['val_rmse']),
            'train_r2_mean': np.mean(all_cv_results['train_r2']),
            'train_r2_std': np.std(all_cv_results['train_r2']),
            'val_r2_mean': np.mean(all_cv_results['val_r2']),
            'val_r2_std': np.std(all_cv_results['val_r2']),
            'best_iterations_mean': np.mean(all_cv_results['best_iterations']),
            'best_iterations_std': np.std(all_cv_results['best_iterations']),
            'detailed_results': pd.DataFrame(all_cv_results)
        }

        return cv_results_summary

    def evaluate_model(self, model, dtest):
        """Evaluate model performance on test set"""
        test_pred = model.predict(dtest)
        return {
            'rmse': np.sqrt(mean_squared_error(dtest.get_label(), test_pred)),
            'r2': r2_score(dtest.get_label(), test_pred),
            'mae': mean_absolute_error(dtest.get_label(), test_pred)
        }, test_pred

    def plot_learning_curves(self, evals_result, title_suffix):
        """Plot training and validation metrics"""
        plt.figure(figsize=(10, 5))
        epochs = len(evals_result['train']['rmse'])
        plt.plot(range(epochs), evals_result['train']['rmse'], label='Train RMSE')
        if 'valid' in evals_result:
            plt.plot(range(epochs), evals_result['valid']['rmse'], label='Validation RMSE')
        plt.legend()
        plt.xlabel('Iterations')
        plt.ylabel('RMSE')
        plt.title(f'Learning Curves - {title_suffix}')
        plt.show()

    def plot_cv_results(self, cv_results_summary):
        """Plot enhanced cross-validation results with both original and new visualization"""
        # Check if it's the new summary format or old list format
        if isinstance(cv_results_summary, dict) and 'detailed_results' in cv_results_summary:
            # New format - use the detailed results
            detailed_results = cv_results_summary['detailed_results']
            
            # Create two visualizations:
            # 1. Original visualization (simple line plot for first repeat)
            plt.figure(figsize=(15, 10))
            
            # Plot 1: Original style - first repeat only
            plt.subplot(2, 2, 1)
            first_repeat = detailed_results[detailed_results['repeat'] == 0]
            plt.plot(first_repeat['train_rmse'], 'o-', label='Train RMSE')
            plt.plot(first_repeat['val_rmse'], 'o-', label='Validation RMSE')
            plt.xlabel('Fold')
            plt.ylabel('RMSE')
            plt.title('Cross-Validation RMSE (First Repeat)')
            plt.legend()
            plt.grid(True)

            # Plot 2: Original style - first repeat R²
            plt.subplot(2, 2, 2)
            plt.plot(first_repeat['train_r2'], 'o-', label='Train R²')
            plt.plot(first_repeat['val_r2'], 'o-', label='Validation R²')
            plt.xlabel('Fold')
            plt.ylabel('R² Score')
            plt.title('Cross-Validation R² Scores (First Repeat)')
            plt.legend()
            plt.grid(True)

            # Plot 3: New visualization - RMSE distribution across all repeats
            plt.subplot(2, 2, 3)
            plt.boxplot([detailed_results['train_rmse'], detailed_results['val_rmse']], 
                       labels=['Train RMSE', 'Validation RMSE'])
            plt.ylabel('RMSE')
            plt.title('RMSE Distribution across 10x5 CV')
            plt.grid(True, alpha=0.3)
            
            # Plot 4: New visualization - R² distribution across all repeats
            plt.subplot(2, 2, 4)
            plt.boxplot([detailed_results['train_r2'], detailed_results['val_r2']], 
                       labels=['Train R²', 'Validation R²'])
            plt.ylabel('R² Score')
            plt.title('R² Distribution across 10x5 CV')
            plt.grid(True, alpha=0.3)

            plt.tight_layout()
            plt.show()
            
            # Print summary statistics
            print("\n=== 10x5 Cross-Validation Summary ===")
            print(f"Average Train RMSE: {cv_results_summary['train_rmse_mean']:.4f} (±{cv_results_summary['train_rmse_std']:.4f})")
            print(f"Average Validation RMSE: {cv_results_summary['val_rmse_mean']:.4f} (±{cv_results_summary['val_rmse_std']:.4f})")
            print(f"Average Train R²: {cv_results_summary['train_r2_mean']:.4f} (±{cv_results_summary['train_r2_std']:.4f})")
            print(f"Average Validation R²: {cv_results_summary['val_r2_mean']:.4f} (±{cv_results_summary['val_r2_std']:.4f})")
            print(f"Average Best Iterations: {cv_results_summary['best_iterations_mean']:.1f} (±{cv_results_summary['best_iterations_std']:.1f})")
            
            # Also print the format that main() expects
            print("\n=== 交叉验证结果 (旧格式兼容) ===")
            print(f"平均训练RMSE: {cv_results_summary['train_rmse_mean']:.4f} (±{cv_results_summary['train_rmse_std']:.4f})")
            print(f"平均验证RMSE: {cv_results_summary['val_rmse_mean']:.4f} (±{cv_results_summary['val_rmse_std']:.4f})")
            print(f"平均训练R²: {cv_results_summary['train_r2_mean']:.4f} (±{cv_results_summary['train_r2_std']:.4f})")
            print(f"平均验证R²: {cv_results_summary['val_r2_mean']:.4f} (±{cv_results_summary['val_r2_std']:.4f})")
            
        else:
            # Old format - keep original plotting
            plt.figure(figsize=(15, 5))

            # Plot RMSE
            plt.subplot(1, 2, 1)
            plt.plot(cv_results_summary['train_rmse'], 'o-', label='Train RMSE')
            plt.plot(cv_results_summary['val_rmse'], 'o-', label='Validation RMSE')
            plt.xlabel('Fold')
            plt.ylabel('RMSE')
            plt.title('Cross-Validation RMSE')
            plt.legend()
            plt.grid(True)

            # Plot R2
            plt.subplot(1, 2, 2)
            plt.plot(cv_results_summary['train_r2'], 'o-', label='Train R²')
            plt.plot(cv_results_summary['val_r2'], 'o-', label='Validation R²')
            plt.xlabel('Fold')
            plt.ylabel('R² Score')
            plt.title('Cross-Validation R² Scores')
            plt.legend()
            plt.grid(True)

            plt.tight_layout()
            plt.show()

    def plot_final_model_performance(self, model, dtest):
        """Plot actual vs predicted values for test set"""
        test_metrics, test_pred = self.evaluate_model(model, dtest)
        y_test = dtest.get_label()

        plt.figure(figsize=(12, 5))

        # Scatter plot of actual vs predicted
        plt.subplot(1, 2, 1)
        plt.scatter(y_test, test_pred, alpha=0.5)
        plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--r')
        plt.xlabel('Actual Values')
        plt.ylabel('Predicted Values')
        plt.title(f'Actual vs Predicted (R² = {test_metrics["r2"]:.3f})')

        # Residual plot
        plt.subplot(1, 2, 2)
        residuals = y_test - test_pred
        plt.scatter(test_pred, residuals, alpha=0.5)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.xlabel('Predicted Values')
        plt.ylabel('Residuals')
        plt.title('Residual Plot')
        plt.tight_layout()
        plt.show()

    def plot_feature_importance(self, model, importance_type='weight', top_n=30):
        """Plot feature importance without category-specific highlighting"""
        fig, ax = plt.subplots(figsize=(12, 10))
        
    
        importance_dict = model.get_score(importance_type=importance_type)
        
        if not importance_dict:
            print("No feature importance data available.")
            return None
            

        importance_df = pd.DataFrame({
            'feature': list(importance_dict.keys()),
            'importance': list(importance_dict.values())
        }).sort_values('importance', ascending=True)
        

        if len(importance_df) > top_n:
            importance_df = importance_df.tail(top_n)
        

        y_pos = np.arange(len(importance_df))
        colors = ['lightblue'] * len(importance_df)
        
        ax.barh(y_pos, importance_df['importance'], color=colors, alpha=0.8)
        ax.set_yticks(y_pos)
        ax.set_yticklabels(importance_df['feature'])
        ax.set_xlabel('Importance')
        
        title = f'Feature Importance ({importance_type})'
        ax.set_title(title)
        

        lu_type_features = importance_df[importance_df['feature'].str.contains('LUtype', na=False)]
        lu_type_importance = lu_type_features['importance'].sum() if not lu_type_features.empty else 0
        
        ax.text(0.7, 0.95, f'LUtype total: {lu_type_importance:.3f}', 
                transform=ax.transAxes, fontsize=10, bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow"))
        
        plt.tight_layout()
        plt.show()
        
        return importance_df

    def analyze_lutype_importance(self, model, X):
     
        importance_dict = model.get_score(importance_type='weight')
        
        if not importance_dict:
            print("No feature importance data available.")
            return
            

        lutype_features = {k: v for k, v in importance_dict.items() if 'LUtype' in k}
        other_features = {k: v for k, v in importance_dict.items() if 'LUtype' not in k}
        
        print(f"\n=== LUtype Feature Importance Analysis ===")
        print(f"Total LUtype-related features: {len(lutype_features)}")
        print(f"Total importance of LUtype features: {sum(lutype_features.values()):.3f}")
        print(f"Average importance of LUtype features: {np.mean(list(lutype_features.values())) if lutype_features else 0:.3f}")
        
        total_importance = sum(importance_dict.values())
        lu_type_percentage = (sum(lutype_features.values()) / total_importance * 100) if total_importance > 0 else 0
        
        print(f"\nLUtype importance percentage: {lu_type_percentage:.2f}%")
        
     
        if lutype_features:
            sorted_lutype = sorted(lutype_features.items(), key=lambda x: x[1], reverse=True)
            print(f"\nTop LUtype-related features:")
            for feature, importance in sorted_lutype[:10]:
                print(f"  {feature}: {importance:.4f}")
        else:
            print("\nNo LUtype-related features found in importance scores.")

    def plot_shap_summary(self, model, X, feature_names=None):
        """Generate SHAP summary plot if SHAP is available"""
        if SHAP_AVAILABLE:
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X)

            plt.figure(figsize=(10, 8))
            shap.summary_plot(shap_values, X, feature_names=feature_names)
            plt.tight_layout()
            plt.show()
        else:
            print("SHAP not available. Install with: pip install shap")

    def save_results(self, model, output_dir='results'):
        """Save all results to disk including feature names"""
        Path(output_dir).mkdir(exist_ok=True)

        # Save model
        model.save_model(f'{output_dir}/xgb_model.json')

        # Save feature names
        if self.feature_names is not None:
            with open(f'{output_dir}/feature_names.txt', 'w') as f:
                f.write('\n'.join(self.feature_names))

        # Save other artifacts
        joblib.dump(self.best_params, f'{output_dir}/best_params.pkl')

        if self.study:
            joblib.dump(self.study, f'{output_dir}/study.pkl')
            study_df = self.study.trials_dataframe()
            study_df.to_csv(f'{output_dir}/trials.csv', index=False)

def detect_outliers_iqr(df, columns, threshold=1.5):
    """Detect outliers using IQR method"""
    outlier_indices = []
    
    for col in columns:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
            outlier_indices.extend(outliers)
            
            if len(outliers) > 0:
                print(f"  {col}: Found {len(outliers)} outliers "
                      f"({len(outliers)/len(df)*100:.2f}%)")
    
    return list(set(outlier_indices))

def remove_outliers_robust(df, y, columns, method='iqr', threshold=2.0):
    """Robust outlier handling method"""
    print("Starting outlier detection...")
    
    if method == 'iqr':
        outlier_indices = detect_outliers_iqr(df, columns, threshold)
    elif method == 'zscore':
        # Z-score method
        z_scores = np.abs(stats.zscore(df[columns]))
        outlier_indices = np.where(z_scores > threshold)[0]
        outlier_indices = list(outlier_indices)
    else:
        return df, y, []
    
    print(f"\nTotal outliers found: {len(outlier_indices)}")
    
    if len(outlier_indices) > 0:
        df_clean = df.drop(outlier_indices)
        y_clean = y.drop(outlier_indices)
        print(f"Data size after removing outliers: {len(df_clean)} (removed {len(outlier_indices)} samples)")
        return df_clean, y_clean, outlier_indices
    
    print("No outliers found")
    return df, y, []

def check_data_leakage(X_train, X_test, y_train, y_test):
    """Check for data leakage"""
    print("\n=== Data Leakage Check ===")
    
    # Check for overlap between train and test sets
    train_indices = set(X_train.index)
    test_indices = set(X_test.index)
    overlap = train_indices.intersection(test_indices)
    
    if overlap:
        print(f"❌ Data leakage detected: {len(overlap)} overlapping samples between train and test sets")
    else:
        print("✅ No overlap between train and test sets")
    
    # Check feature distributions
    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")
    print(f"Number of features: {X_train.shape[1]}")
    
    return len(overlap) == 0

def safe_feature_engineering(X_train, X_test):
    """Safe feature engineering to avoid data leakage"""
    print("Performing safe feature engineering...")
    
    # 1. Logarithmic transformation
    columns_to_log = ['Lon', 'Lat', 'Age', 'BD', 'pH']
    for col in columns_to_log:
        if col in X_train.columns:
            X_train[col + '_log'] = np.log(X_train[col] + 1e-8)
            X_test[col + '_log'] = np.log(X_test[col] + 1e-8)
            print(f"  Created log feature: {col}_log")
    
    # 2. Binning (based on training set quantiles)
    if 'Altitude' in X_train.columns:
        # Use training set to compute bin boundaries
        alt_bins = pd.cut(X_train['Altitude'], bins=5, retbins=True)[1]
        X_train['Altitude_bins'] = pd.cut(X_train['Altitude'], bins=alt_bins, labels=False)
        X_test['Altitude_bins'] = pd.cut(X_test['Altitude'], bins=alt_bins, labels=False)
        print("  Created binned feature: Altitude_bins")
    
    return X_train, X_test

def augment_continuous_features(X, y, continuous_cols, augmentation_factor=2, noise_scale=0.005, random_state=42):
    """
    Augment continuous features with noise - with proper index resetting 
    to prevent alignment issues during training/evaluation.
    """
    np.random.seed(random_state)
    
    # Reset indices to ensure positional indexing (iloc) matches the index
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    
    n_samples = len(X)
    n_augment = int(n_samples * augmentation_factor)

    augmented_X_list = []
    augmented_y_list = []

    for _ in range(n_augment):
        idx = np.random.randint(0, n_samples)
        base_sample = X.iloc[idx].copy()
        
        # Add noise to specified continuous columns
        for col in continuous_cols:
            if col in base_sample:
                std = X[col].std()
                noise = np.random.normal(loc=0, scale=std * noise_scale)
                
                # Logical check: don't allow negative values for typically positive soil metrics
                if col in ['BD', 'Age', 'Altitude'] or 'log' in col:
                    base_sample[col] = max(1e-8, base_sample[col] + noise)
                else:
                    base_sample[col] += noise

        augmented_X_list.append(base_sample)
        augmented_y_list.append(y.iloc[idx])

    # Efficiently combine original and augmented data
    X_augmented = pd.concat([X, pd.DataFrame(augmented_X_list)], ignore_index=True)
    y_augmented = pd.concat([y, pd.Series(augmented_y_list)], ignore_index=True)

    return X_augmented, y_augmented

def enhanced_load_and_prepare_data(augment=True, augmentation_factor=2, 
                                 lu_type_importance_boost=3.0, 
                                 outlier_threshold=2.0):
    """Fixed data leakage version with outlier handling"""
    
    # Load data
    dtype_dict = {
        'Soillayer': 'int8',
        'yi': 'float32'
    }

    df_clean = pd.read_csv('F:/model/df.clean.yi.csv', dtype=dtype_dict)
    data_predictor_VIF = pd.read_csv('F:/model/data.predictor.VIF.yi.csv')

    # Select target soil layer
    target_layer = "subsoil" ###################################################################################SELECT LAYER
    df = df_clean[df_clean['Soillayer'] == (1 if target_layer == 'topsoil' else 2)].copy()

    # Select features
    predictor_columns = data_predictor_VIF['x'].tolist()
    valid_x_cols = [col for col in predictor_columns if col != "Soillayer" and col in df.columns]

    # Data cleaning
    for col in valid_x_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    X = df[valid_x_cols].astype('float32').fillna(0)
    y = df['yi'].astype('float32')

    print(f"Original data size: {len(X)}")
    
    # Outlier detection and handling
    continuous_cols = ['Lon', 'Lat', 'Age', 'BD', 'pH', 'Altitude']
    continuous_cols = [col for col in continuous_cols if col in X.columns]
    
    print("\n=== Outlier Handling ===")
    X_clean, y_clean, outliers = remove_outliers_robust(
        X, y, continuous_cols, threshold=outlier_threshold
    )

    # Data splitting
    X_train, X_test, y_train, y_test = train_test_split(
        X_clean, y_clean, test_size=0.3, random_state=42
    )

    # Safe feature engineering
    X_train, X_test = safe_feature_engineering(X_train, X_test)

    # Data leakage check
    check_data_leakage(X_train, X_test, y_train, y_test)

    # LUtype feature enhancement (only for training set)
    if 'LUtype' in X_train.columns:
        print(f"\n=== LUtype Feature Enhancement ===")
        print(f"Original LUtype distribution:")
        print(X_train['LUtype'].value_counts().sort_index())
        
        print(f"\nEnhancing LUtype importance with factor: {lu_type_importance_boost}")
        
        # Create LUtype feature enhancements
        for i in range(int(lu_type_importance_boost) - 1):
            lu_type_col_name = f'LUtype_boost_{i+1}'
            X_train[lu_type_col_name] = X_train['LUtype']
            X_test[lu_type_col_name] = X_test['LUtype']
            print(f"  Created LUtype copy: {lu_type_col_name}")
        
        # Create interaction terms between LUtype and other important features
        important_features = ['pH', 'BD', 'Age', 'Altitude']
        for feature in important_features:
            if feature in X_train.columns:
                interaction_name = f'LUtype_{feature}_interaction'
                X_train[interaction_name] = X_train['LUtype'] * X_train[feature]
                X_test[interaction_name] = X_test['LUtype'] * X_test[feature]
                print(f"  Created interaction feature: {interaction_name}")
        
        # Create LUtype squared term (non-linear effect)
        X_train['LUtype_squared'] = X_train['LUtype'] ** 2
        X_test['LUtype_squared'] = X_test['LUtype'] ** 2
        print("  Created LUtype_squared")
    
    # Data augmentation (only for training set)
    if augment:
        continuous_cols = [
            'Lon', 'Lat', 'Age', 'BD', 'pH',
            'Lon_log', 'Lat_log', 'Age_log', 'BD_log', 'pH_log',
            'Altitude'
        ]
        continuous_cols = [col for col in continuous_cols if col in X_train.columns]
        
        print(f"\n=== Data Augmentation ===")
        print(f"Training set size before augmentation: {len(X_train)}")
        X_train, y_train = augment_continuous_features(
            X=X_train,  # Only augment training set
            y=y_train,  # Only augment training set
            continuous_cols=continuous_cols,
            augmentation_factor=augmentation_factor,
            noise_scale=0.005,  
            random_state=42
        )
        print(f"Training set size after augmentation: {len(X_train)}")
    else:
        print("\nSkipping data augmentation")

    print(f"\n=== Final Dataset Statistics ===")
    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")
    print(f"Total number of features: {len(X_train.columns)}")
    
    # Statistical feature distribution
    lu_type_cols = [col for col in X_train.columns if 'LUtype' in col]
    
    print(f"\nFeature statistics:")
    print(f"LUtype-related features: {len(lu_type_cols)}")

    dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=X_train.columns.tolist())
    dtest = xgb.DMatrix(X_test, label=y_test, feature_names=X_train.columns.tolist())

    return dtrain, dtest, X, y, X_test, y_test, X_train, y_train

def main():
    """Main function with data leakage prevention"""
    outlier_threshold = 1.0  # Outlier detection threshold
    
    print("=== Starting Enhanced XGBoost Modeling (Fixed Data Leakage Version) ===")
    
    # Use fixed data processing (with outlier handling and data leakage prevention)
    dtrain, dtest, x1, y1, X_test, y_test, X_train, y_train = enhanced_load_and_prepare_data(
        augment=True,
        augmentation_factor=3,
        lu_type_importance_boost=3.0,
        outlier_threshold=outlier_threshold
    )

    tuner = XGBoostTuner(seed=42, n_jobs=17)

    # 1. Hyperparameter tuning
    print("\n=== Starting Hyperparameter Tuning ===")
    best_params = tuner.tune_hyperparameters(dtrain, n_trials=10)
    print(f"Best parameters: {best_params}")

    # 2. Enhanced Cross-validation with 10x5 folds and model saving
    print("\n=== Starting 10x5 Cross-Validation ===")
    cv_results_summary = tuner.cross_validate(best_params, X_train, y_train, n_splits=5,
                                              cv_models_dir='F:/model/results/cv_models',
                                              n_repeats=10)  # 10 repetitions of 5-fold CV
    
    print("\nSaved CV models:")
    for path in cv_results_summary['detailed_results']['model_paths'][:5]:  # Show first 5
        print(f"- {path}")
    print(f"... and {len(cv_results_summary['detailed_results']['model_paths']) - 5} more")
    
    # Plot enhanced CV results
    print("\nPlotting enhanced cross-validation results...")
    tuner.plot_cv_results(cv_results_summary)

    # 3. Train final model
    optimal_rounds = int(cv_results_summary['best_iterations_mean'])
    print(f"\nTraining final model with {optimal_rounds} rounds...")
    final_model, evals_result = tuner.train_model(best_params, dtrain, num_boost_round=optimal_rounds)

    # Plot learning curves
    print("\nPlotting learning curves...")
    tuner.plot_learning_curves(evals_result, "Final Model")

    # 4. Evaluate and plot final performance
    print("\nEvaluating final model...")
    test_metrics, _ = tuner.evaluate_model(final_model, dtest)
    print("\n=== Final Test Metrics ===")
    print(f"RMSE: {test_metrics['rmse']:.4f}")
    print(f"R²: {test_metrics['r2']:.4f}")
    print(f"MAE: {test_metrics['mae']:.4f}")

    print("\nPlotting final model performance...")
    tuner.plot_final_model_performance(final_model, dtest)

    # 5. Analyze LUtype importance
    print(f"\nAnalyzing LUtype feature importance...")
    importance_df = tuner.plot_feature_importance(final_model, top_n=35)
    
    # Detailed analysis of LUtype importance
    tuner.analyze_lutype_importance(final_model, X_test)
    
    # 6. SHAP analysis
    if SHAP_AVAILABLE:
        print("\nGenerating SHAP summary plot...")
        tuner.plot_shap_summary(final_model, X_test)
        
    # 7. Save results
    print("\nSaving results...")
    tuner.save_results(final_model, 'F:/model/results')
    
    # Save CV results summary
    cv_summary_path = 'F:/model/results/cv_summary.csv'
    cv_results_summary['detailed_results'].to_csv(cv_summary_path, index=False)
    print(f"Saved detailed CV results to: {cv_summary_path}")
    
    print(f"\n=== Model Training Summary ===")
    print("✓ Fixed data leakage issues")
    print("✓ Added outlier detection and handling")
    print("✓ Safe feature engineering (avoided data leakage)")
    print("✓ Created multiple LUtype feature copies")
    print("✓ Added LUtype interaction features with other important variables")
    print("✓ Added LUtype squared term (non-linear effect)")
    print("✓ Improved hyperparameter search space for better handling of categorical features")
    print("✓ Test set uses original data (no augmentation noise)")
    print("✓ Implemented 10 repetitions of 5-fold cross-validation for robust evaluation")
    
    print("\n=== Complete! ===")

if __name__ == "__main__":
    main()

In [ ]:
# Current climate mapping: topsoil
import os
import gc
from datetime import datetime
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import xgboost as xgb
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# ===================== Configuration =====================
MODEL_PATH = 'F:/model/results/sixth/passive_top/xgb_model.json'# replace with 'active_sub','SOC_sub','passive_sub'
FEATURE_NAMES_PATH = 'F:/model/results/sixth/passive_top/feature_names.txt'# replace with 'active_sub','SOC_sub','passive_sub'
FEATURE_CSV_PATH = 'F:/model/Mapping_select_features.csv'# replace with correspongding SOC fractions and soil layers
INPUT_TIF_FOLDER = 'F:/cleaned_tifs_no_extremes_iqr'
OUTPUT_TIF_PATH = f'F:/model/results/passive_top.tif'# replace with 'active_sub','SOC_sub','passive_sub'


N_JOBS = 4
CHUNK_SIZE = 500
# Threshold for NoData values (anything outside this range is treated as NaN)
EXTREME_THRESHOLD = 1e10 
# ======================================================

MODEL_TO_TIF_MAP = {
    'BD': 't_bd', 'pH': 't_ph', 'Sand': 't_sand', 'Silt': 't_silt',
    'Clay': 't_clay', 'TC': 't_oc', 'TN': 'TN13', 'TK': 'TK13',
    'Altitude': 'Altitude', 'Lon': 'Lon', 'Lat': 'Lat', 'Age': 'Age',
    'LUtype': 'LUtype', 'Vegetype': 'Vege_type', 'Recovmode': 'Recovery_mode'
}

TIF_TO_MODEL_MAP = {v: k for k, v in MODEL_TO_TIF_MAP.items()}
KEY_FEATURES = ['LUtype', 'Altitude', 'Lon', 'Lat', 'BD', 'pH', 'Vegetype', 'Recovmode']

def clean_extreme_values(array):
    """Replaces infinity and extreme NoData placeholders with NaN"""
    if array is None: return None
    # Handle infinities
    array = np.where(np.isinf(array), np.nan, array)
    # Handle extreme scientific notation (e.g., -3.4e+38)
    array = np.where(np.abs(array) > EXTREME_THRESHOLD, np.nan, array)
    return array

def check_data_quality(df_chunk, model_feature_names):
    issues = []
    for feat in model_feature_names:
        if feat in df_chunk.columns:
            data = df_chunk[feat].values
            nan_count = np.sum(np.isnan(data))
            if nan_count > 0:
                issues.append(f"{feat}: {nan_count} NaN values")
            
            finite_mask = np.isfinite(data)
            if np.any(finite_mask):
                finite_data = data[finite_mask]
                issues.append(f"{feat}: Range [{finite_data.min():.2f}, {finite_data.max():.2f}]")
    return issues

def load_tif_to_chunk(feat_name, tif_folder, chunk_start, chunk_end, ref_shape, coords_x, coords_y):
    tif_filename = None
    if feat_name in MODEL_TO_TIF_MAP:
        tif_base = MODEL_TO_TIF_MAP[feat_name]
        tif_filename = f"{tif_base}.tif"
    elif f"{feat_name}.tif" in os.listdir(tif_folder):
        tif_filename = f"{feat_name}.tif"
    
    if not tif_filename: return None
    tif_path = os.path.join(tif_folder, tif_filename)
    if not os.path.exists(tif_path): return None

    try:
        with rxr.open_rasterio(tif_path, masked=True) as da:
            da_squeezed = da.squeeze()
            if feat_name in ['Lon', 'Lat'] or MODEL_TO_TIF_MAP.get(feat_name) in ['Lon', 'Lat']:
                if 'Lon' in feat_name or MODEL_TO_TIF_MAP.get(feat_name) == 'Lon':
                    chunk_values = np.tile(coords_x.values, (chunk_end - chunk_start, 1))
                    model_feat_name = 'Lon'
                else:
                    lat_values = coords_y.isel(y=slice(chunk_start, chunk_end)).values.reshape(-1, 1)
                    chunk_values = np.tile(lat_values, (1, ref_shape[1]))
                    model_feat_name = 'Lat'
            else:
                chunk_values = da_squeezed.isel(y=slice(chunk_start, chunk_end)).values
                model_feat_name = TIF_TO_MODEL_MAP.get(feat_name, feat_name)

            if np.ma.is_masked(chunk_values):
                chunk_values = chunk_values.filled(np.nan)
            
            # CRITICAL: Clean the data immediately after loading
            flat_values = chunk_values.reshape(-1).astype(np.float32)
            flat_values = clean_extreme_values(flat_values)
            
            return (model_feat_name, flat_values)
    except Exception as e:
        print(f"    ❌ Error loading {tif_path}: {e}")
        return None

def create_dynamic_features(chunk_data, model_feature_names):
    if not chunk_data: return {}
    first_key = next(iter(chunk_data))
    array_shape = chunk_data[first_key].shape
    
    lu_data = chunk_data.get('LUtype', np.zeros(array_shape, dtype=np.float32))
    lu_data_filled = np.where(np.isnan(lu_data), 0, lu_data)
    
    # Example interactions
    enhancements = {
        'LUtype_boost_1': lu_data_filled,
        'LUtype_boost_2': lu_data_filled * 2,
        'LUtype_squared': lu_data_filled ** 2
    }
    
    for name, data in enhancements.items():
        if name in model_feature_names:
            chunk_data[name] = data

    interaction_pairs = [('pH', 'LUtype_pH_interaction'), ('BD', 'LUtype_BD_interaction')]
    for base, inter in interaction_pairs:
        if inter in model_feature_names:
            base_data = chunk_data.get(base, np.zeros(array_shape))
            base_filled = np.where(np.isnan(base_data), 0, base_data)
            chunk_data[inter] = lu_data_filled * base_filled

    for feat in model_feature_names:
        if feat not in chunk_data:
            chunk_data[feat] = np.zeros(array_shape, dtype=np.float32)
    return chunk_data

def load_model_feature_names():
    if os.path.exists(FEATURE_NAMES_PATH):
        with open(FEATURE_NAMES_PATH, 'r') as f:
            return [line.strip() for line in f.readlines() if line.strip()]
    xgb_model = xgb.Booster(); xgb_model.load_model(MODEL_PATH)
    return xgb_model.feature_names

def conservative_process():
    model_feature_names = load_model_feature_names()
    
    ref_tif_path = os.path.join(INPUT_TIF_FOLDER, "Altitude.tif")
    ref_da = rxr.open_rasterio(ref_tif_path, masked=True).squeeze()
    ref_shape = ref_da.shape
    coords_x, coords_y = ref_da['x'], ref_da['y']
    
    xgb_model = xgb.Booster(); xgb_model.load_model(MODEL_PATH)
    result_array = np.full(ref_shape, np.nan, dtype=np.float32)
    
    for chunk_start in range(0, ref_shape[0], CHUNK_SIZE):
        chunk_end = min(chunk_start + CHUNK_SIZE, ref_shape[0])
        total_pixels = (chunk_end - chunk_start) * ref_shape[1]
        print(f"\n🚀 Processing Rows {chunk_start}-{chunk_end}")

        chunk_data = {}
        for feat in set(KEY_FEATURES + model_feature_names):
            loaded = load_tif_to_chunk(feat, INPUT_TIF_FOLDER, chunk_start, chunk_end, ref_shape, coords_x, coords_y)
            if loaded: chunk_data[loaded[0]] = loaded[1]

        chunk_data = create_dynamic_features(chunk_data, model_feature_names)
        
        # Build mask based on key features being valid (not NaN)
        complete_mask = np.ones(total_pixels, dtype=bool)
        for feat in KEY_FEATURES:
            if feat in chunk_data:
                
                complete_mask &= (~np.isnan(chunk_data[feat]))
        
        indices = np.where(complete_mask)[0]
        if len(indices) == 0: continue

        # Prepare DataFrame
        df_chunk = pd.DataFrame({f: chunk_data[f] for f in model_feature_names})
        valid_df = df_chunk.iloc[indices].fillna(0) # XGBoost prefers 0 or a specific value for missing
        
        # Predict
        dtest = xgb.DMatrix(valid_df.values, feature_names=model_feature_names)
        preds = xgb_model.predict(dtest)
        
        # Remap to 2D
        global_indices = (np.arange(total_pixels) + (chunk_start * ref_shape[1]))[indices]
        result_array[global_indices // ref_shape[1], global_indices % ref_shape[1]] = preds
        
        gc.collect()

    # Save
    out_da = xr.DataArray(result_array, coords={"y": coords_y, "x": coords_x}, dims=("y", "x"))
    out_da.rio.to_raster(OUTPUT_TIF_PATH, compress="LZW", nodata=np.nan)
    print(f"✅ Success! Saved to {OUTPUT_TIF_PATH}")

if __name__ == "__main__":
    conservative_process()

In [ ]:
# Current climate mapping: subsoil
import os
import gc
from datetime import datetime
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import xgboost as xgb
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# ===================== Configuration =====================
MODEL_PATH = 'F:/model/results/xgb_model.json'# replace with 'active_sub','SOC_sub','passive_sub'
FEATURE_NAMES_PATH = 'F:/model/results/feature_names.txt'# replace with 'active_sub','SOC_sub','passive_sub'
FEATURE_CSV_PATH = 'F:/model/Mapping_select_features.csv'# replace with correspongding SOC fractions and soil layers
INPUT_TIF_FOLDER = 'F:/cleaned_tifs_no_extremes_iqr'
OUTPUT_TIF_PATH = f'F:/model/results/passive_sub.tif'# replace with 'active_sub','SOC_sub','passive_sub'


N_JOBS = 4
CHUNK_SIZE = 500
# Threshold for NoData values (anything outside this range is treated as NaN)
EXTREME_THRESHOLD = 1e10 
# ======================================================

MODEL_TO_TIF_MAP = {
    'BD': 's_bd', 'pH': 's_ph', 'Sand': 's_sand', 'Silt': 's_silt',
    'Clay': 's_clay', 'TC': 's_oc', 'TN': 'TN46', 'TK': 'TK46',
    'Altitude': 'Altitude', 'Lon': 'Lon', 'Lat': 'Lat', 'Age': 'Age',
    'LUtype': 'LUtype', 'Vegetype': 'Vege_type', 'Recovmode': 'Recovery_mode'
}

TIF_TO_MODEL_MAP = {v: k for k, v in MODEL_TO_TIF_MAP.items()}
KEY_FEATURES = ['LUtype', 'Altitude', 'Lon', 'Lat', 'BD', 'pH', 'Vegetype', 'Recovmode']

def clean_extreme_values(array):
    """Replaces infinity and extreme NoData placeholders with NaN"""
    if array is None: return None
    # Handle infinities
    array = np.where(np.isinf(array), np.nan, array)
    # Handle extreme scientific notation (e.g., -3.4e+38)
    array = np.where(np.abs(array) > EXTREME_THRESHOLD, np.nan, array)
    return array

def check_data_quality(df_chunk, model_feature_names):
    issues = []
    for feat in model_feature_names:
        if feat in df_chunk.columns:
            data = df_chunk[feat].values
            nan_count = np.sum(np.isnan(data))
            if nan_count > 0:
                issues.append(f"{feat}: {nan_count} NaN values")
            
            finite_mask = np.isfinite(data)
            if np.any(finite_mask):
                finite_data = data[finite_mask]
                issues.append(f"{feat}: Range [{finite_data.min():.2f}, {finite_data.max():.2f}]")
    return issues

def load_tif_to_chunk(feat_name, tif_folder, chunk_start, chunk_end, ref_shape, coords_x, coords_y):
    tif_filename = None
    if feat_name in MODEL_TO_TIF_MAP:
        tif_base = MODEL_TO_TIF_MAP[feat_name]
        tif_filename = f"{tif_base}.tif"
    elif f"{feat_name}.tif" in os.listdir(tif_folder):
        tif_filename = f"{feat_name}.tif"
    
    if not tif_filename: return None
    tif_path = os.path.join(tif_folder, tif_filename)
    if not os.path.exists(tif_path): return None

    try:
        with rxr.open_rasterio(tif_path, masked=True) as da:
            da_squeezed = da.squeeze()
            if feat_name in ['Lon', 'Lat'] or MODEL_TO_TIF_MAP.get(feat_name) in ['Lon', 'Lat']:
                if 'Lon' in feat_name or MODEL_TO_TIF_MAP.get(feat_name) == 'Lon':
                    chunk_values = np.tile(coords_x.values, (chunk_end - chunk_start, 1))
                    model_feat_name = 'Lon'
                else:
                    lat_values = coords_y.isel(y=slice(chunk_start, chunk_end)).values.reshape(-1, 1)
                    chunk_values = np.tile(lat_values, (1, ref_shape[1]))
                    model_feat_name = 'Lat'
            else:
                chunk_values = da_squeezed.isel(y=slice(chunk_start, chunk_end)).values
                model_feat_name = TIF_TO_MODEL_MAP.get(feat_name, feat_name)

            if np.ma.is_masked(chunk_values):
                chunk_values = chunk_values.filled(np.nan)
            
            # CRITICAL: Clean the data immediately after loading
            flat_values = chunk_values.reshape(-1).astype(np.float32)
            flat_values = clean_extreme_values(flat_values)
            
            return (model_feat_name, flat_values)
    except Exception as e:
        print(f"    ❌ Error loading {tif_path}: {e}")
        return None

def create_dynamic_features(chunk_data, model_feature_names):
    if not chunk_data: return {}
    first_key = next(iter(chunk_data))
    array_shape = chunk_data[first_key].shape
    
    lu_data = chunk_data.get('LUtype', np.zeros(array_shape, dtype=np.float32))
    lu_data_filled = np.where(np.isnan(lu_data), 0, lu_data)
    
    # Example interactions
    enhancements = {
        'LUtype_boost_1': lu_data_filled,
        'LUtype_boost_2': lu_data_filled * 2,
        'LUtype_squared': lu_data_filled ** 2
    }
    
    for name, data in enhancements.items():
        if name in model_feature_names:
            chunk_data[name] = data

    interaction_pairs = [('pH', 'LUtype_pH_interaction'), ('BD', 'LUtype_BD_interaction')]
    for base, inter in interaction_pairs:
        if inter in model_feature_names:
            base_data = chunk_data.get(base, np.zeros(array_shape))
            base_filled = np.where(np.isnan(base_data), 0, base_data)
            chunk_data[inter] = lu_data_filled * base_filled

    for feat in model_feature_names:
        if feat not in chunk_data:
            chunk_data[feat] = np.zeros(array_shape, dtype=np.float32)
    return chunk_data

def load_model_feature_names():
    if os.path.exists(FEATURE_NAMES_PATH):
        with open(FEATURE_NAMES_PATH, 'r') as f:
            return [line.strip() for line in f.readlines() if line.strip()]
    xgb_model = xgb.Booster(); xgb_model.load_model(MODEL_PATH)
    return xgb_model.feature_names

def conservative_process():
    model_feature_names = load_model_feature_names()
    
    ref_tif_path = os.path.join(INPUT_TIF_FOLDER, "Altitude.tif")
    ref_da = rxr.open_rasterio(ref_tif_path, masked=True).squeeze()
    ref_shape = ref_da.shape
    coords_x, coords_y = ref_da['x'], ref_da['y']
    
    xgb_model = xgb.Booster(); xgb_model.load_model(MODEL_PATH)
    result_array = np.full(ref_shape, np.nan, dtype=np.float32)
    
    for chunk_start in range(0, ref_shape[0], CHUNK_SIZE):
        chunk_end = min(chunk_start + CHUNK_SIZE, ref_shape[0])
        total_pixels = (chunk_end - chunk_start) * ref_shape[1]
        print(f"\n🚀 Processing Rows {chunk_start}-{chunk_end}")

        chunk_data = {}
        for feat in set(KEY_FEATURES + model_feature_names):
            loaded = load_tif_to_chunk(feat, INPUT_TIF_FOLDER, chunk_start, chunk_end, ref_shape, coords_x, coords_y)
            if loaded: chunk_data[loaded[0]] = loaded[1]

        chunk_data = create_dynamic_features(chunk_data, model_feature_names)
        
        # Build mask based on key features being valid (not NaN)
        complete_mask = np.ones(total_pixels, dtype=bool)
        for feat in KEY_FEATURES:
            if feat in chunk_data:
                complete_mask &= (~np.isnan(chunk_data[feat]))
        
        indices = np.where(complete_mask)[0]
        if len(indices) == 0: continue

        # Prepare DataFrame
        df_chunk = pd.DataFrame({f: chunk_data[f] for f in model_feature_names})
        valid_df = df_chunk.iloc[indices].fillna(0) # XGBoost prefers 0 or a specific value for missing
        
        # Predict
        dtest = xgb.DMatrix(valid_df.values, feature_names=model_feature_names)
        preds = xgb_model.predict(dtest)
        
        # Remap to 2D
        global_indices = (np.arange(total_pixels) + (chunk_start * ref_shape[1]))[indices]
        result_array[global_indices // ref_shape[1], global_indices % ref_shape[1]] = preds
        
        gc.collect()

    # Save
    out_da = xr.DataArray(result_array, coords={"y": coords_y, "x": coords_x}, dims=("y", "x"))
    out_da.rio.to_raster(OUTPUT_TIF_PATH, compress="LZW", nodata=np.nan)
    print(f"✅ Success! Saved to {OUTPUT_TIF_PATH}")

if __name__ == "__main__":
    conservative_process()

In [ ]:
# future climate mapping: topsoil
import os
import gc
from datetime import datetime
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import xgboost as xgb
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# ===================== Configuration =====================
MODEL_PATH = 'F:/model/results/sixth/passive_top/xgb_model.json'# replace with 'active_top','SOC_top','passive_top'
FEATURE_NAMES_PATH = 'F:/model/results/sixth/passive_top/feature_names.txt'# replace with 'active_top','SOC_top','passive_top'
FEATURE_CSV_PATH = 'F:/model/Mapping_select_features.csv'# replace with 'active_top','SOC_top','passive_top'
INPUT_TIF_FOLDER = 'F:/cleaned_tifs_no_extremes_iqr'
OUTPUT_TIF_PATH = f'F:/model/results/passive_top_future.tif'# replace with 'active_top','SOC_top','passive_top'


N_JOBS = 4
CHUNK_SIZE = 500
# Threshold for NoData values (anything outside this range is treated as NaN)
EXTREME_THRESHOLD = 1e10 
# ======================================================

MODEL_TO_TIF_MAP = {
    'BD': 't_bd', 'pH': 't_ph', 'Sand': 't_sand', 'Silt': 't_silt',
    'Clay': 't_clay', 'TC': 't_oc', 'TN': 'TN13', 'TK': 'TK13',
    'Altitude': 'Altitude', 'Lon': 'Lon', 'Lat': 'Lat', 
    'LUtype': 'LUtype', 'Vegetype': 'Vege_type', 'Recovmode': 'Recovery_mode',
        'Age_plus100': 'Age',
        'wc_BIO1_MAT': 'MAT',
        'wc_BIO2': 'bio2',
        'wc_BIO3': 'bio3',
        'wc_BIO4': 'bio4',
        'wc_BIO5': 'bio5',
        'wc_BIO6': 'bio6',
        'wc_BIO7': 'bio7',
        'wc_BIO8': 'bio8',
        'wc_BIO9': 'bio9',
        'wc_BIO10': 'bio10',
        'wc_BIO11': 'bio11',
        'wc_BIO12_MAP': 'MAP',
        'wc_BIO13': 'bio13',
        'wc_BIO14': 'bio14',
        'wc_BIO15': 'bio15',
        'wc_BIO16': 'bio16',
        'wc_BIO17': 'bio17',
        'wc_BIO18': 'bio18',
        'wc_BIO19': 'bio19'
}

TIF_TO_MODEL_MAP = {v: k for k, v in MODEL_TO_TIF_MAP.items()}
KEY_FEATURES = ['LUtype', 'Altitude', 'Lon', 'Lat', 'BD', 'pH', 'Vegetype', 'Recovmode']

def clean_extreme_values(array):
    """Replaces infinity and extreme NoData placeholders with NaN"""
    if array is None: return None
    # Handle infinities
    array = np.where(np.isinf(array), np.nan, array)
    # Handle extreme scientific notation (e.g., -3.4e+38)
    array = np.where(np.abs(array) > EXTREME_THRESHOLD, np.nan, array)
    return array

def check_data_quality(df_chunk, model_feature_names):
    issues = []
    for feat in model_feature_names:
        if feat in df_chunk.columns:
            data = df_chunk[feat].values
            nan_count = np.sum(np.isnan(data))
            if nan_count > 0:
                issues.append(f"{feat}: {nan_count} NaN values")
            
            finite_mask = np.isfinite(data)
            if np.any(finite_mask):
                finite_data = data[finite_mask]
                issues.append(f"{feat}: Range [{finite_data.min():.2f}, {finite_data.max():.2f}]")
    return issues

def load_tif_to_chunk(feat_name, tif_folder, chunk_start, chunk_end, ref_shape, coords_x, coords_y):
    tif_filename = None
    if feat_name in MODEL_TO_TIF_MAP:
        tif_base = MODEL_TO_TIF_MAP[feat_name]
        tif_filename = f"{tif_base}.tif"
    elif f"{feat_name}.tif" in os.listdir(tif_folder):
        tif_filename = f"{feat_name}.tif"
    
    if not tif_filename: return None
    tif_path = os.path.join(tif_folder, tif_filename)
    if not os.path.exists(tif_path): return None

    try:
        with rxr.open_rasterio(tif_path, masked=True) as da:
            da_squeezed = da.squeeze()
            if feat_name in ['Lon', 'Lat'] or MODEL_TO_TIF_MAP.get(feat_name) in ['Lon', 'Lat']:
                if 'Lon' in feat_name or MODEL_TO_TIF_MAP.get(feat_name) == 'Lon':
                    chunk_values = np.tile(coords_x.values, (chunk_end - chunk_start, 1))
                    model_feat_name = 'Lon'
                else:
                    lat_values = coords_y.isel(y=slice(chunk_start, chunk_end)).values.reshape(-1, 1)
                    chunk_values = np.tile(lat_values, (1, ref_shape[1]))
                    model_feat_name = 'Lat'
            else:
                chunk_values = da_squeezed.isel(y=slice(chunk_start, chunk_end)).values
                model_feat_name = TIF_TO_MODEL_MAP.get(feat_name, feat_name)

            if np.ma.is_masked(chunk_values):
                chunk_values = chunk_values.filled(np.nan)
            
            # CRITICAL: Clean the data immediately after loading
            flat_values = chunk_values.reshape(-1).astype(np.float32)
            flat_values = clean_extreme_values(flat_values)
            
            return (model_feat_name, flat_values)
    except Exception as e:
        print(f"    ❌ Error loading {tif_path}: {e}")
        return None

def create_dynamic_features(chunk_data, model_feature_names):
    if not chunk_data: return {}
    first_key = next(iter(chunk_data))
    array_shape = chunk_data[first_key].shape
    
    lu_data = chunk_data.get('LUtype', np.zeros(array_shape, dtype=np.float32))
    lu_data_filled = np.where(np.isnan(lu_data), 0, lu_data)
    
    # Example interactions
    enhancements = {
        'LUtype_boost_1': lu_data_filled,
        'LUtype_boost_2': lu_data_filled * 2,
        'LUtype_squared': lu_data_filled ** 2
    }
    
    for name, data in enhancements.items():
        if name in model_feature_names:
            chunk_data[name] = data

    interaction_pairs = [('pH', 'LUtype_pH_interaction'), ('BD', 'LUtype_BD_interaction')]
    for base, inter in interaction_pairs:
        if inter in model_feature_names:
            base_data = chunk_data.get(base, np.zeros(array_shape))
            base_filled = np.where(np.isnan(base_data), 0, base_data)
            chunk_data[inter] = lu_data_filled * base_filled

    for feat in model_feature_names:
        if feat not in chunk_data:
            chunk_data[feat] = np.zeros(array_shape, dtype=np.float32)
    return chunk_data

def load_model_feature_names():
    if os.path.exists(FEATURE_NAMES_PATH):
        with open(FEATURE_NAMES_PATH, 'r') as f:
            return [line.strip() for line in f.readlines() if line.strip()]
    xgb_model = xgb.Booster(); xgb_model.load_model(MODEL_PATH)
    return xgb_model.feature_names

def conservative_process():
    model_feature_names = load_model_feature_names()
    
    ref_tif_path = os.path.join(INPUT_TIF_FOLDER, "Altitude.tif")
    ref_da = rxr.open_rasterio(ref_tif_path, masked=True).squeeze()
    ref_shape = ref_da.shape
    coords_x, coords_y = ref_da['x'], ref_da['y']
    
    xgb_model = xgb.Booster(); xgb_model.load_model(MODEL_PATH)
    result_array = np.full(ref_shape, np.nan, dtype=np.float32)
    
    for chunk_start in range(0, ref_shape[0], CHUNK_SIZE):
        chunk_end = min(chunk_start + CHUNK_SIZE, ref_shape[0])
        total_pixels = (chunk_end - chunk_start) * ref_shape[1]
        print(f"\n🚀 Processing Rows {chunk_start}-{chunk_end}")

        chunk_data = {}
        for feat in set(KEY_FEATURES + model_feature_names):
            loaded = load_tif_to_chunk(feat, INPUT_TIF_FOLDER, chunk_start, chunk_end, ref_shape, coords_x, coords_y)
            if loaded: chunk_data[loaded[0]] = loaded[1]

        chunk_data = create_dynamic_features(chunk_data, model_feature_names)
        
        # Build mask based on key features being valid (not NaN)
        complete_mask = np.ones(total_pixels, dtype=bool)
        for feat in KEY_FEATURES:
            if feat in chunk_data:
                
                complete_mask &= (~np.isnan(chunk_data[feat]))
        
        indices = np.where(complete_mask)[0]
        if len(indices) == 0: continue

        # Prepare DataFrame
        df_chunk = pd.DataFrame({f: chunk_data[f] for f in model_feature_names})
        valid_df = df_chunk.iloc[indices].fillna(0) # XGBoost prefers 0 or a specific value for missing
        
        # Predict
        dtest = xgb.DMatrix(valid_df.values, feature_names=model_feature_names)
        preds = xgb_model.predict(dtest)
        
        # Remap to 2D
        global_indices = (np.arange(total_pixels) + (chunk_start * ref_shape[1]))[indices]
        result_array[global_indices // ref_shape[1], global_indices % ref_shape[1]] = preds
        
        gc.collect()

    # Save
    out_da = xr.DataArray(result_array, coords={"y": coords_y, "x": coords_x}, dims=("y", "x"))
    out_da.rio.to_raster(OUTPUT_TIF_PATH, compress="LZW", nodata=np.nan)
    print(f"✅ Success! Saved to {OUTPUT_TIF_PATH}")

if __name__ == "__main__":
    conservative_process()

In [ ]:
# future climate mapping: subsoil
import os
import gc
from datetime import datetime
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import xgboost as xgb
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# ===================== Configuration =====================
MODEL_PATH = 'F:/model/results/sixth/active_sub/xgb_model.json'# replace with 'active_sub','SOC_sub','passive_sub'
FEATURE_NAMES_PATH = 'F:/model/results/sixth/active_sub/feature_names.txt'# replace with 'active_sub','SOC_sub','passive_sub'
FEATURE_CSV_PATH = 'F:/model/Mapping_select_features.csv'
INPUT_TIF_FOLDER = 'F:/cleaned_tifs_no_extremes_iqr'
OUTPUT_TIF_PATH = f'F:/model/results/active_sub_future.tif'# replace with 'active_sub','SOC_sub','passive_sub'


N_JOBS = 4
CHUNK_SIZE = 500
# Threshold for NoData values (anything outside this range is treated as NaN)
EXTREME_THRESHOLD = 1e10 
# ======================================================

MODEL_TO_TIF_MAP = {
    'BD': 's_bd', 'pH': 's_ph', 'Sand': 's_sand', 'Silt': 's_silt',
    'Clay': 's_clay', 'TC': 's_oc', 'TN': 'TN46', 'TK': 'TK46',
    'Altitude': 'Altitude', 'Lon': 'Lon', 'Lat': 'Lat', 
    'LUtype': 'LUtype', 'Vegetype': 'Vege_type', 'Recovmode': 'Recovery_mode',
        'Age_plus100': 'Age',
        'wc_BIO1_MAT': 'MAT',
        'wc_BIO2': 'bio2',
        'wc_BIO3': 'bio3',
        'wc_BIO4': 'bio4',
        'wc_BIO5': 'bio5',
        'wc_BIO6': 'bio6',
        'wc_BIO7': 'bio7',
        'wc_BIO8': 'bio8',
        'wc_BIO9': 'bio9',
        'wc_BIO10': 'bio10',
        'wc_BIO11': 'bio11',
        'wc_BIO12_MAP': 'MAP',
        'wc_BIO13': 'bio13',
        'wc_BIO14': 'bio14',
        'wc_BIO15': 'bio15',
        'wc_BIO16': 'bio16',
        'wc_BIO17': 'bio17',
        'wc_BIO18': 'bio18',
        'wc_BIO19': 'bio19'

}

TIF_TO_MODEL_MAP = {v: k for k, v in MODEL_TO_TIF_MAP.items()}
KEY_FEATURES = ['LUtype', 'Altitude', 'Lon', 'Lat', 'BD', 'pH', 'Vegetype', 'Recovmode']

def clean_extreme_values(array):
    """Replaces infinity and extreme NoData placeholders with NaN"""
    if array is None: return None
    # Handle infinities
    array = np.where(np.isinf(array), np.nan, array)
    # Handle extreme scientific notation (e.g., -3.4e+38)
    array = np.where(np.abs(array) > EXTREME_THRESHOLD, np.nan, array)
    return array

def check_data_quality(df_chunk, model_feature_names):
    issues = []
    for feat in model_feature_names:
        if feat in df_chunk.columns:
            data = df_chunk[feat].values
            nan_count = np.sum(np.isnan(data))
            if nan_count > 0:
                issues.append(f"{feat}: {nan_count} NaN values")
            
            finite_mask = np.isfinite(data)
            if np.any(finite_mask):
                finite_data = data[finite_mask]
                issues.append(f"{feat}: Range [{finite_data.min():.2f}, {finite_data.max():.2f}]")
    return issues

def load_tif_to_chunk(feat_name, tif_folder, chunk_start, chunk_end, ref_shape, coords_x, coords_y):
    tif_filename = None
    if feat_name in MODEL_TO_TIF_MAP:
        tif_base = MODEL_TO_TIF_MAP[feat_name]
        tif_filename = f"{tif_base}.tif"
    elif f"{feat_name}.tif" in os.listdir(tif_folder):
        tif_filename = f"{feat_name}.tif"
    
    if not tif_filename: return None
    tif_path = os.path.join(tif_folder, tif_filename)
    if not os.path.exists(tif_path): return None

    try:
        with rxr.open_rasterio(tif_path, masked=True) as da:
            da_squeezed = da.squeeze()
            if feat_name in ['Lon', 'Lat'] or MODEL_TO_TIF_MAP.get(feat_name) in ['Lon', 'Lat']:
                if 'Lon' in feat_name or MODEL_TO_TIF_MAP.get(feat_name) == 'Lon':
                    chunk_values = np.tile(coords_x.values, (chunk_end - chunk_start, 1))
                    model_feat_name = 'Lon'
                else:
                    lat_values = coords_y.isel(y=slice(chunk_start, chunk_end)).values.reshape(-1, 1)
                    chunk_values = np.tile(lat_values, (1, ref_shape[1]))
                    model_feat_name = 'Lat'
            else:
                chunk_values = da_squeezed.isel(y=slice(chunk_start, chunk_end)).values
                model_feat_name = TIF_TO_MODEL_MAP.get(feat_name, feat_name)

            if np.ma.is_masked(chunk_values):
                chunk_values = chunk_values.filled(np.nan)
            
            # CRITICAL: Clean the data immediately after loading
            flat_values = chunk_values.reshape(-1).astype(np.float32)
            flat_values = clean_extreme_values(flat_values)
            
            return (model_feat_name, flat_values)
    except Exception as e:
        print(f"    ❌ Error loading {tif_path}: {e}")
        return None

def create_dynamic_features(chunk_data, model_feature_names):
    if not chunk_data: return {}
    first_key = next(iter(chunk_data))
    array_shape = chunk_data[first_key].shape
    
    lu_data = chunk_data.get('LUtype', np.zeros(array_shape, dtype=np.float32))
    lu_data_filled = np.where(np.isnan(lu_data), 0, lu_data)
    
    # Example interactions
    enhancements = {
        'LUtype_boost_1': lu_data_filled,
        'LUtype_boost_2': lu_data_filled * 2,
        'LUtype_squared': lu_data_filled ** 2
    }
    
    for name, data in enhancements.items():
        if name in model_feature_names:
            chunk_data[name] = data

    interaction_pairs = [('pH', 'LUtype_pH_interaction'), ('BD', 'LUtype_BD_interaction')]
    for base, inter in interaction_pairs:
        if inter in model_feature_names:
            base_data = chunk_data.get(base, np.zeros(array_shape))
            base_filled = np.where(np.isnan(base_data), 0, base_data)
            chunk_data[inter] = lu_data_filled * base_filled

    for feat in model_feature_names:
        if feat not in chunk_data:
            chunk_data[feat] = np.zeros(array_shape, dtype=np.float32)
    return chunk_data

def load_model_feature_names():
    if os.path.exists(FEATURE_NAMES_PATH):
        with open(FEATURE_NAMES_PATH, 'r') as f:
            return [line.strip() for line in f.readlines() if line.strip()]
    xgb_model = xgb.Booster(); xgb_model.load_model(MODEL_PATH)
    return xgb_model.feature_names

def conservative_process():
    model_feature_names = load_model_feature_names()
    
    ref_tif_path = os.path.join(INPUT_TIF_FOLDER, "Altitude.tif")
    ref_da = rxr.open_rasterio(ref_tif_path, masked=True).squeeze()
    ref_shape = ref_da.shape
    coords_x, coords_y = ref_da['x'], ref_da['y']
    
    xgb_model = xgb.Booster(); xgb_model.load_model(MODEL_PATH)
    result_array = np.full(ref_shape, np.nan, dtype=np.float32)
    
    for chunk_start in range(0, ref_shape[0], CHUNK_SIZE):
        chunk_end = min(chunk_start + CHUNK_SIZE, ref_shape[0])
        total_pixels = (chunk_end - chunk_start) * ref_shape[1]
        print(f"\n🚀 Processing Rows {chunk_start}-{chunk_end}")

        chunk_data = {}
        for feat in set(KEY_FEATURES + model_feature_names):
            loaded = load_tif_to_chunk(feat, INPUT_TIF_FOLDER, chunk_start, chunk_end, ref_shape, coords_x, coords_y)
            if loaded: chunk_data[loaded[0]] = loaded[1]

        chunk_data = create_dynamic_features(chunk_data, model_feature_names)
        
        # Build mask based on key features being valid (not NaN)
        complete_mask = np.ones(total_pixels, dtype=bool)
        for feat in KEY_FEATURES:
            if feat in chunk_data:
                complete_mask &= (~np.isnan(chunk_data[feat]))
        
        indices = np.where(complete_mask)[0]
        if len(indices) == 0: continue

        # Prepare DataFrame
        df_chunk = pd.DataFrame({f: chunk_data[f] for f in model_feature_names})
        valid_df = df_chunk.iloc[indices].fillna(0) # XGBoost prefers 0 or a specific value for missing
        
        # Predict
        dtest = xgb.DMatrix(valid_df.values, feature_names=model_feature_names)
        preds = xgb_model.predict(dtest)
        
        # Remap to 2D
        global_indices = (np.arange(total_pixels) + (chunk_start * ref_shape[1]))[indices]
        result_array[global_indices // ref_shape[1], global_indices % ref_shape[1]] = preds
        
        gc.collect()

    # Save
    out_da = xr.DataArray(result_array, coords={"y": coords_y, "x": coords_x}, dims=("y", "x"))
    out_da.rio.to_raster(OUTPUT_TIF_PATH, compress="LZW", nodata=np.nan)
    print(f"✅ Success! Saved to {OUTPUT_TIF_PATH}")

if __name__ == "__main__":
    conservative_process()